In [ ]:
# ===============================================================
# 0) Imports & Global Configuration
# ===============================================================
# Bu blok proje boyunca kullanılacak tüm kütüphaneleri ve genel ayarları içerir.
# This block contains all required imports and global configuration settings.

import os
import json
import warnings
import itertools
from pathlib import Path
import time

# Sayısal işlemler ve veri yapıları
# Numerical operations and data structures
import numpy as np
import pandas as pd

# Görselleştirme kütüphaneleri
# Visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.image as mpimg  # 3F için gerekli / needed for 3F


# İstatistiksel testler: Friedman, Wilcoxon, Ki-kare, çoklu karşılaştırma düzeltmeleri
# Statistical tests: Friedman, Wilcoxon, Chi-square, multiple-comparison corrections
import scipy.stats as st
from scipy.stats import friedmanchisquare, wilcoxon, chi2_contingency
from statsmodels.stats.multitest import multipletests

# Hiperparametre optimizasyonu için Optuna
# Optuna for hyperparameter optimization
import optuna
from joblib import dump

# Veri ön işleme araçları
# Preprocessing tools
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.impute import SimpleImputer

# Veri bölme ve performans metriği fonksiyonları
# Data splitting and model evaluation metrics
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import (
    f1_score, roc_auc_score, average_precision_score,
    confusion_matrix, roc_curve, precision_recall_curve
)

# K-best feature selection
# K-best feature selection
from sklearn.feature_selection import SelectKBest, f_classif

# Kullanılacak makine öğrenmesi modelleri
# Machine learning classifiers to be trained
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier  # Bagging için ağaç tabanlı temel model
from xgboost import XGBClassifier


# ===============================================================
#  GLOBAL CONSTANTS (E-COMMERCE CHURN PROJECT)
# ===============================================================

# Deneylerin tekrarlanabilir olması için sabit random seed
# Fixed random seed for reproducibility
RANDOM_BASE = 42

# Modellerin kaç bağımsız tekrar ile eğitileceği
# Number of outer-loop repetitions for each model
N_REPS = 10

# Eğitim / doğrulama / test oranları 
# Train / validation / test split ratios 
# 65/15/20 global split -> inner (train,val) over trainval(80%)
TEST_FRACTION = 0.20
VAL_FRACTION_GLOBAL = 0.15

# Train/val oranlarını train+val (80%) altına indirge
# Convert global val ratio into proportions within the 80% train+val split
val_in_trainval   = VAL_FRACTION_GLOBAL / (1.0 - TEST_FRACTION)   # 0.1875
train_in_trainval = 1.0 - val_in_trainval                         # 0.8125

# Özellik seçimi (K-Best) için sınırlar
# Bounds for feature selection (K-Best)
# Not: Bu projede toplam etkin özellik sayısı 18 olduğu için K_MAX = 18 seçildi.
# Note: In this project we have 18 effective features, so K_MAX = 18 is used.
K_MIN = 3
K_MAX = 18  

# Optuna’nın optimize edeceği metrik (ROC-AUC churn analizi için idealdir)
# Metric to optimize with Optuna (ROC-AUC is ideal for churn analysis)
# Burada dengesiz churn problemi için AUC tercih edildi.
# here AUC is preferred for imbalanced churn.
# Optuna target can be one of: "ap" (PR-AUC), "f1", or "auc"
OPTIMIZE_FOR = "auc"

# Her model için Optuna deneme sayısı ( standart: 150)
# Number of Optuna trials per model ( standard: 150)
N_TRIALS_BY_MODEL = {
    "Random Forest": 150,
    "XGBoost":       150,
    "KNN":           150,
    "Bagging":       150,
}

# ===============================================================
#  UPDATED PATHS FOR NEW DIRECTORY
# ===============================================================

# Root folder
# Tüm çıktıların kaydedileceği ana klasör
# Main directory where all outputs will be saved
BASE_OUT_DIR = Path(r"C:\Users\ulku\Desktop\ecommerce\featselect_optuna_final")

# Hangi metriğin optimize edildiğini belirtmek için etiket
# Tag indicating which metric is optimized
RUN_TAG = f"OBJ_{OPTIMIZE_FOR}"

# Alt klasörlerin otomatik oluşturulması
# Creating output subdirectories
MODEL_DIR    = BASE_OUT_DIR / f"models_{RUN_TAG}"
OUT_DIR      = BASE_OUT_DIR / f"outputs_{RUN_TAG}"
EXPL_DIR     = BASE_OUT_DIR / f"explanations_plots_{RUN_TAG}"
LIME_DIR     = EXPL_DIR / "lime_html"
STATS_PLOTS  = BASE_OUT_DIR / f"stats_plots_{RUN_TAG}"
EDA_DIR      = BASE_OUT_DIR / "eda_outputs"
EDA_DIR.mkdir(parents=True, exist_ok=True)

# Klasörleri oluştur
# Create directories if they do not exist
for d in [BASE_OUT_DIR, MODEL_DIR, OUT_DIR, EXPL_DIR, LIME_DIR, STATS_PLOTS, EDA_DIR]:
    d.mkdir(parents=True, exist_ok=True)

# ===============================================================
#  DATA PATH (EXCEL FILE)
# ===============================================================

# Veri setinin bulunduğu Excel dosyası
# Path to the Excel dataset file
DATA_XLS_PATH = r"C:\Users\ulku\Desktop\ecommerce\featselect_optuna_final\ecommerceChurnDataset.xlsx"

# IMPORTANT:
# Bu projede hedef değişken (label) 'Churn' isimli sütundur.
# In this project, the target variable (label) is the column named 'Churn'.
# Blok 1'de TARGET_COL = "Churn" olarak açıkça tanımlanacaktır.
# In Block 1, we will explicitly set TARGET_COL = "Churn".


# ===============================================================
#  GLOBAL FLAGS
# ===============================================================

# En iyi modelleri diske kaydetmek için
# Whether to save best models to disk
SAVE_MODELS = True

# EDA grafikleri üretilecek mi?
# Whether to generate EDA plots
DO_EDA_PLOTS = True

# SHAP ve LIME açıklayıcı grafiklerini çalıştırmak için
# Whether to run SHAP and LIME explainers
DO_EXPLAINERS = True

# Optuna çıktılarında gereksiz logları azalt
# Reduce verbosity of Optuna logs
optuna.logging.set_verbosity(optuna.logging.WARNING)

# ===============================================================
# Global class definitions (for confusion_matrix & curves)
# ===============================================================
SAFE_K_CAP = 10   # Bagging-KNN ve KNN için global güvenli sınır (global safe cap)
classes_sorted = np.array([0, 1])   # Active = 0, Churned = 1
pos_label = 1                       # Positive class
neg_label = 0



print("=== Global configuration loaded. ===")
print(f"Base output directory : {BASE_OUT_DIR}")
print(f"Objective to optimize : {OPTIMIZE_FOR}")
print(f"Number of runs (N_REPS): {N_REPS}")
print("Trials per model      :", N_TRIALS_BY_MODEL)
print(f"Data Excel path       : {DATA_XLS_PATH}")


c:\Users\ulku\miniconda3\envs\ml310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


=== Global configuration loaded. ===
Base output directory : C:\Users\ulku\Desktop\ecommerce\featselect_optuna_final
Objective to optimize : auc
Number of runs (N_REPS): 10
Trials per model      : {'Random Forest': 150, 'XGBoost': 150, 'KNN': 150, 'Bagging': 150}
Data Excel path       : C:\Users\ulku\Desktop\ecommerce\featselect_optuna_final\ecommerceChurnDataset.xlsx


In [ ]:
# ===============================================================
# 1) Load Data & Basic Inspection
# ===============================================================
# Bu blok, Excel veri setini yükler, temel yapısını inceler ve
# hedef (label) ile ID sütunlarını net olarak tanımlar.
# This block loads the Excel dataset, performs basic inspection,
# and explicitly defines the target (label) and ID columns.

# ---------------------------------------------------------------
# 1.1) Veri setini Excel'den oku
# 1.1) Read dataset from Excel file
# ---------------------------------------------------------------
print("=== BLOCK 1: Loading dataset from Excel ===")
print(f"Reading data from: {DATA_XLS_PATH}")

df_raw = pd.read_excel(DATA_XLS_PATH)

print("\nFirst 5 rows of the raw dataset / Ham verinin ilk 5 satırı:")
display(df_raw.head())

print("\nDataset shape (rows, columns) / Veri seti boyutu (satır, sütun):")
print(df_raw.shape)

print("\nColumn list / Sütun listesi:")
print(list(df_raw.columns))

# ---------------------------------------------------------------
# 1.2) Hedef ve ID sütunlarını tanımla
# 1.2) Define target and ID columns
# ---------------------------------------------------------------
# Bu projede hedef değişken 'Churn' sütunudur.
# In this project, the target variable is the 'Churn' column.
TARGET_COL = "Churn"

# Müşteri kimliğini temsil eden sütun 'CustomerID' olarak alınır.
# The customer identifier column is taken as 'CustomerID'.
ID_COL = "CustomerID"

# Güvenlik: Bu sütunlar gerçekten mevcut mu kontrol edelim.
# Safety check: Ensure these columns actually exist in the dataframe.
missing_critical = [col for col in [TARGET_COL, ID_COL] if col not in df_raw.columns]
if missing_critical:
    raise ValueError(f"Critical columns not found in dataset / Veri setinde kritik sütun(lar) eksik: {missing_critical}")

print(f"\nTarget column (label) / Hedef sütun (etiket): {TARGET_COL}")
print(f"ID column / Kimlik sütunu: {ID_COL}")

# ---------------------------------------------------------------
# 1.3) Hedef değişkenin (Churn) tipini ve dağılımını kontrol et
# 1.3) Check type and distribution of the target variable (Churn)
# ---------------------------------------------------------------
# Hedef vektör (y) – etiket sütunu
# Target vector (y) – label column
y_raw = df_raw[TARGET_COL]

print("\nTarget column dtype / Hedef sütun veri tipi:", y_raw.dtype)
print("\nRaw value counts of target / Hedef sütunun ham değer dağılımı:")
print(y_raw.value_counts(dropna=False))

# Hedef sütunu sayısal değilse (örneğin 'Yes'/'No', 'Churned'/'Active'),
# bunu ikili (0/1) forma map etmeye çalışıyoruz.
# If the target column is not numeric (e.g. 'Yes'/'No', 'Churned'/'Active'),
# we try to map it to a binary (0/1) form.
if not np.issubdtype(y_raw.dtype, np.number):
    print("\nTarget is not numeric. Trying to map to {0,1} / Hedef sayısal değil. {0,1} formatına dönüştürülüyor.")

    # Pozitif ve negatif sınıf için sık görülen etiketler
    # Common labels for positive and negative classes
    pos_labels = {"yes", "y", "churned", "exited", "1", "true", "t"}
    neg_labels = {"no", "n", "active", "0", "false", "f", "stayed"}

    def map_churn_value(v):
        if pd.isna(v):
            return np.nan
        s = str(v).strip().lower()
        if s in pos_labels:
            return 1
        if s in neg_labels:
            return 0
        # Bilinmeyen etiketler için hata fırlat.
        # Raise an error for unknown/unmapped labels.
        raise ValueError(f"Unexpected label in Churn column: {v}")

    df_raw[TARGET_COL] = df_raw[TARGET_COL].apply(map_churn_value)
    y_raw = df_raw[TARGET_COL]

# Hedefi int tipine çevir (NaN yoksa).
# Cast target to integer type (if there are no NaNs).
if y_raw.isna().any():
    raise ValueError("Churn column contains NaN values after mapping. Please check the original labels.")
else:
    df_raw[TARGET_COL] = df_raw[TARGET_COL].astype(int)

print("\nMapped/verified target distribution (0=Active, 1=Churned) /")
print("Haritalanmış/doğrulanmış hedef dağılımı (0=Aktif, 1=Kaybedilen):")
print(df_raw[TARGET_COL].value_counts())

# Pozitif ve negatif sınıf etiketlerini sabitle 
# Fix positive and negative label identifiers 
POS_LABEL = 1  # churned
NEG_LABEL = 0  # active

# ---------------------------------------------------------------
# 1.4) Özellik sütunlarını (feature columns) çıkar
# 1.4) Derive feature columns
# ---------------------------------------------------------------
# ID ve hedef dışındaki tüm sütunlar özellik olarak alınır.
# All columns except ID and target are treated as features.
feature_cols = [c for c in df_raw.columns if c not in [TARGET_COL, ID_COL]]

print("\nNumber of feature columns / Özellik sütunu sayısı:", len(feature_cols))
print("Feature columns / Özellik sütunları:")
print(feature_cols)

# Mini kontrol: Beklenen özellik sayısı ile tutarlı mı?
# Mini check: Is it consistent with the expected number of features?
if len(feature_cols) != 18:
    print("\n[WARNING] Expected 18 feature columns but found", len(feature_cols))
    print("[UYARI] Beklenen 18 özellik sütunu yerine", len(feature_cols), "tane bulundu.")
    print("Please verify the dataset columns / Lütfen veri seti sütunlarını kontrol edin.")

# ---------------------------------------------------------------
# 1.5) Ham veri setini kaydet (opsiyonel ama faydalı)
# 1.5) Save a snapshot of the raw dataset (optional but useful )
# ---------------------------------------------------------------
raw_snapshot_path = EDA_DIR / "raw_dataset_snapshot.xlsx"
df_raw.to_excel(raw_snapshot_path, index=False)

print(f"\nRaw dataset snapshot saved to / Ham veri anlık görüntüsü kaydedildi: {raw_snapshot_path}")

print("\n=== BLOCK 1 completed successfully. Dataset is loaded and basic structure is verified. ===")

# -----------------------------------

# ===============================================================
# NAN ANALYSIS 
# ===============================================================
# Bu blok veri setindeki eksik değerleri inceler, raporlar ve kaydeder.
# This block inspects, reports, and exports missing-value statistics.

print("\n\n========== NaN ANALYSIS REPORT ==========\n")

# --- 1) NaN count and percentage table ---
nan_counts = df_raw[feature_cols].isna().sum()
nan_percent = (nan_counts / len(df_raw)) * 100

nan_report = pd.DataFrame({
    "NaN Count": nan_counts,
    "NaN Percentage (%)": nan_percent.round(2)
})

print("=== NaN Count / Percentage per Column ===")
display(nan_report)

# Save report to Excel
nan_report_path = BASE_OUT_DIR / "nan_report.xlsx"
nan_report.to_excel(nan_report_path, index=True)
print(f"\nNaN report saved to: {nan_report_path}")


# --- 2) Rows containing any NaN ---
nan_rows = df_raw[df_raw[feature_cols].isna().any(axis=1)]

print("\n=== Rows That Contain Any NaN ===")
print(f"Total rows with NaN: {len(nan_rows)}\n")

# Show sample of NaN rows
display(nan_rows.head(15))

# Save full NaN rows to Excel
nan_rows_path = BASE_OUT_DIR / "rows_with_nan.xlsx"
nan_rows.to_excel(nan_rows_path, index=False)

print(f"\nRows with NaN saved to: {nan_rows_path}")


# --- 3) Save original feature table before any scaling ---
features_original_units = df_raw[feature_cols].copy()
features_original_units_path = OUT_DIR / "features_original_units.xlsx"
features_original_units.to_excel(features_original_units_path, index=False)

print("\n=========================================")
print("Original feature table saved to:")
print(features_original_units_path)
print("=========================================\n")

# ===============================================================
# EXTENSION: NAN MAP ANALIZI (X MATRISI UZERINDEN)
# ===============================================================
# Bu blok, yalnızca özellik matrisi X üzerinde NaN maskesi oluşturur
# ve hangi satır/sütunlarda NaN olduğunu boolean formatında gösterir.
# This block creates a NaN mask on the feature matrix X and displays
# which rows/columns contain NaN values in boolean form.

print("\n========== NAN MAP ANALIZI ==========\n")

# Feature matrix X (ham özellikler)
X = df_raw[feature_cols].copy()
# Label vector y (global for later blocks)
y = df_raw[TARGET_COL].copy()

# NaN maskesi (True = NaN, False = dolu)
# NaN mask (True = missing, False = not missing)
nan_map = X.isna()

# Sadece NaN olan satırları göster
nan_rows = nan_map[nan_map.any(axis=1)]

print("=== Rows Containing NaN Values (Boolean Mask) ===")
display(nan_rows)

print("\n=== NaN Count Per Column ===")
nan_count_sorted = X.isna().sum().sort_values(ascending=False)
display(nan_count_sorted)

# Maskeyi Excel'e kaydet 
# Save NaN mask to Excel 
nan_map_path = BASE_OUT_DIR / "nan_map_boolean.xlsx"
nan_map.to_excel(nan_map_path, index=False)

print(f"\nNaN boolean map saved to: {nan_map_path}")

# ===============================================================
#  NUMERICAL & CATEGORICAL FEATURE SUMMARY
# ===============================================================
# Bu blok, veri setindeki sayısal ve kategorik sütunları ayırır,
# sayılarını verir ve inceleme için ekrana basar.
# This block separates numerical and categorical columns, prints
# their counts, and displays them for inspection.

print("\n========== FEATURE TYPE SUMMARY ==========\n")

# ---------------------------
# Detect numerical columns
# ---------------------------
numeric_cols = df_raw[feature_cols].select_dtypes(include=[np.number]).columns.tolist()

# ---------------------------
# Detect categorical columns
# ---------------------------
categorical_cols = [c for c in feature_cols if c not in numeric_cols]

print("=== Numerical Columns (Sayısal Değişkenler) ===")
print(f"Count (Sayı): {len(numeric_cols)}")
print(numeric_cols)

print("\n=== Categorical Columns (Kategorik Değişkenler) ===")
print(f"Count (Sayı): {len(categorical_cols)}")
print(categorical_cols)

# Save lists to Excel 
# Kayıt 
feature_types_path = BASE_OUT_DIR / "feature_types.xlsx"
pd.DataFrame({
    "Numerical": pd.Series(numeric_cols),
    "Categorical": pd.Series(categorical_cols)
}).to_excel(feature_types_path, index=False)

print(f"\nFeature type summary saved to: {feature_types_path}")



=== BLOCK 1: Loading dataset from Excel ===
Reading data from: C:\Users\ulku\Desktop\ecommerce\featselect_optuna_final\ecommerceChurnDataset.xlsx

First 5 rows of the raw dataset / Ham verinin ilk 5 satırı:


,CustomerID,Churn,Tenure,PreferredLoginDevice,CityTier,WarehouseToHome,PreferredPaymentMode,Gender,HourSpendOnApp,NumberOfDeviceRegistered,PreferedOrderCat,SatisfactionScore,MaritalStatus,NumberOfAddress,Complain,OrderAmountHikeFromlastYear,CouponUsed,OrderCount,DaySinceLastOrder,CashbackAmount
0,50001,1,4.0,Mobile Phone,3,6.0,Debit Card,Female,3.0,3,Laptop & Accessory,2,Single,9,1,11.0,1.0,1.0,5.0,159.93
1,50002,1,NaN,Phone,1,8.0,UPI,Male,3.0,4,Mobile,3,Single,7,1,15.0,0.0,1.0,0.0,120.90
2,50003,1,NaN,Phone,1,30.0,Debit Card,Male,2.0,4,Mobile,3,Single,6,1,14.0,0.0,1.0,3.0,120.28
3,50004,1,0.0,Phone,3,15.0,Debit Card,Male,2.0,4,Laptop & Accessory,5,Single,8,0,23.0,0.0,1.0,3.0,134.07
4,50005,1,0.0,Phone,1,12.0,CC,Male,NaN,3,Mobile,5,Single,3,0,11.0,1.0,1.0,3.0,129.60



Dataset shape (rows, columns) / Veri seti boyutu (satır, sütun):
(5630, 20)

Column list / Sütun listesi:
['CustomerID', 'Churn', 'Tenure', 'PreferredLoginDevice', 'CityTier', 'WarehouseToHome', 'PreferredPaymentMode', 'Gender', 'HourSpendOnApp', 'NumberOfDeviceRegistered', 'PreferedOrderCat', 'SatisfactionScore', 'MaritalStatus', 'NumberOfAddress', 'Complain', 'OrderAmountHikeFromlastYear', 'CouponUsed', 'OrderCount', 'DaySinceLastOrder', 'CashbackAmount']

Target column (label) / Hedef sütun (etiket): Churn
ID column / Kimlik sütunu: CustomerID

Target column dtype / Hedef sütun veri tipi: int64

Raw value counts of target / Hedef sütunun ham değer dağılımı:
Churn
0    4682
1     948
Name: count, dtype: int64

Mapped/verified target distribution (0=Active, 1=Churned) /
Haritalanmış/doğrulanmış hedef dağılımı (0=Aktif, 1=Kaybedilen):
Churn
0    4682
1     948
Name: count, dtype: int64

Number of feature columns / Özellik sütunu sayısı: 18
Feature columns / Özellik sütunları:
['Tenure

,NaN Count,NaN Percentage (%)
Tenure,264,4.69
PreferredLoginDevice,0,0.00
CityTier,0,0.00
WarehouseToHome,251,4.46
PreferredPaymentMode,0,0.00
Gender,0,0.00
HourSpendOnApp,255,4.53
NumberOfDeviceRegistered,0,0.00
PreferedOrderCat,0,0.00
SatisfactionScore,0,0.00



NaN report saved to: C:\Users\ulku\Desktop\ecommerce\featselect_optuna_final\nan_report.xlsx

=== Rows That Contain Any NaN ===
Total rows with NaN: 1856



...


Rows with NaN saved to: C:\Users\ulku\Desktop\ecommerce\featselect_optuna_final\rows_with_nan.xlsx

Original feature table saved to:
C:\Users\ulku\Desktop\ecommerce\featselect_optuna_final\outputs_OBJ_auc\features_original_units.xlsx


========== NAN MAP ANALIZI ==========

=== Rows Containing NaN Values (Boolean Mask) ===


,Tenure,PreferredLoginDevice,CityTier,WarehouseToHome,PreferredPaymentMode,Gender,HourSpendOnApp,NumberOfDeviceRegistered,PreferedOrderCat,SatisfactionScore,MaritalStatus,NumberOfAddress,Complain,OrderAmountHikeFromlastYear,CouponUsed,OrderCount,DaySinceLastOrder,CashbackAmount
1,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False
6,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
7,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5610,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False
5621,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False
5622,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False
5623,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False



=== NaN Count Per Column ===


DaySinceLastOrder              307
OrderAmountHikeFromlastYear    265
Tenure                         264
OrderCount                     258
CouponUsed                     256
HourSpendOnApp                 255
WarehouseToHome                251
PreferredLoginDevice             0
PreferredPaymentMode             0
CityTier                         0
SatisfactionScore                0
PreferedOrderCat                 0
NumberOfDeviceRegistered         0
Gender                           0
Complain                         0
NumberOfAddress                  0
MaritalStatus                    0
CashbackAmount                   0
dtype: int64


NaN boolean map saved to: C:\Users\ulku\Desktop\ecommerce\featselect_optuna_final\nan_map_boolean.xlsx

========== FEATURE TYPE SUMMARY ==========

=== Numerical Columns (Sayısal Değişkenler) ===
Count (Sayı): 13
['Tenure', 'CityTier', 'WarehouseToHome', 'HourSpendOnApp', 'NumberOfDeviceRegistered', 'SatisfactionScore', 'NumberOfAddress', 'Complain', 'OrderAmountHikeFromlastYear', 'CouponUsed', 'OrderCount', 'DaySinceLastOrder', 'CashbackAmount']

=== Categorical Columns (Kategorik Değişkenler) ===
Count (Sayı): 5
['PreferredLoginDevice', 'PreferredPaymentMode', 'Gender', 'PreferedOrderCat', 'MaritalStatus']

Feature type summary saved to: C:\Users\ulku\Desktop\ecommerce\featselect_optuna_final\feature_types.xlsx


In [ ]:
# ===============================================================
# 2) BASIC CLEANING, IMPUTATION & PREPROCESSOR DEFINITION - 
# Temel veri temizleme -Eksik veri doldurma - 
# Ön işleyici tanımlama Preprocessor Definition → Ön işleyici tanımlama 
# (makine öğrenmesi öncesi kullanılacak Pipeline / ColumnTransformer yapılarının tanımlanması)
# ===============================================================
# Bu blok, eksik değerleri (NaN) giderir, modelleme için temiz # X_full ve y_full tablolarını
# oluşturur ve ardından e-ticaret churn veri seti için leakage-free bir ön işleme (preprocessor)
# yapısı tanımlar + örnek çıktıları Excel'e kaydeder.

# This block handles missing values (NaNs), prepares clean X_full  and y_full for modeling
# and then defines a leakage-free preprocessing pipeline for the e-commerce churn dataset, with
# example exports to Excel 
# ===============================================================

print("\n=== BLOCK 2: BASIC CLEANING, IMPUTATION & PREPROCESSOR SETUP ===\n")

# ---------------------------------------------------------------
# 2.1) Genel özet: etiket ve özellik bilgisi
# 2.1) General summary: label and feature info
# ---------------------------------------------------------------
n_samples = df_raw.shape[0]
n_features = len(feature_cols)

print(f"Label column: {TARGET_COL} | Negative={NEG_LABEL} | Positive={POS_LABEL}")
print(f"n_samples: {n_samples} | n_features: {n_features}")
print("Feature columns (first 10) / Özellik sütunları (ilk 10):")
print(feature_cols[:10])

# ---------------------------------------------------------------
# 2.2) Çalışma kopyası oluştur (ham veriye dokunmadan)
# 2.2) Create a working copy (keep raw data intact)
# ---------------------------------------------------------------
df_clean = df_raw.copy()

# Sadece özellikler üzerinden NaN durumuna bakarız
# Check NaNs only on feature columns
nan_counts_before = df_clean[feature_cols].isna().sum()
total_nan_before = int(nan_counts_before.sum())

print("\nNaN summary BEFORE imputation / Doldurma ÖNCESI NaN özeti:")
print(nan_counts_before[nan_counts_before > 0].sort_values(ascending=False))
print(f"\nTotal NaN cells in features before imputation: {total_nan_before}")

# ---------------------------------------------------------------
# 2.3) Sayısal ve kategorik sütunlara göre NaN doldurma
# 2.3) Impute NaNs for numerical and categorical columns separately
# ---------------------------------------------------------------

# Sayısal sütunlarda median ile doldur
# Fill numerical columns with median values
for col in numeric_cols:
    if df_clean[col].isna().any():
        median_val = df_clean[col].median()
        df_clean[col].fillna(median_val, inplace=True)
        print(f"[NUMERIC] Filled NaNs in '{col}' with median = {median_val:.4f}")

# Kategorik sütunlarda en sık görülen değer (mode) ile doldur
# Fill categorical columns with most frequent value (mode)
for col in categorical_cols:
    if df_clean[col].isna().any():
        mode_val = df_clean[col].mode(dropna=True)
        if len(mode_val) > 0:
            fill_val = mode_val.iloc[0]
            df_clean[col].fillna(fill_val, inplace=True)
            print(f"[CATEGORICAL] Filled NaNs in '{col}' with mode = {fill_val}")
        else:
            # Eğer tüm değerler NaN ise, 'Unknown' ile doldur
            # If all values are NaN (edge case), fill with 'Unknown'
            df_clean[col].fillna("Unknown", inplace=True)
            print(f"[CATEGORICAL] Column '{col}' had all NaNs, filled with 'Unknown'.")

# ---------------------------------------------------------------
# 2.4) NaN sonrası kontrol
# 2.4) Check NaNs after imputation
# ---------------------------------------------------------------
nan_counts_after = df_clean[feature_cols].isna().sum()
total_nan_after = int(nan_counts_after.sum())

print("\nNaN summary AFTER imputation / Doldurma SONRASI NaN özeti:")
print(nan_counts_after[nan_counts_after > 0])

print(f"\nTotal NaN cells in features after imputation: {total_nan_after}")

if total_nan_after == 0:
    print("\n[OK] All NaNs in feature columns have been handled.")
else:
    print("\n[WARNING] Some NaNs remain in feature columns. Please inspect.")

# ---------------------------------------------------------------
# 2.5) Modelleme için X_full ve y_full oluştur
# 2.5) Create X_full and y_full for modeling
# ---------------------------------------------------------------
X_full = df_clean[feature_cols].copy()
y_full = df_clean[TARGET_COL].copy()

print("\nX_full shape:", X_full.shape)
print("y_full shape:", y_full.shape)

# ---------------------------------------------------------------
# 2.6) Temizlenmiş veri setini Excel'e kaydet
# 2.6) Save cleaned dataset to Excel
# ---------------------------------------------------------------
clean_full_path = OUT_DIR / "cleaned_dataset_for_model.xlsx"
df_clean.to_excel(clean_full_path, index=False)

print(f"\nCleaned dataset (full) saved to:")
print(clean_full_path)


# ===============================================================
# 2B) PREPROCESSING PIPELINE (LEAKAGE-FREE) — from our previous work - Ön İşleme (Sızıntısız / Veri Kaçağı Olmadan, Pipeline içinde)
# ===============================================================
# Temiz veri mantığını bozmadan, daha önce tanımladığımız ColumnTransformer tabanlı preprocessor
# yapısını ekliyoruz. Bu yapı asıl eğitim döngüsünde train split üzerinde fit edilecek, 
# böylece veri sızıntısı engellenecek. Yani bu blokta, e-ticaret churn veri seti için sızıntısız (leakage-free)
# bir ön işleme pipeline'ı tanımlanır.

# From this point on, we add our previously 
# defined ColumnTransformer-based preprocessor. In the training loop it will be fitted 
# only on the train split to avoid data leakage. In this block, we define a leakage-free preprocessing pipeline
# for the e-commerce churn dataset.
# ===============================================================

def build_preprocessor_all_robust(X_train: pd.DataFrame) -> ColumnTransformer:
    """
    E-ticaret churn veri seti için ön işleme adımları.
    Preprocessing steps for the e-commerce churn dataset.

    - Numeric features:
        * SimpleImputer(strategy='median')
        * RobustScaler()

    - Categorical features:
        * SimpleImputer(strategy='most_frequent')
        * OneHotEncoder(handle_unknown='ignore', sparse_output=False)

    Not:
    Burada SADECE ColumnTransformer yapısını tanımlarız.
    Asıl fit işlemi, eğitim pipeline'ı içinde (train verisi üzerinde)
    yapılacak ve böylece veri sızıntısı (data leakage) önlenmiş olacaktır.

    Note:
    We only define the ColumnTransformer structure here.
    Actual fitting will be done inside the training pipelines
    on the training split to avoid data leakage.
    """
    # Tüm feature sütunlarını al (X_train zaten Churn ve CustomerID'den arındırılmış durumda)
    # Get all feature columns (X_train is already without Churn and CustomerID)
    feature_cols_local = list(X_train.columns)

    # Numeric ve categorical sütunları otomatik tespit et
    # Auto-detect numeric and categorical columns
    num_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()
    cat_cols = X_train.select_dtypes(exclude=[np.number]).columns.tolist()

    print("\n=== PREPROCESSOR COLUMN GROUPS ===")
    print("Numeric columns   :", num_cols)
    print("Categorical cols  :", cat_cols)

    # ---- Numeric pipeline: median impute + robust scale ----
    # Sayısal değişkenler için: eksikleri median ile doldur, sonra RobustScaler uygula.
    # For numeric features: impute with median, then apply RobustScaler.
    numeric_transformer = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler",  RobustScaler())
    ])

    # ---- Categorical pipeline: most_frequent + OneHot ----
    # Kategorik değişkenler için: eksikleri en sık değerle doldur, one-hot encode et.
    # For categorical features: impute with most frequent value, then one-hot encode.
    categorical_transformer = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot",  OneHotEncoder(handle_unknown="ignore", sparse_output=False))
    ])

   # ColumnTransformer: numeric ve categorical pipeline'ları tek bir preprocessor içinde birleştir.
   # ColumnTransformer: combine numeric and categorical pipelines into a single preprocessor.
    preprocessor = ColumnTransformer(
        transformers=[
            ("num", numeric_transformer, num_cols),
            ("cat", categorical_transformer, cat_cols),
        ],
        remainder="drop"   # Kullanılmayan sütunları at
        # Drop all columns not listed (there should not be any in this setup).
    )

    return preprocessor


def clean_names(names):
    """
    Pipeline sonrası oluşan feature isimlerinden önekleri temizler.
    Remove transformer prefixes from feature names after the pipeline.

    Örnekler / Examples:
        'num__Tenure'                         → 'Tenure'
        'cat__PreferredPaymentMode_COD'       → 'PreferredPaymentMode_COD'

    Bu fonksiyon:
        - Önemli değişken tablolarında
        - Feature selection çıktılarında
        - SHAP / LIME gibi açıklanabilirlik araçlarında
        - Kaydedilen Excel/CSV sonuçlarında
    isimlerin daha okunabilir olmasını sağlar.

    This helper keeps feature names readable in:
        - feature importance tables
        - feature selection and Optuna logs
        - SHAP / LIME explainability
        - saved Excel/CSV outputs
    """
    cleaned = []
    for n in names:
        if "__" in n:
            cleaned.append(n.split("__", 1)[1])
        else:
            cleaned.append(n)
    return cleaned


# ===============================================================
# PREPROCESS → TRANSFORMED MATRIX EXPORT TO EXCEL (OPTIONAL)
# ===============================================================
# Bu blok, model eğitiminden bağımsız olarak, preprocessor'ü TÜM X üzerinde
# fit+transform eder ve ortaya çıkan feature matrisi Excel'e kaydeder.
# This block fits + transforms the preprocessor on FULL X (for inspection only)
# and saves the resulting feature matrix to Excel.

def export_preprocessed_X(preprocessor, X: pd.DataFrame, output_path: Path):
    """
    Verilen preprocessor'ü kullanarak X'i dönüştürür ve sonucu Excel'e kaydeder.
    Transform X using the given preprocessor and save the result to Excel.

    Not:
    - Bu fonksiyon TEK BAŞINA sadece veri yapısını incelemek içindir.
    - Eğitim pipeline'ında kullanılacak asıl fit, train split üzerinde yapılacaktır.
    - Böylece veri sızıntısı (leakage) önlenmiş olur.

    Note:
    - This is for inspection.
    - Actual training fit will happen on the train split inside the model pipeline.
    """

    print("\nApplying preprocessing to full feature matrix (for INSPECTION ONLY)...")

    # Fit + transform on full X (for inspection, NOT for training logic)
    X_transformed = preprocessor.fit_transform(X)

    # ColumnTransformer + OneHot sonrası feature isimlerini al
    # Get feature names after ColumnTransformer + OneHot
    try:
        feature_names = preprocessor.get_feature_names_out(X.columns)
    except TypeError:
        feature_names = preprocessor.get_feature_names_out()

    # İsimleri temizle (num__Tenure → Tenure, cat__Gender_Male → Gender_Male)
    # Clean transformed feature names (num__Tenure → Tenure)
    feature_names = clean_names(feature_names)

    # DataFrame'e dönüştür
    # Convert back to DataFrame
    X_df = pd.DataFrame(X_transformed, columns=feature_names)

    # Excel'e kaydet
    # Save to Excel
    output_path = Path(output_path)
    X_df.to_excel(output_path, index=False)

    print(f"\nPreprocessed feature matrix saved to:\n{output_path}")

# --------------------------------------------------------------
    # EKRANDA İLK 10 SATIRI GÖSTER — TR + EN açıklamalı
    # SHOW FIRST 10 ROWS ON SCREEN — with TR + EN explanation
    # --------------------------------------------------------------
    print("\n=== PREPROCESSED FEATURE MATRIX — FIRST 10 ROWS ===")
    print("=== ÖN İŞLEME SONRASI ÖZELLİK MATRİSİ — İLK 10 SATIR ===\n")
    print("(Bu tablo modelin gördüğü nihai feature uzayını temsil eder.)")
    print("(This table represents the final feature space used by the model.)\n")

    try:
        # Jupyter notebook varsa güzel görünümlü tablo
        display(X_df.head(10))
    except NameError:
        # VS Code / terminal modu için fallback
        print(X_df.head(10))

    return X_df

# ===============================================================
#  Original vs Scaled Comparison (Numeric Features)
# ===============================================================
# Bu blok, orijinal X ile ölçeklenmiş X'i (X_pre_df) yan yana
# karşılaştırmak için örnek bir tablo üretir.
# This block creates a side-by-side comparison table of original
# vs scaled numeric features.

def export_original_vs_scaled_sample(
    X_original: pd.DataFrame,
    X_scaled: pd.DataFrame,
    output_path: Path,
    n_rows: int = 50
):
    """
    Orijinal ve ölçeklenmiş numeric değişkenleri yan yana gösteren
    karşılaştırmalı bir tablo oluşturur ve Excel'e kaydeder.

    Creates a comparative table with original and scaled numeric
    features side by side and saves it to Excel.

    Parametreler / Parameters:
        X_original : Orijinal (ölçeklenmemiş) feature DataFrame
                     Original, unscaled feature DataFrame
        X_scaled   : Preprocess sonrası ölçeklenmiş DataFrame
                     Scaled feature DataFrame after preprocessing
        output_path: Kaydedilecek Excel dosyasının yolu
                     Path to the Excel file to save
        n_rows     : Kaç satır örnek alınacağı (örneğin 50)
                     Number of rows to include as a sample
    """

    # Sadece sayısal sütunlar üzerinde karşılaştırma yapıyoruz
    # We only compare numeric columns
    num_cols = X_original.select_dtypes(include=[np.number]).columns.tolist()

    # Hem orijinalde hem scaled'de olan sütunları bul
    # Find numeric columns that exist in both original and scaled tables
    common_numeric = [c for c in num_cols if c in X_scaled.columns]

    if not common_numeric:
        print("No common numeric columns found between original and scaled data.")
        return None

    # Karşılaştırma DataFrame'i oluştur
    # Build comparison DataFrame
    comp_rows = min(n_rows, len(X_original))
    comp_df = pd.DataFrame()

    for col in common_numeric:
        comp_df[f"{col}_orig"]   = X_original[col].head(comp_rows).values
        comp_df[f"{col}_scaled"] = X_scaled[col].head(comp_rows).values

    # Excel'e kaydet
    # Save to Excel
    output_path = Path(output_path)
    comp_df.to_excel(output_path, index=False)

    print(f"\nOriginal vs Scaled numeric sample saved to:\n{output_path}")

    # Ekranda ilk 10 satırı göster
    # Show first 10 rows on screen
    print("\n=== ORIGINAL vs SCALED (NUMERIC) — FIRST 10 ROWS ===")
    print("=== ORİJİNAL vs ÖLÇEKLENMİŞ (SAYISAL) — İLK 10 SATIR ===\n")

    try:
        display(comp_df.head(10))
    except NameError:
        print(comp_df.head(10))

    return comp_df


# --------------------------------------------------------------
# 2C) PREPROCESSOR'U OLUŞTUR VE ÖRNEK ÇIKTILARI AL
# 2C) BUILD PREPROCESSOR AND EXPORT SAMPLE OUTPUTS
# --------------------------------------------------------------
# Burada preprocessor'ü X_full üzerinde SADECE inceleme amaçlı
# fit+transform ediyoruz. Asıl eğitim bloklarında train split
# üzerinde tekrar fit edilecek.
#
# Here we fit+transform the preprocessor on X_full ONLY for
# inspection tables. Actual training will re-fit it
# on the train split inside the model pipelines.

# Çalışma için X referansını X_full üzerinden kuruyoruz
X = X_full.copy()

# Preprocessor yapısını oluştur ve özetini ekrana yazdır
# Build the preprocessor using the current X and print its structure
preprocessor = build_preprocessor_all_robust(X)
print("\nPreprocessor structure:\n", preprocessor)

# Preprocess edilmiş özellik matrisini Excel'e çıkar (inceleme için)
# Export the preprocessed feature matrix to Excel (for inspection)
X_pre_df = export_preprocessed_X(
    preprocessor=preprocessor,
    X=X,
    output_path=EDA_DIR / "preprocessed_features_snapshot.xlsx"
)



# Orijinal ve ölçeklenmiş numeric özellikler için karşılaştırmalı tablo oluştur
# Create side-by-side comparison for original vs scaled numeric features
orig_vs_scaled_path = EDA_DIR / "original_vs_scaled_numeric_sample.xlsx"

orig_vs_scaled_df = export_original_vs_scaled_sample(
    X_original=X,
    X_scaled=X_pre_df,
    output_path=orig_vs_scaled_path,
    n_rows=50  # örnek tablo için 50 satır yeterli
)

print("\n=== BLOCK 2 completed successfully. Cleaned data, preprocessor, and inspection exports are ready. ===")



=== BLOCK 2: BASIC CLEANING, IMPUTATION & PREPROCESSOR SETUP ===

Label column: Churn | Negative=0 | Positive=1
n_samples: 5630 | n_features: 18
Feature columns (first 10) / Özellik sütunları (ilk 10):
['Tenure', 'PreferredLoginDevice', 'CityTier', 'WarehouseToHome', 'PreferredPaymentMode', 'Gender', 'HourSpendOnApp', 'NumberOfDeviceRegistered', 'PreferedOrderCat', 'SatisfactionScore']

NaN summary BEFORE imputation / Doldurma ÖNCESI NaN özeti:
DaySinceLastOrder              307
OrderAmountHikeFromlastYear    265
Tenure                         264
OrderCount                     258
CouponUsed                     256
HourSpendOnApp                 255
WarehouseToHome                251
dtype: int64

Total NaN cells in features before imputation: 1856
[NUMERIC] Filled NaNs in 'Tenure' with median = 9.0000
[NUMERIC] Filled NaNs in 'WarehouseToHome' with median = 14.0000
[NUMERIC] Filled NaNs in 'HourSpendOnApp' with median = 3.0000
[NUMERIC] Filled NaNs in 'OrderAmountHikeFromlastYear' w

C:\Users\ulku\AppData\Local\Temp\ipykernel_14900\1939696452.py:55: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_clean[col].fillna(median_val, inplace=True)
C:\Users\ulku\AppData\Local\Temp\ipykernel_14900\1939696452.py:55: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For examp


Cleaned dataset (full) saved to:
C:\Users\ulku\Desktop\ecommerce\featselect_optuna_final\outputs_OBJ_auc\cleaned_dataset_for_model.xlsx

=== PREPROCESSOR COLUMN GROUPS ===
Numeric columns   : ['Tenure', 'CityTier', 'WarehouseToHome', 'HourSpendOnApp', 'NumberOfDeviceRegistered', 'SatisfactionScore', 'NumberOfAddress', 'Complain', 'OrderAmountHikeFromlastYear', 'CouponUsed', 'OrderCount', 'DaySinceLastOrder', 'CashbackAmount']
Categorical cols  : ['PreferredLoginDevice', 'PreferredPaymentMode', 'Gender', 'PreferedOrderCat', 'MaritalStatus']

Preprocessor structure:
 ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', RobustScaler())]),
                                 ['Tenure', 'CityTier', 'WarehouseToHome',
                                  'HourSpendOnApp', 'NumberOfDeviceRegistered',
       

,Tenure,CityTier,WarehouseToHome,HourSpendOnApp,NumberOfDeviceRegistered,SatisfactionScore,NumberOfAddress,Complain,OrderAmountHikeFromlastYear,CouponUsed,...,Gender_Male,PreferedOrderCat_Fashion,PreferedOrderCat_Grocery,PreferedOrderCat_Laptop & Accessory,PreferedOrderCat_Mobile,PreferedOrderCat_Mobile Phone,PreferedOrderCat_Others,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single
0,-0.416667,1.0,-0.727273,0.0,-1.0,-0.5,1.50,1.0,-0.8,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.000000,0.0,-0.545455,0.0,0.0,0.0,1.00,1.0,0.0,-1.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
2,0.000000,0.0,1.454545,-1.0,0.0,0.0,0.75,1.0,-0.2,-1.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
3,-0.750000,1.0,0.090909,-1.0,0.0,1.0,1.25,0.0,1.6,-1.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
4,-0.750000,0.0,-0.181818,0.0,-1.0,1.0,0.00,0.0,-0.8,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
5,-0.750000,0.0,0.727273,0.0,1.0,1.0,-0.25,1.0,1.4,3.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
6,0.000000,1.0,-0.272727,-1.0,-1.0,-0.5,0.25,0.0,-0.2,-1.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
7,0.000000,0.0,-0.727273,0.0,-1.0,-0.5,0.00,1.0,0.2,1.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
8,0.333333,1.0,-0.454545,0.0,0.0,0.0,-0.25,1.0,-0.2,-1.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
9,0.000000,0.0,1.545455,-1.0,1.0,0.0,-0.25,0.0,-0.6,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0



Original vs Scaled numeric sample saved to:
C:\Users\ulku\Desktop\ecommerce\featselect_optuna_final\eda_outputs\original_vs_scaled_numeric_sample.xlsx

=== ORIGINAL vs SCALED (NUMERIC) — FIRST 10 ROWS ===
=== ORİJİNAL vs ÖLÇEKLENMİŞ (SAYISAL) — İLK 10 SATIR ===



,Tenure_orig,Tenure_scaled,CityTier_orig,CityTier_scaled,WarehouseToHome_orig,WarehouseToHome_scaled,HourSpendOnApp_orig,HourSpendOnApp_scaled,NumberOfDeviceRegistered_orig,NumberOfDeviceRegistered_scaled,...,OrderAmountHikeFromlastYear_orig,OrderAmountHikeFromlastYear_scaled,CouponUsed_orig,CouponUsed_scaled,OrderCount_orig,OrderCount_scaled,DaySinceLastOrder_orig,DaySinceLastOrder_scaled,CashbackAmount_orig,CashbackAmount_scaled
0,4.0,-0.416667,3,1.0,6.0,-0.727273,3.0,0.0,3,-1.0,...,11.0,-0.8,1.0,0.0,1.0,-0.5,5.0,0.4,159.93,-0.066176
1,9.0,0.000000,1,0.0,8.0,-0.545455,3.0,0.0,4,0.0,...,15.0,0.0,0.0,-1.0,1.0,-0.5,0.0,-0.6,120.90,-0.837177
2,9.0,0.000000,1,0.0,30.0,1.454545,2.0,-1.0,4,0.0,...,14.0,-0.2,0.0,-1.0,1.0,-0.5,3.0,0.0,120.28,-0.849425
3,0.0,-0.750000,3,1.0,15.0,0.090909,2.0,-1.0,4,0.0,...,23.0,1.6,0.0,-1.0,1.0,-0.5,3.0,0.0,134.07,-0.577016
4,0.0,-0.750000,1,0.0,12.0,-0.181818,3.0,0.0,3,-1.0,...,11.0,-0.8,1.0,0.0,1.0,-0.5,3.0,0.0,129.60,-0.665317
5,0.0,-0.750000,1,0.0,22.0,0.727273,3.0,0.0,5,1.0,...,22.0,1.4,4.0,3.0,6.0,2.0,7.0,0.8,139.19,-0.475875
6,9.0,0.000000,3,1.0,11.0,-0.272727,2.0,-1.0,3,-1.0,...,14.0,-0.2,0.0,-1.0,1.0,-0.5,0.0,-0.6,120.86,-0.837967
7,9.0,0.000000,1,0.0,6.0,-0.727273,3.0,0.0,3,-1.0,...,16.0,0.2,2.0,1.0,2.0,0.0,0.0,-0.6,122.93,-0.797076
8,13.0,0.333333,3,1.0,9.0,-0.454545,3.0,0.0,4,0.0,...,14.0,-0.2,0.0,-1.0,1.0,-0.5,2.0,-0.2,126.83,-0.720036
9,9.0,0.000000,1,0.0,31.0,1.545455,2.0,-1.0,5,1.0,...,12.0,-0.6,1.0,0.0,1.0,-0.5,1.0,-0.4,122.93,-0.797076



=== BLOCK 2 completed successfully. Cleaned data, preprocessor, and inspection exports are ready. ===


In [ ]:
# ===============================================================
# 3) Optional EDA (Correlations, Group-Wise Boxplots, Churn & Categorical EDA)
#     + 3A–3E: Churn & Categorical EDA Summaries
#     + 3F: Categorical Overview Panel
# ===============================================================
# Bu blok, isteğe bağlı (opsiyonel) keşifsel veri analizi (EDA)
# üretir: korelasyon ısı haritası, churn durumuna göre boxplot'lar,
# churn dağılımı grafikleri, kategorik değişken EDA'sı ve
# chi-square testleri.
#
# This block performs optional Exploratory Data Analysis (EDA):
# correlation heatmap, boxplots by churn status, churn distribution
# plots, categorical EDA and chi-square tests.
# ===============================================================


if DO_EDA_PLOTS:

    # EDA stil ayarları
    # EDA style settings
    sns.set(style="whitegrid")

    # EDA'da kullanılacak veri: temizlenmiş veri seti
    # Data used for EDA: cleaned dataset
    data = df_clean.copy()
    X = X_full.copy()

    neg_label = NEG_LABEL  # 0 = Active
    pos_label = POS_LABEL  # 1 = Churned

    try:
        # =======================================================
        # 3) Correlation Heatmap & Group-wise Boxplots
        # =======================================================

        # ---------- Correlation Heatmap ----------
        # Sadece sayısal sütunlar üzerinden korelasyon
        # Correlation only among numeric columns
        num_cols = X.select_dtypes(include=[np.number]).columns
        corr = X[num_cols].corr()

        plt.figure(figsize=(12, 8))
        sns.heatmap(corr, annot=True, cmap="coolwarm", fmt=".2f", linewidths=.5)
        plt.title("Correlation Heatmap of Numerical Features (E-commerce Dataset)",
                  fontsize=16, fontweight='semibold')
        plt.xlabel("Features", fontsize=13)
        plt.ylabel("Features", fontsize=13)
        plt.tight_layout()
        plt.savefig(OUT_DIR / "eda_correlation_heatmap_ecommerce.png", dpi=200)
        plt.close()

        # ---------- Group-wise Boxplots ----------
        # Sayısal özellikler için churn=0 vs churn=1 karşılaştırmalı boxplot'lar
        # Group-wise boxplots for numeric features: churn=0 vs churn=1
        feats = list(num_cols)

        if len(feats) > 0:
            n = len(feats)
            n_cols = 3
            n_rows = int(np.ceil(n / n_cols))
            fig, axes = plt.subplots(n_rows, n_cols, figsize=(5 * n_cols, 3.5 * n_rows))
            axes = axes.ravel() if n_rows * n_cols > 1 else [axes]

            last_i = 0
            for i, col in enumerate(feats):
                ax = axes[i]

                # Group definitions (0 = ACTIVE, 1 = CHURNED)
                d0 = data.loc[data[TARGET_COL] == neg_label, col].dropna()   # Active customers
                d1 = data.loc[data[TARGET_COL] == pos_label, col].dropna()   # Churned customers

                # Boxplot
                ax.boxplot(
                    [d0, d1],
                    labels=["Active", "Churned"],
                    showfliers=True,
                    patch_artist=True,
                    boxprops=dict(facecolor='#D0E4F5')
                )

                # Title & labels
                ax.set_title(col, fontsize=14, fontweight='semibold')
                ax.set_ylabel("Feature Value", fontsize=16)
                ax.tick_params(axis='x', labelsize=16)
                ax.tick_params(axis='y', labelsize=16)
                ax.grid(axis='y', linestyle=':', linewidth=0.4)

                # Tick styling
                for tick in ax.get_xticklabels():
                    tick.set_fontweight('bold')
                    tick.set_fontsize(18)
                    tick.set_color('#2B2B2B')

                last_i = i

            # Kullanılmayan alt grafikleri kapat
            # Disable unused subplots
            for j in range(last_i + 1, len(axes)):
                axes[j].axis("off")

            # Main figure title
            fig.suptitle(
                "Distribution of Numerical Features by Churn Status (Active vs Churned)",
                y=1.02,
                fontsize=22,
                fontweight='bold'
            )

            plt.tight_layout()
            plt.savefig(OUT_DIR / "eda_boxplots_by_churn.png", dpi=300, bbox_inches='tight')
            plt.close()

        # =======================================================
        # 3A) Churn Distribution Summary (Counts + Percentages + Plots)
        # =======================================================
        
        # Folder for churn plots
        churn_dir = EDA_DIR / "churn_summary"
        churn_dir.mkdir(parents=True, exist_ok=True)

        # Count and percentage
        count_summary = data[TARGET_COL].value_counts().sort_index()
        active_count = count_summary.get(neg_label, 0)
        churn_count  = count_summary.get(pos_label, 0)

        print(f"Active customers ({TARGET_COL} = {neg_label}): {active_count}")
        print(f"Churned customers ({TARGET_COL} = {pos_label}): {churn_count}")

        print("\nPercentage distribution (%):")
        percentage_summary = (count_summary / count_summary.sum() * 100).round(2)
        print(percentage_summary)

        # Bar plot — Churn vs Active Count/Percentage
        plt.figure(figsize=(6, 4))
        sns.barplot(x=percentage_summary.index, y=percentage_summary.values, color="#4C72B0" )
        plt.xticks([neg_label, pos_label], [f'Active ({neg_label})', f'Churned ({pos_label})'])
        plt.ylabel("Percentage (%)")
        plt.title("Churn Distribution (Bar Plot)\nChurn Dağılımı")
        plt.tight_layout()
        plt.savefig(churn_dir / "churn_barplot.png", dpi=200)
        plt.close()

        # Pie chart — Churn Proportion
        plt.figure(figsize=(6, 6))
        plt.pie(
            count_summary.values,
            labels=[f'Active ({neg_label})', f'Churned ({pos_label})'],
            autopct='%1.1f%%',
            startangle=90
        )
        plt.title("Churn Distribution (Pie Chart)\nChurn Dağılımı")
        plt.tight_layout()
        plt.savefig(churn_dir / "churn_piechart.png", dpi=200)
        plt.close()

        # =======================================================
        # 3B) Categorical EDA (Countplots, Churn Rates, Chi-Square Tests)
        # =======================================================

        cat_cols_for_eda = [
            "PreferredLoginDevice",
            "PreferredPaymentMode",
            "Gender",
            "PreferedOrderCat",
            "MaritalStatus"
        ]

        # Klasörü oluştur (EDA_DIR daha önce tanımlı)
        cat_dir = EDA_DIR / "categorical"
        cat_dir.mkdir(parents=True, exist_ok=True)

        results_chi = []   # Chi-Square sonuçları için tablo / results table

        for col in cat_cols_for_eda:

            if col not in data.columns:
                print(f"\n[WARNING] Column '{col}' not found in dataset. Skipping.")
                continue

            print(f"\n=== {col} ===")

            # 1) COUNT PLOT — Category distributions — Kategori dağılımları
            # plt.figure(figsize=(8, 5))
            # sns.countplot(data=data, x=col, order=data[col].value_counts().index ,palette="Set2")
            # plt.title(f"{col} — Category Distribution\nKategori Dağılımı")
            # plt.xticks(rotation=45)
            # plt.tight_layout()
            # plt.savefig(cat_dir / f"{col}_countplot.png", dpi=300)
            # plt.close()

            # 2) CHURN RATE BARPLOT — Churn rate per category - Her kategoride churn oranı
            # churn_rate = data.groupby(col)[TARGET_COL].mean()

            # plt.figure(figsize=(8, 5))
            # churn_rate.plot(kind="bar", color="#4C72B0")
            # plt.ylabel("Churn Rate (Oran)")
            # plt.title(f"{col} — Churn Rate by Category\nKategori Bazında Churn Oranı")
            # plt.xticks(rotation=45)
            # plt.tight_layout()
            # plt.savefig(cat_dir / f"{col}_churnrate.png", dpi=300)
            # plt.close()


            # 1) COUNT PLOT — Category distributions — Kategori dağılımları
            plt.figure(figsize=(8, 5))
            sns.countplot(data=data, x=col, order=data[col].value_counts().index ,palette="Set2")
            plt.title(f"{col} — Category Distribution\nKategori Dağılımı")
            plt.xticks(rotation=45, fontsize=16, fontweight='bold') # ← FONTSIZE VE FONTWEIGHT EKLENDİ
            plt.yticks(fontsize=16)  # Y ekseni de büyütüldü
            plt.xlabel(col, fontsize=16, fontweight='bold')  # X ekseni etiketi
            plt.ylabel("Count", fontsize=16, fontweight='bold')  # Y ekseni etiketi
            plt.tight_layout()
            plt.savefig(cat_dir / f"{col}_countplot.png", dpi=300)
            plt.close()

            # 2) CHURN RATE BARPLOT — Churn rate per category - Her kategoride churn oranı
            churn_rate = data.groupby(col)[TARGET_COL].mean()

            plt.figure(figsize=(8, 5))
            churn_rate.plot(kind="bar", color="#4C72B0")
            plt.ylabel("Churn Rate (Oran)", fontsize=16, fontweight='bold') #y etiketi
            plt.xlabel(col, fontsize=16, fontweight='bold')  # X ekseni etiketi
            plt.yticks(fontsize=16)  # Y ekseni de büyütüldü
            plt.title(f"{col} — Churn Rate by Category\nKategori Bazında Churn Oranı")
            plt.xticks(rotation=45, fontsize=16, fontweight='bold') # ← FONTSIZE VE FONTWEIGHT EKLENDİ
            plt.tight_layout()
            plt.savefig(cat_dir / f"{col}_churnrate.png", dpi=300)
            plt.close()

            # 3) Pivot Table — Active vs Churn counts
            pivot_tbl = pd.crosstab(data[col], data[TARGET_COL])
            print("Pivot Table (Active vs Churn):")
            print(pivot_tbl)

            # 4) Chi-Square Test — Kategorik bağımlılık testi
            chi2, p, dof, exp = chi2_contingency(pivot_tbl)

            results_chi.append({
                "Feature": col,
                "Chi2": chi2,
                "p_value": p,
                "Degrees_of_Freedom": dof
            })

        # Chi-Square sonuçları tabloya dönüştürülür
        chi_df = pd.DataFrame(results_chi)
        print("\n=== Chi-Square Test Results (Summary) ===")
        print(chi_df)

        chi_df.to_excel(cat_dir / "chi_square_results.xlsx", index=False)

        # =======================================================
        # 3C) Chi-Square Summary — Display + Save
        # =======================================================

        try:
            # Ekranda tam tabloyu göster (Jupyter varsa)
            print("\n=== FULL CHI-SQUARE RESULTS TABLE ===\n")
            display(chi_df)
        except NameError:
            # display yoksa sadece print ile devam et
            print("\n=== FULL CHI-SQUARE RESULTS TABLE ===\n")
            print(chi_df)
        

        # Save outputs (Excel + CSV)
        output_dir = EDA_DIR / "categorical"
        output_dir.mkdir(parents=True, exist_ok=True)
        chi_df.to_excel(cat_dir / "chi_square_results_full.xlsx", index=False)
        chi_df.to_csv(cat_dir / "chi_square_results_full.csv", index=False)

        print("\nChi-square results saved to:")
        print(cat_dir / "chi_square_results_full.xlsx")
        print(cat_dir / "chi_square_results_full.csv")

        # Save Chi-square results as an image (PNG)
        fig, ax = plt.subplots(figsize=(10, len(chi_df) * 0.4 + 1))
        ax.axis('off')

        tbl = plt.table(
            cellText=chi_df.values,
            colLabels=chi_df.columns,
            loc='center',
            cellLoc='center'
        )

        tbl.auto_set_font_size(False)
        tbl.set_fontsize(9)
        tbl.scale(1, 1.3)

        png_path = cat_dir / "chi_square_results_table.png"
        plt.savefig(png_path, dpi=200, bbox_inches='tight')
        plt.close()

        print("PNG table saved to:", png_path)

        # =======================================================
        # 3D) Statistically Significant Features (p < 0.05)
        # =======================================================

        significant = chi_df[chi_df["p_value"] < 0.05].copy()

        print("\n=== Statistically Significant Features (p < 0.05) ===\n")
        try:
            display(significant)
        except NameError:
            print(significant)

        sig_path_excel = cat_dir / "chi_square_significant.xlsx"
        sig_path_csv   = cat_dir / "chi_square_significant.csv"

        significant.to_excel(sig_path_excel, index=False)
        significant.to_csv(sig_path_csv, index=False)

        print("\nSignificant features saved to:")
        print(sig_path_excel)
        print(sig_path_csv)

        # Save significant table as PNG
        fig, ax = plt.subplots(figsize=(10, len(significant) * 0.4 + 1))
        ax.axis('off')

        tbl = plt.table(
            cellText=significant.values,
            colLabels=significant.columns,
            loc='center',
            cellLoc='center'
        )

        tbl.auto_set_font_size(False)
        tbl.set_fontsize(10)
        tbl.scale(1, 1.3)

        png_sig_path = cat_dir / "chi_square_significant_table.png"
        plt.savefig(png_sig_path, dpi=200, bbox_inches='tight')
        plt.close()

        print("\nPNG table saved to:", png_sig_path)

        # =======================================================
        # 3E) EDA Summary Overview (One-Page Figure)
        # =======================================================
        # Küçük bir özet tablo: churn yüzdeleri + chi-square sonuçları         
        # Bu bölüm, EDA görsel tablosu üretir.

        # Small summary table: churn percentages + chi-square results
        # This section creates a compact one-page EDA overview figure 

        summary_dir = EDA_DIR / "summary"
        summary_dir.mkdir(parents=True, exist_ok=True)

        # Küçük bir özet tablo: churn yüzdeleri + chi-square sonuçları
        # Small summary table: churn percentages + chi-square results
        overview_df = chi_df.copy()
        overview_df["Churn_0_pct"] = percentage_summary.get(neg_label, np.nan)
        overview_df["Churn_1_pct"] = percentage_summary.get(pos_label, np.nan)

        fig, ax = plt.subplots(figsize=(12, len(overview_df) * 0.4 + 1.5))
        ax.axis('off')

        tbl = plt.table(
            cellText=overview_df.values,
            colLabels=overview_df.columns,
            loc='center',
            cellLoc='center'
        )
        tbl.auto_set_font_size(False)
        tbl.set_fontsize(9)
        tbl.scale(1, 1.3)

        plt.title(
            "EDA Summary Overview\nChurn Percentages & Chi-Square Results",
            fontsize=14,
            pad=12
        )

        overview_png = summary_dir / "eda_summary_overview.png"
        plt.savefig(overview_png, dpi=250, bbox_inches='tight')
        plt.close()

        print("\nEDA summary overview PNG saved to:")
        print(overview_png)

    except Exception as e:
        warnings.warn(f"EDA plotting skipped due to error: {e}")


# ===============================================================
# 3F) Categorical EDA — All plots combined into one figure
# ===============================================================
# Bu fonksiyon, kategorik değişkenlere ait tüm EDA grafikleri
# (countplot + churn rate) için tek sayfalık bir özet panel üretir.
#
# This function creates a single-page panel that combines all
# categorical EDA plots (countplot + churn rate) for all variables.
# ===============================================================

def create_categorical_overview(cat_cols, output_dir):
    """
    Kategorik değişkenlere ait tüm grafik dosyalarını (countplot + churn rate)
    tek bir sayfada birleştirir.
    
    Combines all categorical EDA plots (countplot + churn rate)
    into one single-page summary figure.
    """

    print("\nCreating categorical overview panel...")

    images = []
    labels = []

    # Her sütun için 2 resim var: countplot + churnrate
    for col in cat_cols:
        count_path = output_dir / f"{col}_countplot.png"
        churn_path = output_dir / f"{col}_churnrate.png"

        if count_path.exists():
            images.append(mpimg.imread(count_path))
            labels.append(f"{col} — Countplot")

        if churn_path.exists():
            images.append(mpimg.imread(churn_path))
            labels.append(f"{col} — Churn Rate")

    if len(images) == 0:
        print("No categorical images found.")
        return

    # Grid boyutu (2 sütun, N satır)
    n_cols = 2
    n_rows = int(np.ceil(len(images) / n_cols))

    fig, axes = plt.subplots(n_rows, n_cols, figsize=(12, 4 * n_rows))
    axes = axes.ravel()

    for i, img in enumerate(images):
        axes[i].imshow(img)
        axes[i].axis("off")
        axes[i].set_title(labels[i], fontsize=16, fontweight="bold")

    # Kalan boş hücreleri kapat
    for j in range(i + 1, len(axes)):
        axes[j].axis("off")

    plt.suptitle(
        "Categorical EDA Overview — All Variables\n(Kategorik Değişkenler Genel Özeti)",
        fontsize=16, fontweight="bold", y=1.02
    )

    overview_path = output_dir / "categorical_overview.png"
    plt.tight_layout()
    plt.savefig(overview_path, dpi=350, bbox_inches="tight")
    plt.close()

    print(f"\nCategorical overview saved to:\n{overview_path}")


# --- 3F fonksiyon çağrısı (sadece EDA aktif ise) / function call only if EDA enabled ---
create_categorical_overview(
        cat_cols=[
            "PreferredLoginDevice",
            "PreferredPaymentMode",
            "Gender",
            "PreferedOrderCat",
            "MaritalStatus"
        ],
        output_dir=EDA_DIR / "categorical"
    )


C:\Users\ulku\AppData\Local\Temp\ipykernel_14900\1765986458.py:73: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  ax.boxplot(
C:\Users\ulku\AppData\Local\Temp\ipykernel_14900\1765986458.py:73: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  ax.boxplot(
C:\Users\ulku\AppData\Local\Temp\ipykernel_14900\1765986458.py:73: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  ax.boxplot(
C:\Users\ulku\AppData\Local\Temp\ipykernel_14900\1765986458.py:73: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  ax.boxplot(


Active customers (Churn = 0): 4682
Churned customers (Churn = 1): 948

Percentage distribution (%):
Churn
0    83.16
1    16.84
Name: count, dtype: float64

=== PreferredLoginDevice ===


C:\Users\ulku\AppData\Local\Temp\ipykernel_14900\1765986458.py:206: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.countplot(data=data, x=col, order=data[col].value_counts().index ,palette="Set2")


Pivot Table (Active vs Churn):
Churn                    0    1
PreferredLoginDevice           
Computer              1310  324
Mobile Phone          2417  348
Phone                  955  276

=== PreferredPaymentMode ===


C:\Users\ulku\AppData\Local\Temp\ipykernel_14900\1765986458.py:206: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.countplot(data=data, x=col, order=data[col].value_counts().index ,palette="Set2")


...

Pivot Table (Active vs Churn):
Churn             0    1
MaritalStatus           
Divorced        724  124
Married        2642  344
Single         1316  480

=== Chi-Square Test Results (Summary) ===
                Feature        Chi2       p_value  Degrees_of_Freedom
0  PreferredLoginDevice   73.536794  1.075692e-16                   2
1  PreferredPaymentMode   77.896820  9.708709e-15                   6
2                Gender    4.662908  3.082094e-02                   1
3      PreferedOrderCat  288.639394  2.770833e-60                   5
4         MaritalStatus  188.671040  1.073011e-41                   2

=== FULL CHI-SQUARE RESULTS TABLE ===



,Feature,Chi2,p_value,Degrees_of_Freedom
0,PreferredLoginDevice,73.536794,1.075692e-16,2
1,PreferredPaymentMode,77.896820,9.708709e-15,6
2,Gender,4.662908,3.082094e-02,1
3,PreferedOrderCat,288.639394,2.770833e-60,5
4,MaritalStatus,188.671040,1.073011e-41,2



Chi-square results saved to:
C:\Users\ulku\Desktop\ecommerce\featselect_optuna_final\eda_outputs\categorical\chi_square_results_full.xlsx
C:\Users\ulku\Desktop\ecommerce\featselect_optuna_final\eda_outputs\categorical\chi_square_results_full.csv
PNG table saved to: C:\Users\ulku\Desktop\ecommerce\featselect_optuna_final\eda_outputs\categorical\chi_square_results_table.png

=== Statistically Significant Features (p < 0.05) ===



,Feature,Chi2,p_value,Degrees_of_Freedom
0,PreferredLoginDevice,73.536794,1.075692e-16,2
1,PreferredPaymentMode,77.896820,9.708709e-15,6
2,Gender,4.662908,3.082094e-02,1
3,PreferedOrderCat,288.639394,2.770833e-60,5
4,MaritalStatus,188.671040,1.073011e-41,2



Significant features saved to:
C:\Users\ulku\Desktop\ecommerce\featselect_optuna_final\eda_outputs\categorical\chi_square_significant.xlsx
C:\Users\ulku\Desktop\ecommerce\featselect_optuna_final\eda_outputs\categorical\chi_square_significant.csv

PNG table saved to: C:\Users\ulku\Desktop\ecommerce\featselect_optuna_final\eda_outputs\categorical\chi_square_significant_table.png

EDA summary overview PNG saved to:
C:\Users\ulku\Desktop\ecommerce\featselect_optuna_final\eda_outputs\summary\eda_summary_overview.png

Creating categorical overview panel...

Categorical overview saved to:
C:\Users\ulku\Desktop\ecommerce\featselect_optuna_final\eda_outputs\categorical\categorical_overview.png


In [34]:
# ===============================================================
# 3G) Descriptive Statistics Table (Tanımlayıcı İstatistikler)
# ===============================================================

if DO_EDA_PLOTS:
    
    # Veri:  temizlenmiş veri seti
    data = df_clean.copy()
    
    # Sayısal sütunları otomatik tespit et
    numerical_cols = data.select_dtypes(include=[np.number]).columns.tolist()
    
    # Hedef değişken ve CustomerID'yi çıkar
    if TARGET_COL in numerical_cols: 
        numerical_cols.remove(TARGET_COL)
    if 'CustomerID' in numerical_cols:
        numerical_cols. remove('CustomerID')
    
    print(f"\n=== Numerical columns for descriptive stats:  {len(numerical_cols)} ===")
    print(numerical_cols)
    
    # Tanımlayıcı istatistikler hesapla
    desc_stats = data[numerical_cols].describe().T

        # Sütun adlarını düzenle (25% → Q1, 75% → Q3)  
    desc_stats = desc_stats.rename(columns={
        '25%': '25%(Q1)',
        '50%': '50%(Median)',
        '75%': '75%(Q3)'
    })
    
    desc_stats['skewness'] = data[numerical_cols].skew()
    desc_stats['kurtosis'] = data[numerical_cols].kurtosis()
    
    # Sütun sırasını düzenle  
    desc_stats = desc_stats[['count', 'mean', 'std', 'min', '25%(Q1)', '50%(Median)', '75%(Q3)', 'max', 'skewness', 'kurtosis']]
    
    print("\n=== Descriptive Statistics ===")
    try:
        display(desc_stats)
    except NameError:
        print(desc_stats)
    
    # Klasör oluştur
    desc_dir = EDA_DIR / "summary"
    desc_dir.mkdir(parents=True, exist_ok=True)
    
    # Excel'e kaydet
    desc_stats. to_excel(desc_dir / "descriptive_statistics.xlsx")
    
    # PNG olarak kaydet
    fig, ax = plt.subplots(figsize=(16, len(desc_stats) * 0.4 + 1.5))  # Genişlik artırıldı (10 sütun için)
    ax.axis('off')
    
    tbl = plt.table(
        cellText=desc_stats.round(2).values,
        colLabels=desc_stats.columns,
        rowLabels=desc_stats.index,
        loc='center',
        cellLoc='center'
    )
    tbl.auto_set_font_size(False)
    tbl.set_fontsize(8)
    tbl.scale(1, 1.4)
    
    plt.title(
        "Descriptive Statistics of Numerical Features\n(Sayısal Değişkenlerin Tanımlayıcı İstatistikleri)", 
        fontsize=14, pad=12, fontweight='bold'
    )
    plt.savefig(desc_dir / "descriptive_statistics_table.png", dpi=250, bbox_inches='tight')
    plt.close()
    
    print(f"\n✅ Descriptive statistics saved to:")
    print(f"   {desc_dir / 'descriptive_statistics.xlsx'}")
    print(f"   {desc_dir / 'descriptive_statistics_table.png'}")


=== Numerical columns for descriptive stats:  13 ===
['Tenure', 'CityTier', 'WarehouseToHome', 'HourSpendOnApp', 'NumberOfDeviceRegistered', 'SatisfactionScore', 'NumberOfAddress', 'Complain', 'OrderAmountHikeFromlastYear', 'CouponUsed', 'OrderCount', 'DaySinceLastOrder', 'CashbackAmount']

=== Descriptive Statistics ===


,count,mean,std,min,25%(Q1),50%(Median),75%(Q3),max,skewness,kurtosis
Tenure,5630.0,10.134103,8.357951,0.0,3.00,9.00,15.0000,61.00,0.773278,0.154613
CityTier,5630.0,1.654707,0.915389,1.0,1.00,1.00,3.0000,3.00,0.735326,-1.401529
WarehouseToHome,5630.0,15.566785,8.345961,5.0,9.00,14.00,20.0000,127.00,1.678346,10.606302
HourSpendOnApp,5630.0,2.934636,0.705528,0.0,2.00,3.00,3.0000,5.00,-0.040982,-0.557728
NumberOfDeviceRegistered,5630.0,3.688988,1.023999,1.0,3.00,4.00,4.0000,6.00,-0.396969,0.582849
SatisfactionScore,5630.0,3.066785,1.380194,1.0,2.00,3.00,4.0000,5.00,-0.142626,-1.125136
NumberOfAddress,5630.0,4.214032,2.583586,1.0,2.00,3.00,6.0000,22.00,1.088639,0.959229
Complain,5630.0,0.284902,0.451408,0.0,0.00,0.00,1.0000,1.00,0.953347,-1.091518
OrderAmountHikeFromlastYear,5630.0,15.674600,3.591058,11.0,13.00,15.00,18.0000,26.00,0.835439,-0.125375
CouponUsed,5630.0,1.716874,1.857640,0.0,1.00,1.00,2.0000,16.00,2.630035,9.722813



✅ Descriptive statistics saved to:
   C:\Users\ulku\Desktop\ecommerce\featselect_optuna_final\eda_outputs\summary\descriptive_statistics.xlsx
   C:\Users\ulku\Desktop\ecommerce\featselect_optuna_final\eda_outputs\summary\descriptive_statistics_table.png


In [40]:
# ===============================================================
# 3Ga) Categorical Frequency Analysis (Kategorik Frekans Analizi)
# ===============================================================

if DO_EDA_PLOTS:
    
    print("\n=== Categorical Frequency Analysis ===")
    
    # Veri
    data = df_clean. copy()
    
    # Kategorik sütunları otomatik tespit et
    categorical_cols = data.select_dtypes(include=['object', 'category']).columns.tolist()
    
    # Hedef değişkeni çıkar
    if TARGET_COL in categorical_cols:
        categorical_cols.remove(TARGET_COL)
    
    # CustomerID varsa çıkar
    if 'CustomerID' in categorical_cols:
        categorical_cols.remove('CustomerID')
    
    print(f"Categorical columns: {categorical_cols}")
    
    # Her kategorik değişken için frekans tablosu oluştur
    frequency_results = []
    
    for col in categorical_cols: 
        # Value counts hesapla
        counts = data[col].value_counts()
        percentages = data[col].value_counts(normalize=True) * 100
        
        # DataFrame oluştur
        for category in counts.index:
            frequency_results.append({
                'Variable': col,
                'Category':  category,
                'Frequency':  counts[category],
                'Percentage': round(percentages[category], 2)
            })
    
    # DataFrame'e dönüştür
    frequency_df = pd.DataFrame(frequency_results)
    
    print("\n=== Categorical Frequency Table ===")
    try:
        display(frequency_df)
    except NameError:
        print(frequency_df)
    
    # Klasör oluştur
    cat_dir = EDA_DIR / "categorical"
    cat_dir.mkdir(parents=True, exist_ok=True)
    
    # Excel'e kaydet (her değişken ayrı sheet)
    with pd.ExcelWriter(cat_dir / "categorical_frequency_analysis.xlsx") as writer:
        # Tüm sonuçlar tek tabloda
        frequency_df.to_excel(writer, sheet_name='All_Variables', index=False)
        
        # Her değişken için ayrı sheet
        for col in categorical_cols:
            col_data = frequency_df[frequency_df['Variable'] == col]. copy()
            col_data = col_data[['Category', 'Frequency', 'Percentage']]
            col_data.to_excel(writer, sheet_name=col[: 31], index=False)  # Excel sheet name max 31 char
    
    # CSV'ye de kaydet
    frequency_df.to_csv(cat_dir / "categorical_frequency_analysis.csv", index=False)
    
    # PNG tablo olarak kaydet
    fig, ax = plt.subplots(figsize=(12, len(frequency_df) * 0.35 + 1.5))
    ax.axis('off')
    
    tbl = plt.table(
        cellText=frequency_df. values,
        colLabels=frequency_df.columns,
        loc='center',
        cellLoc='left'
    )
    tbl.auto_set_font_size(False)
    tbl.set_fontsize(9)
    tbl.scale(1, 1.3)
    
    # Başlık ekle
    plt.title(
        "Categorical Variables - Frequency Analysis\n(Kategorik Değişkenlerin Frekans Analizi)",
        fontsize=14,
        fontweight='bold',
        pad=15
    )
    
    plt.savefig(cat_dir / "categorical_frequency_table.png", dpi=250, bbox_inches='tight')
    plt.close()
    
    print(f"\n✅ Categorical frequency analysis saved to:")
    print(f"   {cat_dir / 'categorical_frequency_analysis.xlsx'}")
    print(f"   {cat_dir / 'categorical_frequency_analysis.csv'}")
    print(f"   {cat_dir / 'categorical_frequency_table.png'}")
    
    # Özet istatistik
    print(f"\n📊 Summary:")
    print(f"   Total categorical variables: {len(categorical_cols)}")
    print(f"   Total categories: {len(frequency_df)}")
    for col in categorical_cols:
        n_cat = len(frequency_df[frequency_df['Variable'] == col])
        print(f"   - {col}: {n_cat} categories")
    
    # ===============================================================
# 3Gb) Categorical Frequency Bar Charts (BONUS)
# ===============================================================

if DO_EDA_PLOTS: 
    
    print("\n=== Creating Categorical Frequency Bar Charts ===")
    
    # Grid boyutu
    n_cols_grid = 2
    n_rows_grid = int(np.ceil(len(categorical_cols) / n_cols_grid))
    
    # Bar chart grid
    fig, axes = plt. subplots(n_rows_grid, n_cols_grid, figsize=(14, 5 * n_rows_grid))
    axes = axes.ravel() if n_rows_grid * n_cols_grid > 1 else [axes]
    
    for i, col in enumerate(categorical_cols):
        ax = axes[i]
        
        # Frekans hesapla
        counts = data[col].value_counts()
        
        # Bar chart çiz
        counts.plot(kind='bar', ax=ax, color='steelblue', edgecolor='black', alpha=0.8)
        
        # Başlık ve etiketler
        ax.set_title(f"{col}\n(n={len(data)})", fontsize=14, fontweight='bold')
        ax.set_xlabel('Category', fontsize=13)
        ax.set_ylabel('Frequency', fontsize=13)
        ax.tick_params(axis='x', rotation=45, labelsize=12)
        ax.grid(axis='y', linestyle=':', alpha=0.5)
        
        # Yüzde değerleri ekle (bar üstünde)
        for j, (cat, count) in enumerate(counts.items()):
            percentage = (count / len(data)) * 100
            ax.text(j, count + 50, f'{percentage:.1f}%', 
                   ha='center', va='bottom', fontsize=12, fontweight='bold')
    
    # Kullanılmayan subplotları kapat
    for j in range(len(categorical_cols), len(axes)):
        axes[j].axis('off')
    
    # Ana başlık
    fig.suptitle(
        "Categorical Variables - Frequency Distribution\nKategorik Değişkenlerin Frekans Dağılımı",
        fontsize=16,
        fontweight='bold',
        y=0.995
    )
    
    plt.tight_layout()
    
    # Kaydet
    plt.savefig(cat_dir / "categorical_frequency_barcharts.png", dpi=300, bbox_inches='tight')
    plt.close()
    
    print(f"✅ Bar charts saved to:")
    print(f"   {cat_dir / 'categorical_frequency_barcharts.png'}")


=== Categorical Frequency Analysis ===
Categorical columns: ['PreferredLoginDevice', 'PreferredPaymentMode', 'Gender', 'PreferedOrderCat', 'MaritalStatus']

=== Categorical Frequency Table ===


,Variable,Category,Frequency,Percentage
0,PreferredLoginDevice,Mobile Phone,2765,49.11
1,PreferredLoginDevice,Computer,1634,29.02
2,PreferredLoginDevice,Phone,1231,21.87
3,PreferredPaymentMode,Debit Card,2314,41.10
4,PreferredPaymentMode,Credit Card,1501,26.66
5,PreferredPaymentMode,E wallet,614,10.91
6,PreferredPaymentMode,UPI,414,7.35
7,PreferredPaymentMode,COD,365,6.48
8,PreferredPaymentMode,CC,273,4.85
9,PreferredPaymentMode,Cash on Delivery,149,2.65



✅ Categorical frequency analysis saved to:
   C:\Users\ulku\Desktop\ecommerce\featselect_optuna_final\eda_outputs\categorical\categorical_frequency_analysis.xlsx
   C:\Users\ulku\Desktop\ecommerce\featselect_optuna_final\eda_outputs\categorical\categorical_frequency_analysis.csv
   C:\Users\ulku\Desktop\ecommerce\featselect_optuna_final\eda_outputs\categorical\categorical_frequency_table.png

📊 Summary:
   Total categorical variables: 5
   Total categories: 21
   - PreferredLoginDevice: 3 categories
   - PreferredPaymentMode: 7 categories
   - Gender: 2 categories
   - PreferedOrderCat: 6 categories
   - MaritalStatus: 3 categories

=== Creating Categorical Frequency Bar Charts ===
✅ Bar charts saved to:
   C:\Users\ulku\Desktop\ecommerce\featselect_optuna_final\eda_outputs\categorical\categorical_frequency_barcharts.png


In [29]:
# ===============================================================
# 3H) Histogram — Distribution of Numerical Features
# ===============================================================

if DO_EDA_PLOTS:    
    
    print("\n=== Creating Histograms for Numerical Features ===")
    
    # Veri
    data = df_clean.copy()
    
    # Sayısal sütunları otomatik tespit et
    numerical_cols = data.select_dtypes(include=[np.number]).columns.tolist()
    
    # Hedef değişken ve CustomerID'yi çıkar
    if TARGET_COL in numerical_cols:  
        numerical_cols.remove(TARGET_COL)
    if 'CustomerID' in numerical_cols:  
        numerical_cols.remove('CustomerID')
    
    print(f"Numerical columns for histograms: {len(numerical_cols)}")
    
    # Grid boyutu hesapla (2 SÜTUN)  ← DEĞİŞTİ
    n_cols_grid = 2  # ✅ 4'ten 2'ye değiştirildi
    n_rows_grid = int(np.ceil(len(numerical_cols) / n_cols_grid))
    
    # Histogram grid (daha büyük boyut, word'e daha uygun)
    fig, axes = plt.subplots(n_rows_grid, n_cols_grid, figsize=(12, 4 * n_rows_grid))  # ✅ Boyut ayarlandı
    axes = axes.ravel() if n_rows_grid * n_cols_grid > 1 else [axes]
    
    for i, col in enumerate(numerical_cols):
        ax = axes[i]
        
        # Histogram çiz
        ax.hist(data[col].dropna(), bins=30, color='steelblue', edgecolor='black', alpha=0.7)
        
        # Başlık ve etiketler
        ax.set_title(col, fontsize=14, fontweight='bold')  # ✅ Font büyütüldü
        ax.set_xlabel('Value', fontsize=12)  # ✅ Font büyütüldü
        ax.set_ylabel('Frequency', fontsize=12)  # ✅ Font büyütüldü
        ax.grid(axis='y', linestyle=':', linewidth=0.4, alpha=0.7)
        
        # Ortalama ve median çizgileri
        mean_val = data[col].mean()
        median_val = data[col].median()
        ax.axvline(mean_val, color='red', linestyle='--', linewidth=2, label=f'Mean: {mean_val:.2f}')  # ✅ Kalınlık artırıldı
        ax. axvline(median_val, color='green', linestyle='-.', linewidth=2, label=f'Median: {median_val:.2f}')  # ✅ Kalınlık artırıldı
        ax.legend(fontsize=10, loc='upper right')  # ✅ Font büyütüldü
    
    # Kullanılmayan subplotları kapat
    for j in range(len(numerical_cols), len(axes)):
        axes[j].axis('off')
    
    # Ana başlık
    fig.suptitle(
        "Distribution of Numerical Features (Histograms)\nSayısal Değişkenlerin Dağılımı",
        fontsize=20,  # ✅ Font büyütüldü
        fontweight='bold',
        y=0.995  # ✅ Başlık konumu ayarlandı
    )
    
    plt.tight_layout()
    
    # Kaydet
    hist_dir = EDA_DIR / "distributions"
    hist_dir.mkdir(parents=True, exist_ok=True)
    plt.savefig(hist_dir / "histograms_numerical_features.png", dpi=300, bbox_inches='tight')
    plt.close()
    
    print(f"✅ Histograms saved to:")
    print(f"   {hist_dir / 'histograms_numerical_features.png'}")


=== Creating Histograms for Numerical Features ===
Numerical columns for histograms: 13
✅ Histograms saved to:
   C:\Users\ulku\Desktop\ecommerce\featselect_optuna_final\eda_outputs\distributions\histograms_numerical_features.png


In [ ]:
# ===============================================================
# 3I) Shapiro-Wilk Normality Test - Normallik testi
# ===============================================================

if DO_EDA_PLOTS: 
    
    print("\n=== Running Shapiro-Wilk Normality Tests ===")
    
    from scipy. stats import shapiro
    
    # Veri
    data = df_clean.copy()
    
    # Sayısal sütunları otomatik tespit et
    numerical_cols = data.select_dtypes(include=[np.number]).columns.tolist()
    
    # Hedef değişken ve CustomerID'yi çıkar
    if TARGET_COL in numerical_cols:  
        numerical_cols.remove(TARGET_COL)
    if 'CustomerID' in numerical_cols:  
        numerical_cols.remove('CustomerID')
    
    print(f"Testing normality for {len(numerical_cols)} features...")
    
    # Normallik testi sonuçları
    normality_results = []
    
    for col in numerical_cols:
        # Eksik verileri çıkar
        data_col = data[col].dropna()
        
        # Shapiro-Wilk testi (maksimum 5000 örnek kullan, büyük veri setleri için)
        if len(data_col) > 5000:
            data_col_sample = data_col.sample(n=5000, random_state=42)
            stat, p = shapiro(data_col_sample)
            note = "Sampled (n=5000)"
        else:
            stat, p = shapiro(data_col)
            note = "Full data"
        
        # Sonuç
        is_normal = "Yes" if p > 0.05 else "No"
        
        normality_results.append({
            'Feature': col,
            'Shapiro_Stat': round(stat, 4),
            'p_value': f"{p:.4f}" if p >= 0.001 else "< 0.001",
            'Normal_Distribution': is_normal,
            'Note': note
        })
    
    # DataFrame'e dönüştür
    normality_df = pd.DataFrame(normality_results)
    
    print("\n=== Shapiro-Wilk Normality Test Results ===")
    try:
        display(normality_df)
    except NameError:  
        print(normality_df)
    
    # Klasör oluştur
    norm_dir = EDA_DIR / "normality"
    norm_dir.mkdir(parents=True, exist_ok=True)
    
    # Excel'e kaydet
    normality_df.to_excel(norm_dir / "shapiro_wilk_test_results.xlsx", index=False)
    normality_df.to_csv(norm_dir / "shapiro_wilk_test_results.csv", index=False)
    
    # PNG tablo olarak kaydet
    fig, ax = plt.subplots(figsize=(12, len(normality_df) * 0.4 + 1.5))  # ✅ KONTROL EDİLDİ
    ax.axis('off')
    
    tbl = plt.table(
        cellText=normality_df.values,
        colLabels=normality_df.columns,
        loc='center',
        cellLoc='center'
    )
    tbl.auto_set_font_size(False)
    tbl.set_fontsize(9)
    tbl.scale(1, 1.4)
    
    # Başlık ekle
    plt.title(
        "Shapiro-Wilk Normality Test Results\n(Normallik Testi Sonuçları)",
        fontsize=14,
        fontweight='bold',
        pad=12
    )
    
    plt.savefig(norm_dir / "shapiro_wilk_test_table.png", dpi=250, bbox_inches='tight')
    plt.close()
    
    print(f"\n✅ Shapiro-Wilk test results saved to:")
    print(f"   {norm_dir / 'shapiro_wilk_test_results.xlsx'}")
    print(f"   {norm_dir / 'shapiro_wilk_test_results.csv'}")
    print(f"   {norm_dir / 'shapiro_wilk_test_table.png'}")
    
    # Özet istatistik
    n_normal = len(normality_df[normality_df['Normal_Distribution'] == 'Yes'])
    n_not_normal = len(normality_df[normality_df['Normal_Distribution'] == 'No'])
    
    print(f"\n📊 Summary:")
    print(f"   Normal distribution: {n_normal}/{len(numerical_cols)}")
    print(f"   Non-normal distribution: {n_not_normal}/{len(numerical_cols)}")


=== Running Shapiro-Wilk Normality Tests ===
Testing normality for 13 features...

=== Shapiro-Wilk Normality Test Results ===


,Feature,Shapiro_Stat,p_value,Normal_Distribution,Note
0,Tenure,0.9245,< 0.001,No,Sampled (n=5000)
1,CityTier,0.6207,< 0.001,No,Sampled (n=5000)
2,WarehouseToHome,0.8553,< 0.001,No,Sampled (n=5000)
3,HourSpendOnApp,0.8231,< 0.001,No,Sampled (n=5000)
4,NumberOfDeviceRegistered,0.8959,< 0.001,No,Sampled (n=5000)
5,SatisfactionScore,0.8851,< 0.001,No,Sampled (n=5000)
6,NumberOfAddress,0.8815,< 0.001,No,Sampled (n=5000)
7,Complain,0.5634,< 0.001,No,Sampled (n=5000)
8,OrderAmountHikeFromlastYear,0.9176,< 0.001,No,Sampled (n=5000)
9,CouponUsed,0.7119,< 0.001,No,Sampled (n=5000)



✅ Shapiro-Wilk test results saved to:
   C:\Users\ulku\Desktop\ecommerce\featselect_optuna_final\eda_outputs\normality\shapiro_wilk_test_results.xlsx
   C:\Users\ulku\Desktop\ecommerce\featselect_optuna_final\eda_outputs\normality\shapiro_wilk_test_results.csv
   C:\Users\ulku\Desktop\ecommerce\featselect_optuna_final\eda_outputs\normality\shapiro_wilk_test_table.png

📊 Summary:
   Normal distribution: 0/13
   Non-normal distribution: 13/13


In [68]:
# ===============================================================
# 4) Utility Functions (metrics, CI, stats, plotting helpers)
#    E-commerce Churn: class 0 = Active, class 1 = Churned
# ===============================================================
# Bu blokta, tekrarlı deneyler sonucunda elde edilen sınıflandırma
# metriklerini (duyarlılık, özgüllük, F1, MCC, vb.) hesaplamak,
# güven aralıkları oluşturmak, modelleri istatistiksel olarak
# karşılaştırmak ve özet confusion matrix grafikleri üretmek için
# kullanılacak yardımcı fonksiyonlar tanımlanmaktadır.
#
# In this block, we define helper functions to compute classification
# metrics (sensitivity, specificity, F1, MCC, etc.), build confidence
# intervals, perform statistical model comparisons (Friedman + Wilcoxon-Holm),
# and plot aggregated confusion matrices for all models.
# ===============================================================


def safe_div(a, b):
    """
    Sıfıra bölmeyi güvenli şekilde ele alır.
    Payda 0 ise NaN döndürür, aksi halde a/b hesaplar.

    Safely handle division by zero.
    Returns NaN if denominator is 0, otherwise returns a/b.
    """
    return np.nan if b == 0 else a / b

# Not: Buradaki tp/fp/fn/tn, confusion_matrix(y_true, y_pred, labels=[0,1]) ile elde edilen (0=Active, 1=Churned) sırasına göre düşünülmüştür.

def compute_row_metrics(tn, fp, fn, tp):
    """
    Confusion matrix elemanlarından temel sınıflandırma metriklerini hesaplar.
    Burada sınıf etiketleri:
        class 0 = Active (NEG_LABEL)
        class 1 = Churned (POS_LABEL)

    Compute common binary classification metrics from confusion matrix entries.
    Assume:
        class 0 = Active (NEG_LABEL)
        class 1 = Churned (POS_LABEL)
    """
    tn = float(tn); fp = float(fp); fn = float(fn); tp = float(tp)

    sens = safe_div(tp, tp + fn)   # sensitivity / recall (for positive class = churn)
    spec = safe_div(tn, tn + fp)   # specificity (for negative class = active)
    prec = safe_div(tp, tp + fp)   # precision (positive predictive value)
    rec  = sens

    # F1-score
    if np.isnan(prec) or np.isnan(rec) or (prec + rec) == 0:
        f1 = np.nan
    else:
        f1 = 2 * prec * rec / (prec + rec)

    # Matthews correlation coefficient (MCC)
    denom_mcc = (tp + fp) * (tp + fn) * (tn + fp) * (tn + fn)
    mcc = np.nan if denom_mcc <= 0 else ((tp * tn) - (fp * fn)) / np.sqrt(denom_mcc)

    # Geometric mean of sensitivity and specificity (G-Mean)
    gmean = np.nan if (np.isnan(sens) or np.isnan(spec)) else np.sqrt(sens * spec)

    # Accuracy
    total = tn + fp + fn + tp
    acc = safe_div(tn + tp, total)

    return sens, spec, prec, rec, f1, mcc, gmean, acc


def mean_std_ci_t(x, alpha=0.05):
    """
    Bir dizi ölçüm için ortalama, standart sapma ve t-tabanlı
    (1-alpha)*100% güven aralığını hesaplar.

    Compute mean, standard deviation and t-based (1-alpha)*100%
    confidence interval for a given set of measurements.
    """
    x = np.asarray(x, dtype=float)
    x = x[~np.isnan(x)]
    n = x.size

    m = float(np.mean(x)) if n > 0 else np.nan
    s = float(np.std(x, ddof=1)) if n > 1 else np.nan

    if n > 1:
        tcrit = st.t.ppf(1 - alpha/2, df=n-1)
        half = tcrit * s / np.sqrt(n)
        lo, hi = m - half, m + half
    else:
        lo, hi = np.nan, np.nan

    return m, s, lo, hi, int(n)


def paired_ttest_ci(a, b, alpha=0.05):
    """
    Aynı koşullarda elde edilen iki modelin sonuçları için eşleştirilmiş
    t-testi ve güven aralığı hesaplar (örneğin aynı run'lar üzerindeki
    iki modelin AUC değerleri).

    Perform paired t-test + CI for two paired samples (e.g., two models
    evaluated on the same runs, using AUC or F1).
    Returns:
        n, mean_diff, ci95, t, p
    """
    a = np.asarray(a, dtype=float)
    b = np.asarray(b, dtype=float)

    n_all = min(a.size, b.size)
    a = a[:n_all]; b = b[:n_all]

    mask = (~np.isnan(a)) & (~np.isnan(b))
    a = a[mask]; b = b[mask]
    n = a.size

    if n <= 1:
        return {
            'n': int(n),
            'mean_diff': np.nan,
            'ci95': (np.nan, np.nan),
            't': np.nan,
            'p': np.nan
        }

    d = a - b
    md = float(np.mean(d))
    sd = float(np.std(d, ddof=1))

    tstat, pval = st.ttest_1samp(d, popmean=0.0)
    tcrit = st.t.ppf(1 - alpha/2, df=n - 1)
    half = tcrit * sd / np.sqrt(n)
    ci = (md - half, md + half)

    return {
        'n': int(n),
        'mean_diff': md,
        'ci95': ci,
        't': float(tstat),
        'p': float(pval)
    }


def friedman_and_pairwise(df_long, metric, alpha=0.05):
    """
    Birden fazla modeli, birden çok run üzerinden karşılaştırmak için
    Friedman testi ve Wilcoxon-Holm post-hoc analizini uygular.

    Apply Friedman test + Wilcoxon-Holm post-hoc analysis to compare
    multiple models over multiple runs.

    Parameters
    ----------
    df_long : pd.DataFrame
        En az şu sütunları içermelidir:
            ['run', 'model', metric]
        Should contain at least:
            ['run', 'model', metric]
    metric : str
        Karşılaştırılacak metrik sütununun adı (örn. 'AUC', 'F1', 'Accuracy').
        Name of the metric column (e.g., 'AUC', 'F1', 'Accuracy').
    """
    pivot = df_long.pivot_table(index='run', columns='model', values=metric)
    pivot = pivot.dropna(axis=0, how='any')
    models = list(pivot.columns)

    if len(models) < 2 or pivot.shape[0] < 2:
        empty = {
            'friedman': {
                'stat': np.nan,
                'p': np.nan,
                'k': len(models),
                'n': pivot.shape[0]
            }
        }
        return empty, pd.DataFrame(
            columns=['metric','model_a','model_b','p_raw','mean_diff','p_holm','better_model']
        )

    # Friedman testi / Friedman test
    stat, p = friedmanchisquare(*[pivot[m].values for m in models])
    fried = {'stat': stat, 'p': p, 'k': len(models), 'n': pivot.shape[0]}

    # Pairwise Wilcoxon with Holm correction
    rows = []
    for i in range(len(models)):
        for j in range(i+1, len(models)):
            a, b = models[i], models[j]
            vals_a, vals_b = pivot[a].values, pivot[b].values
            try:
                _, p_raw = wilcoxon(vals_a, vals_b,
                                    zero_method='wilcox',
                                    correction=False,
                                    alternative='two-sided',
                                    mode='auto')
                diff_mean = float(np.mean(vals_a - vals_b))
            except ValueError:
                p_raw, diff_mean = np.nan, np.nan
            rows.append([metric, a, b, p_raw, diff_mean])

    pw = pd.DataFrame(rows, columns=['metric','model_a','model_b','p_raw','mean_diff'])

    mask = ~pw['p_raw'].isna()
    if mask.any():
        _, p_holm, _, _ = multipletests(pw.loc[mask, 'p_raw'].values, method='holm')
        pw.loc[mask, 'p_holm'] = p_holm
    else:
        pw['p_holm'] = np.nan

    pw['better_model'] = np.where(
        pw['mean_diff'] > 0, pw['model_a'],
        np.where(pw['mean_diff'] < 0, pw['model_b'], 'equal')
    )

    return {'friedman': fried}, pw


def normalize_cm(cm, mode=None):
    """
    Confusion matrix'i normalize eder.

        mode = None  -> normalize yok (ham sayılar)
        mode = 'row' -> her satır 1 olacak şekilde normalize
        mode = 'all' -> tüm elemanların toplamı 1 olacak şekilde normalize

    Normalize confusion matrix:

        mode = None  -> no normalization (raw counts)
        mode = 'row' -> rows sum to 1
        mode = 'all' -> all elements sum to 1
    """
    cm = np.asarray(cm, dtype=float)
    if mode is None:
        return cm
    if mode == "row":
        row_sum = cm.sum(axis=1, keepdims=True)
        row_sum[row_sum == 0] = 1.0
        return cm / row_sum
    if mode == "all":
        s = cm.sum()
        return cm / s if s > 0 else cm
    raise ValueError("normalize mode must be None, 'row' or 'all'.")


def format_cell(mean, std, normalized):
    """
    Confusion matrix hücrelerinde gösterilecek mean±std metnini biçimlendirir.

    Format mean±std text for confusion matrix cells.
    """
    return f"{mean:.3f}±{std:.3f}" if normalized else f"{mean:.1f}±{std:.1f}"


def aggregate_cm_mean_std(df_conf_all, normalize=None):
    """
    Çoklu run'lar boyunca her model için confusion matrix'leri
    toplayıp mean ve std hesaplar.

    Aggregate confusion matrices over multiple runs for each model.

    Parameters
    ----------
    df_conf_all : pd.DataFrame
        Şu sütunları içermelidir:
            ['model', 'tn', 'fp', 'fn', 'tp']

        Must contain:
            ['model', 'tn', 'fp', 'fn', 'tp']
    """
    out = {}
    for model, g in df_conf_all.groupby("model"):
        mats = []
        for _, r in g.iterrows():
            cm = np.array([[r['tn'], r['fp']],
                           [r['fn'], r['tp']]], dtype=float)
            mats.append(normalize_cm(cm, normalize))
        if not mats:
            continue
        arr = np.stack(mats, axis=0)
        out[model] = {
            'mean': arr.mean(axis=0),
            'std':  arr.std(axis=0, ddof=1) if arr.shape[0] > 1 else np.zeros((2,2)),
            'n':    int(arr.shape[0])
        }
    return out


def plot_confusion_mean_std(agg, normalize=None, classes=None, out_path=None, cmap="Blues"):
    """
    Tüm modeller için, run'lar üzerinden ortalama ± std değerlerini içeren
    confusion matrix grafikleri çizer.

    Plot annotated confusion matrices (mean ± std) for all models.

    E-commerce churn bağlamı:
        class 0 = Active
        class 1 = Churned
    """
    models = list(agg.keys())
    if not models:
        raise ValueError("No models to plot.")
    n = len(models)
    n_cols = 2 if n == 4 else min(3, n)
    n_rows = int(np.ceil(n / n_cols))
    figsize = (5*n_cols, 4.5*n_rows)
    normalized = (normalize is not None)

    # Default labels for churn problem
    if classes is None:
        classes = ["Active", "Churned"]

    fig, axes = plt.subplots(n_rows, n_cols, figsize=figsize)
    if n_rows == 1 and n_cols == 1:
        axes = np.array([[axes]])
    elif n_rows == 1:
        axes = np.array([axes])
    elif n_cols == 1:
        axes = np.array([[ax] for ax in axes])

    vmin, vmax = (0.0, 1.0) if normalized else (None, None)

    idx = 0
    for r in range(n_rows):
        for c in range(n_cols):
            ax = axes[r, c]
            if idx >= n:
                ax.axis("off")
                continue

            model = models[idx]
            M, S, n_runs = agg[model]['mean'], agg[model]['std'], agg[model]['n']

            im = ax.imshow(M, cmap=cmap, vmin=vmin, vmax=vmax)

            for i in range(2):
                for j in range(2):
                    ax.text(
                        j, i,
                        format_cell(M[i, j], S[i, j], normalized),
                        color="black", ha="center", va="center", fontsize=14,
                        zorder=10
                    )
            
            
            ax.grid(False)              # ✅ major grid OFF (çizgi-yazı problemi buradan geliyor)
            ax.set_axisbelow(True)      # ✅ grid çizgileri yazıların ALTINDA kalsın

            ax.set_xticks([0, 1]); ax.set_yticks([0, 1])
            

            ax.set_xticklabels(["Active 0", "Churned 1"])
            ax.set_yticklabels(["True 0", "True 1"])
            #ax.set_xlabel(f"0={classes[0]}, 1={classes[1]}")
            #ax.set_ylabel(f"0={classes[0]}, 1={classes[1]}")


            ax.set_title(f"{model} — mean±std (n={n_runs})")
            ax.set_xticks(np.arange(-.5, 2, 1), minor=True)
            ax.set_yticks(np.arange(-.5, 2, 1), minor=True)
            ax.grid(which='minor', color='w', linestyle='-', linewidth=1.5)
            ax.tick_params(which='minor', bottom=False, left=False)

            idx += 1

    # Colorbar
    cbar_ax = fig.add_axes([0.92, 0.15, 0.02, 0.7])
    fig.colorbar(axes[0, 0].images[0], cax=cbar_ax)

    supt = "Confusion Matrices — Active vs Churned (mean ± std)"
    if normalize == "row":
        supt += " (row-normalized)"
    elif normalize == "all":
        supt += " (global-normalized)"

    fig.suptitle(supt, y=0.98, fontsize=14)
    fig.tight_layout(rect=[0, 0, 0.90, 0.97])

    if out_path is not None:
        out_path = Path(out_path)
        out_path.parent.mkdir(parents=True, exist_ok=True)
        fig.savefig(out_path, dpi=200, bbox_inches="tight")
    plt.close(fig)



In [6]:
# ===============================================================
# 5) Optuna Objective & Param Sanitization  (FULL SAFE VERSION)
#     E-commerce Churn: 0 = Active, 1 = Churned
# ===============================================================
# Bu blokta:
#   - Random Forest, XGBoost, KNN ve Bagging için Optuna objective fonksiyonu
#   - Hiperparametre sözlüğü temizliği (param sanitization)
#   - Seçilen feature isimlerini güvenli şekilde çıkarma fonksiyonu
# tanımlanır.
#
# In this block we define:
#   - Optuna objective function for Random Forest, XGBoost, KNN, Bagging
#   - Parameter dictionary cleanup (param sanitization)
#   - Helper to safely extract selected feature names after preprocessing
# ===============================================================


def objective(trial, model_name, X_train, y_train, X_val, y_val, preprocessor_fixed, seed=None):
    """
    Optuna objective function for E-commerce churn classification.
    SAFE Optuna objective for E-commerce Churn classification.

    Includes:
      - Random Forest
      - XGBoost
      - SAFE KNN
      - SAFE Bagging (Tree / KNN)
      - Feature selection K ∈ [K_MIN, K_MAX] (clipped by n_features)

    Bu fonksiyon:
      - Verilen model adına göre (Random Forest, XGBoost, KNN, Bagging)
        uygun hiperparametre aralığını tanımlar,
      - Sabit bir preprocessor (Block 2'deki ColumnTransformer) + SelectKBest ile
        özellik seçimi yapar,
      - Yalnızca eğitim verisinde fit eder ve doğrulama (validation) setinde
        seçilen metriği (AP, AUC veya F1) döndürür.

    Not / Note:
    Recommended for this imbalanced churn dataset:
      - Set OPTIMIZE_FOR = "auc" in Block 0 (AUC is robust for imbalance).
      
      - Churn problemi dengesiz olduğundan, genelde AUC ile optimize etmek
        (OPTIMIZE_FOR = "auc") literatürle uyumludur.
    """

    rng_seed = (RANDOM_BASE if seed is None else int(seed))
    pos_cls = POS_LABEL  # global pozitif sınıf (1 = Churned)
    neg_cls = NEG_LABEL  # global negatif sınıf (0 = Active)

    # ============================================================
    # RANDOM FOREST
    # ============================================================
    if model_name == "Random Forest":
        params = {
            "n_estimators": trial.suggest_int("n_estimators", 200, 2000, step=200),
            "max_depth": trial.suggest_int("max_depth", 2, 80),
            "min_samples_split": trial.suggest_int("min_samples_split", 2, 50),
            "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 20),
            "max_features": trial.suggest_categorical(
                "max_features", ["sqrt", "log2", None, 0.3, 0.5, 0.7]
            ),
            "criterion": trial.suggest_categorical(
                "criterion", ["gini", "entropy", "log_loss"]
            ),
            "bootstrap": trial.suggest_categorical("bootstrap", [True, False]),
            "class_weight": "balanced",  # churn problemi için önemli
            "n_jobs": -1,
            "random_state": rng_seed,
        }
        clf = RandomForestClassifier(**params)

    # ============================================================
    # XGBOOST
    # ============================================================
    elif model_name == "XGBoost":
        # class 1 = Churned (positive class)
        pos_n = int(np.sum(y_train == pos_cls))
        neg_n = int(np.sum(y_train == neg_cls))
        spw_base = (neg_n / max(1, pos_n)) if pos_n > 0 else 1.0

        params = {
            "n_estimators": trial.suggest_int("n_estimators", 300, 3000, step=300),
            "max_depth": trial.suggest_int("max_depth", 2, 20),
            "learning_rate": trial.suggest_float("learning_rate", 1e-4, 0.3, log=True),
            "subsample": trial.suggest_float("subsample", 0.3, 1.0),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.3, 1.0),
            "min_child_weight": trial.suggest_float(
                "min_child_weight", 1e-3, 50.0, log=True
            ),
            "gamma": trial.suggest_float("gamma", 0.0, 10.0),
            "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 10.0, log=True),
            "reg_lambda": trial.suggest_float("reg_lambda", 1e-3, 10.0, log=True),
            # dengesiz veri için: neg/pos oranı etrafında tarama
            "scale_pos_weight": trial.suggest_float(
                "scale_pos_weight", spw_base * 0.5, spw_base * 2.0, log=True
            ),
            "eval_metric": "logloss",
            "tree_method": "hist",
            "n_jobs": -1,
            "random_state": rng_seed,
        }
        clf = XGBClassifier(**params)

    # ============================================================
    # SAFE KNN
    # ============================================================
    elif model_name == "KNN":
        # SAFE upper bound: global SAFE_K_CAP ve train boyutuyla sınırla
        safe_k = min(SAFE_K_CAP, len(X_train) - 1)

        params = {
            "n_neighbors": trial.suggest_int("n_neighbors", 1, safe_k),
            "weights": trial.suggest_categorical("weights", ["uniform", "distance"]),
            "p": trial.suggest_int("p", 1, 5),
            "leaf_size": trial.suggest_int("leaf_size", 10, 100),
            "metric": trial.suggest_categorical(
                "metric",
                ["minkowski", "euclidean", "manhattan", "chebyshev"],
            ),
        }
        clf = KNeighborsClassifier(**params)

    # ============================================================
    # SAFE BAGGING (Tree or KNN base estimator)
    # ============================================================
    elif model_name == "Bagging":
        base_choice = trial.suggest_categorical("base_estimator", ["tree", "knn"])

        if base_choice == "tree":
            base_est = DecisionTreeClassifier(
                max_depth=trial.suggest_int("max_depth", 1, 50),
                min_samples_split=trial.suggest_int("min_samples_split", 2, 50),
                min_samples_leaf=trial.suggest_int("min_samples_leaf", 1, 20),
                class_weight="balanced",
                random_state=rng_seed,
            )
        else:
            safe_k_bag = min(SAFE_K_CAP, len(X_train) - 1)
            base_est = KNeighborsClassifier(
                n_neighbors=trial.suggest_int("n_neighbors", 1, safe_k_bag),
                weights=trial.suggest_categorical(
                    "weights", ["uniform", "distance"]
                ),
            )

        params = {
            "n_estimators": trial.suggest_int("n_estimators", 50, 500, step=50),
            "max_samples": trial.suggest_float("max_samples", 0.3, 1.0),
            "max_features": trial.suggest_float("max_features", 0.3, 1.0),
            "bootstrap": trial.suggest_categorical("bootstrap", [True, False]),
            "bootstrap_features": trial.suggest_categorical(
                "bootstrap_features", [False, True]
            ),
            "n_jobs": -1,
            "random_state": rng_seed,
        }

        clf = BaggingClassifier(estimator=base_est, **params)

    else:
        raise ValueError(f"Unknown model: {model_name}")

    # ============================================================
    # FEATURE SELECTION  (K_MIN–K_MAX, veri boyutuna göre kırpılmış)
    # ============================================================
    n_feat = X_train.shape[1]

    # Block 0'da K_MIN ve K_MAX global olarak tanımlı (3 ve 18).
    k_min_eff = int(max(1, min(K_MIN, n_feat)))
    k_max_eff = int(min(K_MAX, n_feat))
    if k_max_eff < k_min_eff:
        k_max_eff = k_min_eff

    k_best = trial.suggest_int("k_best", k_min_eff, k_max_eff)

    selector = SelectKBest(score_func=f_classif, k=k_best)

    # Tam pipeline: preproc + feature selection + classifier
    pipe = Pipeline(
        [
            ("prep", preprocessor_fixed),  # Block 2: ColumnTransformer
            ("feat", selector),
            ("clf", clf),
        ]
    )

    # Sadece TRAIN üzerinde fit (validation seti sadece değerlendirme için)
    pipe.fit(X_train, y_train)

    # ============================================================
    # VALIDATION METRICS  (Churn: POS_LABEL = 1)
    # ============================================================
    y_pred = pipe.predict(X_val)
    f1_val = f1_score(y_val, y_pred, average="binary", pos_label=pos_cls)

    # Pozitif sınıf (Churn) için olasılık skorları
    clf_classes = pipe.named_steps["clf"].classes_
    if pos_cls in clf_classes:
        pos_idx = int(np.where(clf_classes == pos_cls)[0][0])
    else:
        # Güvenlik için fallback (ikili problemde genelde index 1)
        pos_idx = 1 if len(clf_classes) > 1 else 0

    proba_val = pipe.predict_proba(X_val)[:, pos_idx]

    ap_val = average_precision_score(y_val, proba_val)
    auc_val = roc_auc_score(y_val, proba_val)

    # Optuna dashboard / analiz için yan attr olarak sakla
    trial.set_user_attr("val_f1", float(f1_val))
    trial.set_user_attr("val_ap", float(ap_val))
    trial.set_user_attr("val_auc", float(auc_val))

    # Hangi metriğe göre optimize edeceğimizi seç (Block 0'da OPTIMIZE_FOR set)
    opt = str(OPTIMIZE_FOR).lower()

    if opt == "ap":
        return ap_val
    elif opt == "auc":
        return auc_val
    else:
        return f1_val



# ===============================================================
# PARAM SANITIZATION — FINAL CLEANUP BEFORE RETRAINING
# ===============================================================

def sanitize_params_for_final(model_name, best_params_all):
    """
    Optuna sonrası, modeli final olarak yeniden eğitmeden önce
    parametre sözlüğünü temizler.

    This function:
      - Removes pipeline-only keys (k_best, k_features, gamma_mode, base_estimator)
      - Enforces SAFE bounds for KNN hyperparameters (n_neighbors ≤ SAFE_K_CAP)
      - Returns (clean_params, base_choice) for Bagging
    """

    p = best_params_all.copy()

    # Feature selection kalıntılarını kaldır
    for k in ["k_best", "k_features"]:
        p.pop(k, None)

    # ------------- SAFE KNN CLEANUP -------------
    if model_name == "KNN":
        cleaned = {}
        for k, v in p.items():
            if k == "n_neighbors":
                cleaned[k] = min(v, SAFE_K_CAP)  # global cap ile uyumlu
            else:
                cleaned[k] = v
        return cleaned, None

    # ------------- SAFE BAGGING CLEANUP -------------
    if model_name == "Bagging":
        base_choice = p.pop("base_estimator", None)
        cleaned = {
            k: v
            for k, v in p.items()
            if k
            in [
                "n_estimators",
                "max_samples",
                "max_features",
                "bootstrap",
                "bootstrap_features",
                "n_jobs",
                "random_state",
            ]
        }
        return cleaned, base_choice

    # Diğer modeller için (RF, XGBoost) sadece FS anahtarlarını silmek yeterli
    return p, None



# ===============================================================
# FEATURE NAME EXTRACTION (after Preprocessing + SelectKBest)
# ===============================================================

def get_selected_feature_info(fitted_pipe: Pipeline, X_ref: pd.DataFrame):
    """
    Seçilen feature'ların isimlerini güvenli şekilde çıkarır.

    Uses:
      - Preprocessor (prep) -> get_feature_names_out
      - SelectKBest (feat)  -> get_support mask

    Returns:
      mask                : boolean seçilim maskesi (veya None)
      selected_raw_names  : preprocessor çıkışındaki orijinal isimler
      selected_clean_names: clean_names ile prefix'leri temizlenmiş isimler
      all_base_names      : preprocessor sonrası tüm feature isimleri
    """

    feat = fitted_pipe.named_steps.get("feat", None)
    prep = fitted_pipe.named_steps.get("prep", None)

    # 1) PREPROCESSOR sonrası feature isimlerini al
    # Get feature names after preprocessing
    if prep is not None and hasattr(prep, "get_feature_names_out"):
        try:
            base_names = list(prep.get_feature_names_out(X_ref.columns))
        except TypeError:
            base_names = list(prep.get_feature_names_out())
    else:
        base_names = list(X_ref.columns)

    # 2) SelectKBest maskesine göre seçilen feature'ları filtrele
    # Filter with SelectKBest mask (if available)
    if feat is not None and hasattr(feat, "get_support"):
        mask = feat.get_support()

        if len(mask) != len(base_names):
            warnings.warn(
                f"get_selected_feature_info: mask length {len(mask)} != "
                f"feature name length {len(base_names)}. Truncating to min length."
            )
            min_len = min(len(mask), len(base_names))
            mask = np.array(mask[:min_len])
            base_arr = np.array(base_names[:min_len])
        else:
            base_arr = np.array(base_names)

        sel_names = list(base_arr[mask])
    else:
        mask = None
        sel_names = base_names

    # 3) Transformer prefix'lerini temizle (num__ / cat__ vb.)
    #    Clean transformer prefixes in feature names (num__, cat__, etc.)
    sel_names_clean = clean_names(sel_names)

    return mask, sel_names, sel_names_clean, base_names



In [ ]:
# ===============================================================
# 6) Main Loop: Splits + Optuna + Test Evaluation (SAFE VERSION)
#     E-commerce Churn (0 = Active, 1 = Churned)
# ===============================================================
# Bu blokta:
#   - OUTER + INNER StratifiedShuffleSplit yapısı
#   - Her rep için:
#       * Outer: TrainVal (%80) + Test (%20)
#       * Inner: Train + Validation (TrainVal içinde)
#   - Modeller: Random Forest, XGBoost, KNN, Bagging
#   - Optuna ile hiperparametre optimizasyonu (Her model için Optuna ile hiperparametre aranır.)
#   - Seçilen en iyi parametreyle TrainVal üzerinde yeniden eğitim (En iyi parametrelerle model yeniden eğitilir ve test setinde değerlendirilir.)
#   - Test setinde (hold-out) nihai değerlendirme
#   - Tüm metriklerin ve confusion matrix’lerin loglanması
#   - Sonuçlar tablo halinde kaydedilir ve özetlenir.
#
# In this block:
#   - We use the SAME OUTER + INNER StratifiedShuffleSplit.
#   - For each repetition (rep):
#       * Outer split: TrainVal (80%) + Test (20%)
#       * Inner split: Train + Validation within TrainVal
#   - Models: Random Forest, XGBoost, KNN, Bagging
#   - Hyperparameters optimized via Optuna (Hyperparameters are optimized with Optuna.)
#   - Retrain best model on full TrainVal (The best model is retrained and evaluated on the test set.)
#   - Evaluate on held-out Test
#   - Log all metrics and confusion matrices for later tables/plots.
#   - All evaluation metrics are stored and summarized in tables.
# ===============================================================

# ---------------------------------------------------------------
# 6A) Global SAFE K ve model listeleri - Ortak klasörler ve yapı
# ---------------------------------------------------------------
SAFE_K_CAP = 10   # Bagging-KNN ve KNN için global güvenli sınır (global safe cap)

# Kullanacağımız 4 model:
# Only the 4 models we keep:
model_names = ["KNN", "XGBoost", "Bagging", "Random Forest"]

# Sonuçları depolamak için yapılar
# Storage structures for results
best_hyperparameters    = {m: [] for m in model_names}
selected_features_store = {m: {} for m in model_names}

test_confusions = {m: [] for m in model_names}
test_roc_curves = {m: [] for m in model_names}
test_pr_curves  = {m: [] for m in model_names}

test_auc_list = {m: [] for m in model_names}
test_ap_list  = {m: [] for m in model_names}
test_f1_list  = {m: [] for m in model_names}

final_pipes       = {m: {} for m in model_names}
saved_model_paths = {m: [] for m in model_names}
split_store       = {}
ap_records        = []

# kullanılacak ek log tabloları
# Extra logging tables 
metrics_rows     = []   # tn, fp, fn, tp + derived metrics
conf_rows        = []   # raw confusion counts per model/rep
run_summary_rows = []   # full per-run summary (train/val/test sizes, times, params, etc.)

# Sonuç klasörü 
# Results folder 
RESULTS_DIR = BASE_OUT_DIR / "results"
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

# Model klasörü 
# Models folder 
MODELS_DIR  = BASE_OUT_DIR / "models"
MODELS_DIR.mkdir(parents=True, exist_ok=True)

# ---------------------------------------------------------------
# 6B) Optuna progress callback (tüm modeller için-isteğe bağlı canlı izleme)
# ---------------------------------------------------------------
# Bu callback, her trial sonrası Optuna tarafından çağrılır ve
# model adı + trial numarası + o trial’ın değeri + o ana kadarki en iyi
# değeri ekrana yazar. Eğitim sürecini canlı takip etmene yardımcı olur.
#
# This callback is called after each Optuna trial, printing:
# model name, trial number, current value, and best-so-far.
# It helps monitoring the training progress in VS Code / Jupyter.

def optuna_progress_callback(study, trial):
    model_name = study.user_attrs.get("model_name", "UNKNOWN")
    val = trial.value
    best = study.best_value
    print(
        f"[{model_name}] Trial {trial.number + 1} finished. "
        f"value={val:.4f}, best_so_far={best:.4f}"
    )


# ---------------------------------------------------------------
# 6C) Main Training Loop — OUTER + INNER SPLITS
# ---------------------------------------------------------------
# Blok yapısına sadık olarak veri setini böler: OUTER (TrainVal/Test) + INNER (Train/Val)
# We strictly follow for splitting the dataset into.

# ---------------------------------------------------------------
# Which dataset to use? (Hangi X, y kullanılacak?)
# ---------------------------------------------------------------
# Eğer Block 2 sonunda X_full / y_full oluşturduysak onları kullanIırız.
# If we created X_full / y_full at the end of Block 2, we use them.
if 'X_full' in globals() and 'y_full' in globals():
    X_data = X_full.copy()
    y_data = y_full.copy()
    print("Block 6 - Temizlenmiş tam veri seti")
# Eğer doğrudan X ve y varsa onları kullan.
# If raw X and y exist, use them directly.
elif "X" in globals() and "y" in globals():
    X_data = X.copy()
    y_data = y.copy()

    print("Using X / y for Block 6.")
else:
    raise RuntimeError(
        "Neither (X_full, y_full) nor (X, y) found. "
        "Please run Blocks 1–2 to create the dataset."
    )


for rep in range(N_REPS):

    # Farklı seviyeler için deterministik random seed’ler
    # Deterministic seeds for different levels
    seed_outer = int(RANDOM_BASE + 10007 * rep + 13)
    seed_inner = int(RANDOM_BASE + 20011 * rep + 97)
    seed_model = int(RANDOM_BASE + 30029 * rep + 211)
    seed_tpe   = int(RANDOM_BASE + 40039 * rep + 509)

    print("\n" + "#" * 80)
    print(f"### REP {rep + 1}/{N_REPS} — OUTER + INNER SPLITS (E-commerce Churn)")
    print("#" * 80)

    # ======================================================
    # OUTER SPLIT: TrainVal (1 - TEST_FRACTION) + Test (20% TEST)
    # ======================================================
    outer = StratifiedShuffleSplit(
        n_splits=1,
        test_size=TEST_FRACTION,       # Block 0'da tanımlı (örn. 0.2)
        random_state=seed_outer
    )


    (trainval_idx, test_idx), = outer.split(X_data, y_data)

    X_trainval = X_data.iloc[trainval_idx]
    y_trainval = y_data.iloc[trainval_idx]

    X_test = X_data.iloc[test_idx]
    y_test = y_data.iloc[test_idx]


    # İleride referans ve yeniden üretilebilirlik için indeksleri sakla
    # Store indices for reproducibility and later inspection
    split_store[rep] = {
        "trainval_idx": trainval_idx.tolist(), # burada train+val ayrımı
        "test_idx": test_idx.tolist()
    }


    print(f"- TrainVal size: {len(X_trainval)}  |  Test size: {len(X_test)}")

    # ======================================================
    # INNER SPLIT: Train + Validation (TrainVal içinde)
    # ======================================================
    inner = StratifiedShuffleSplit(
        n_splits=1,
        train_size=train_in_trainval,  # Block 0'da tanımlı oran
        test_size=val_in_trainval,     # Block 0'da tanımlı oran
        random_state=seed_inner
    )

    (tr_idx, val_idx), = inner.split(X_trainval, y_trainval)

    X_train = X_trainval.iloc[tr_idx]
    y_train = y_trainval.iloc[tr_idx]

    X_val = X_trainval.iloc[val_idx]
    y_val = y_trainval.iloc[val_idx]


    print(f"- Inner Train size: {len(X_train)} | Val size: {len(X_val)}")

    # Bu rep için preprocessor, SADECE train üzerinde fit edilecek şekilde
    # yapılandırılıyor (fit işlemi pipeline içinde).
    # Here we just build the structure; fitting happens inside the pipeline.
    PREPROC = build_preprocessor_all_robust(X_train)

    # ======================================================
    # OPTUNA LOOP — All Models in model_names
    # ======================================================
    for model_name in model_names:

        print("\n" + "=" * 70)
        print(f"REP {rep + 1}/{N_REPS}  —  Model: {model_name}")
        print("=" * 70)

        run_start = time.time()

        # ----------------- Optuna Study -------------------------
        study = optuna.create_study(
            direction="maximize",
            sampler=optuna.samplers.TPESampler(seed=seed_tpe),
            pruner=optuna.pruners.MedianPruner(n_warmup_steps=10)
        )
        study.set_user_attr("model_name", model_name)

        n_trials = N_TRIALS_BY_MODEL.get(model_name, 150)
        print(f"Optuna trials for {model_name}: {n_trials} | Optimize for: {OPTIMIZE_FOR.upper()}")

        study.optimize(
            lambda tr: objective(
                tr, model_name,
                X_train, y_train,
                X_val, y_val,
                preprocessor_fixed=PREPROC,
                seed=seed_model
            ),
            n_trials=n_trials,
            n_jobs=1,
            callbacks=[optuna_progress_callback]  # Çok gürültülü olursa bu satırı yorumlayabilirsin.
        )

        # ------------------------------------------------------
        # En iyi trial ve validation metrikleri
        # ------------------------------------------------------
        best_params_all = study.best_params.copy()
        best_trial      = study.best_trial

        best_val_f1  = float(best_trial.user_attrs.get("val_f1",  np.nan))
        best_val_ap  = float(best_trial.user_attrs.get("val_ap",  np.nan))
        best_val_auc = float(best_trial.user_attrs.get("val_auc", np.nan))

        best_hyperparameters[model_name].append({
            "rep": rep,
            "best_params": best_params_all.copy(),
            "val_f1":  best_val_f1,
            "val_ap":  best_val_ap,
            "val_auc": best_val_auc,
            "optimize_for": OPTIMIZE_FOR,
            "context": "E-commerce: Active (0) vs Churned (1)"
        })

        # Optuna parametrelerini, final eğitimden önce temizle
        # Sanitize Optuna params before final training
        clean_params, base_choice = sanitize_params_for_final(model_name, best_params_all)

        # ------------------------------------------------------
        # Feature Selection için K değeri
        # ------------------------------------------------------
        k_best = int(best_params_all.get("k_best", min(K_MAX, X_train.shape[1])))

        # ------------------------------------------------------
        # FINAL MODELİ OLUŞTUR (SAFE KNN & SAFE BAGGING dahil)
        # ------------------------------------------------------
        if model_name == "Random Forest":
            clf_best = RandomForestClassifier(
                **clean_params,
                n_jobs=-1,
                class_weight="balanced",
                random_state=seed_model
            )

        elif model_name == "XGBoost":
            clf_best = XGBClassifier(
                **clean_params,
                eval_metric="logloss",
                n_jobs=-1,
                random_state=seed_model
            )

        elif model_name == "KNN":
            # ABSOLUTE SAFETY for n_neighbors
            raw_k  = clean_params.get("n_neighbors", 5)
            safe_k = min(raw_k, max(1, len(X_train) - 1), SAFE_K_CAP)
            clean_params["n_neighbors"] = safe_k

            clf_best = KNeighborsClassifier(**clean_params)

        elif model_name == "Bagging":
            # --------------------------------------------------
            # BASE ESTIMATOR FIX (tree or knn) ===  Bagging mantığına göre ===
            # --------------------------------------------------
            if base_choice is None:
                base_choice = best_params_all.get("base_estimator", "tree")

            if base_choice == "tree":
                tree_params = {
                    k: best_params_all[k]
                    for k in best_params_all
                    if k in ["max_depth","min_samples_split","min_samples_leaf",
                             "criterion","splitter","max_features"]
                }
                base_est = DecisionTreeClassifier(
                    **tree_params,
                    class_weight="balanced",
                    random_state=seed_model
                )

            else:
                # SAFE BAGGING KNN
                raw_k  = best_params_all.get("n_neighbors", 5)
                safe_k = min(raw_k, SAFE_K_CAP, max(1, len(X_train) - 1))

                knn_params = {
                    k: best_params_all[k]
                    for k in best_params_all
                    if k in ["weights","p","leaf_size","metric"]
                }
                knn_params["n_neighbors"] = safe_k

                base_est = KNeighborsClassifier(**knn_params)

            # --------------------------------------------------
            # BAGGING PARAMETERS
            # --------------------------------------------------
            bag_params = {
                k: best_params_all[k]
                for k in best_params_all
                if k in ["n_estimators","max_samples","max_features",
                         "bootstrap","bootstrap_features","n_jobs","random_state"]
            }

            clf_best = BaggingClassifier(
                estimator=base_est,
                random_state=seed_model,
                **bag_params
            )

        else:
            raise ValueError(f"Unknown model in model_names: {model_name}")

        # ------------------------------------------------------
        # FINAL PIPELINE: PREP + SelectKBest + CLASSIFIER
        # ------------------------------------------------------
        selector_final = SelectKBest(score_func=f_classif, k=k_best)

        final_pipe = Pipeline([
            ("prep", PREPROC),
            ("feat", selector_final),
            ("clf",  clf_best)
        ])

        # TrainVal (X_trainval, y_trainval) üzerinde final eğitim
        # Final training on full TrainVal
        final_pipe.fit(X_trainval, y_trainval)
        final_pipes[model_name][rep] = final_pipe

        # ------------------------------------------------------
        # STORE SELECTED FEATURES - Seçilen feature'lar
        # ------------------------------------------------------
        mask, sel_names, sel_names_clean, base_names = get_selected_feature_info(final_pipe, X_data)

        selected_features_store[model_name][rep] = {
            "k_best": k_best,
            "selected_feature_names_clean": sel_names_clean
        }

        # -----------------------------------------------------------------
        # TEST METRICS (hold-out test set) - Test setinde değerlendirme
        # -----------------------------------------------------------------
        # Pozitif sınıf (churn) index’i
        clf_classes = final_pipe.named_steps["clf"].classes_
        if pos_label in clf_classes:
            pos_idx = int(np.where(clf_classes == pos_label)[0][0])
        else:
            pos_idx = 1 if len(clf_classes) > 1 else 0

        proba_test = final_pipe.predict_proba(X_test)[:, pos_idx]
        y_pred     = final_pipe.predict(X_test)

        # Ana metrikler
        auc_test = roc_auc_score(y_test, proba_test)
        ap_test  = average_precision_score(y_test, proba_test)
        f1_test  = f1_score(y_test, y_pred, pos_label=pos_label)

        # Confusion matrix
        cm = confusion_matrix(y_test, y_pred, labels=classes_sorted)
        tn, fp, fn, tp = cm.ravel()

        # Ek türetilmiş metrikler (sens, spec, mcc, gmean, acc)
        sens, spec, prec, rec, f1_ext, mcc, gmean, acc = compute_row_metrics(tn, fp, fn, tp)

        # ROC ve PR eğrileri
        fpr, tpr, _ = roc_curve(y_test, proba_test, pos_label=pos_label)
        prec_curve, rec_curve, _ = precision_recall_curve(y_test, proba_test, pos_label=pos_label)

        test_confusions[model_name].append(cm)
        test_roc_curves[model_name].append((fpr, tpr, auc_test))
        test_pr_curves[model_name].append((rec_curve, prec_curve, ap_test))

        test_auc_list[model_name].append(float(auc_test))
        test_ap_list[model_name].append(float(ap_test))
        test_f1_list[model_name].append(float(f1_test))

        # Validation + test kayıtları
        ap_records.append({
            "model": model_name,
            "rep": rep,
            "auc_test": float(auc_test),
            "ap_test":  float(ap_test),
            "f1_test":  float(f1_test),
            "val_f1":   float(best_val_f1),
            "val_ap":   float(best_val_ap),
            "val_auc":  float(best_val_auc),
            "context": "Active (0) vs Churned (1)",
            "optimize_for": OPTIMIZE_FOR
        })

        # Kısa özet çıktısı (ekran)
        # Short textual summary (on screen)
        print(f"[{model_name}] REP {rep + 1} TEST RESULTS:")
        print(f"  AUC = {auc_test:.4f} | AP = {ap_test:.4f} | F1 = {f1_ext:.4f} | ACC = {acc:.4f}")
        print(f"  Sens = {sens:.4f} | Spec = {spec:.4f} | MCC = {mcc:.4f} | G-Mean = {gmean:.4f}")

        # ---- Global loglar (tabloya çevirmek için) ----
        metrics_rows.append({
            "rep": rep,
            "model": model_name,
            "tn": tn, "fp": fp, "fn": fn, "tp": tp,
            "sens": sens, "spec": spec, "prec": prec, "rec": rec,
            "f1": f1_ext, "mcc": mcc, "gmean": gmean, "acc": acc,
            "ap": ap_test, "auc": auc_test
        })

        conf_rows.append({
            "rep": rep,
            "model": model_name,
            "tn": tn, "fp": fp, "fn": fn, "tp": tp
        })

        run_end = time.time()
        run_time_min = (run_end - run_start) / 60.0

        run_summary_rows.append({
            "model": model_name,
            "rep": rep,
            "n_trainval": len(X_trainval),
            "n_train": len(X_train),
            "n_val": len(X_val),
            "n_test": len(X_test),
            "auc": auc_test,
            "ap": ap_test,
            "f1": f1_ext,
            "accuracy": acc,
            "sensitivity": sens,
            "specificity": spec,
            "mcc": mcc,
            "gmean": gmean,
            "run_time_min": run_time_min,
            "best_params_raw": str(best_params_all),
            "best_params_clean": str(clean_params),
            "k_best": k_best,
            "seed_outer": seed_outer,
            "seed_inner": seed_inner,
            "seed_tpe": seed_tpe,
            "seed_model": seed_model,
        })

        # ----------------- Model kaydetme (opsiyonel) -----------------
        # Model saving (optional, controlled by SAVE_MODELS)
        if SAVE_MODELS:
            safe_model = model_name.replace(" ", "")
            fname = MODEL_DIR / (
                f"{safe_model}__rep{rep:02d}__EcommerceChurn__OBJ{OPTIMIZE_FOR}"
                f"__F1{f1_ext:.4f}__AUC{auc_test:.4f}__AP{ap_test:.4f}.joblib"
            )
            dump(final_pipe, fname, compress=3)
            saved_model_paths[model_name].append(str(fname))
            print(f"Final pipeline saved to: {fname}")


# ---------------------------------------------------------------
# 6D) Run özetlerinin Excel'e kaydedilmesi
# ---------------------------------------------------------------
# Bu kısım, Block 7 ve 8'de kullanacağımız özet tabloların temelini atar.
# This part saves the detailed run summaries for use in Block 7/8.

if len(run_summary_rows) > 0:
    df_run_summary = pd.DataFrame(run_summary_rows)
    excel_path_runs = RESULTS_DIR / "all_model_runs_detailed_MAIN.xlsx"
    df_run_summary.to_excel(excel_path_runs, index=False)
    print(f"\nDetaylı MAIN model run özeti Excel olarak kaydedildi: {excel_path_runs}")

    df_metrics = pd.DataFrame(metrics_rows)
    excel_path_metrics = RESULTS_DIR / "all_model_test_metrics_MAIN.xlsx"
    df_metrics.to_excel(excel_path_metrics, index=False)
    print(f"Test metrikleri (tn,fp,fn,tp + türetilmiş) Excel olarak kaydedildi: {excel_path_metrics}")


Block 6 - Temizlenmiş tam veri seti

################################################################################
### REP 1/10 — OUTER + INNER SPLITS (E-commerce Churn)
################################################################################
- TrainVal size: 4504  |  Test size: 1126
- Inner Train size: 3659 | Val size: 845

=== PREPROCESSOR COLUMN GROUPS ===
Numeric columns   : ['Tenure', 'CityTier', 'WarehouseToHome', 'HourSpendOnApp', 'NumberOfDeviceRegistered', 'SatisfactionScore', 'NumberOfAddress', 'Complain', 'OrderAmountHikeFromlastYear', 'CouponUsed', 'OrderCount', 'DaySinceLastOrder', 'CashbackAmount']
Categorical cols  : ['PreferredLoginDevice', 'PreferredPaymentMode', 'Gender', 'PreferedOrderCat', 'MaritalStatus']

REP 1/10  —  Model: KNN
Optuna trials for KNN: 150 | Optimize for: AUC
[KNN] Trial 1 finished. value=0.8372, best_so_far=0.8372
[KNN] Trial 2 finished. value=0.8506, best_so_far=0.8506
[KNN] Trial 3 finished. value=0.8898, best_so_far=0.8898
[KNN] Tria

In [9]:
print("model_names exists:", "model_names" in globals())
print("ap_records exists:", "ap_records" in globals())
print("length ap_records:", len(ap_records) if "ap_records" in globals() else "NO")
print("y exists:", "y" in globals())


model_names exists: True
ap_records exists: True
length ap_records: 40
y exists: True


In [15]:
print("model_names" in globals(),
      "ap_records" in globals(),
      "y" in globals())


True True True


In [14]:
# ===============================================================
# 6.5) Block 6 State Save — Block 7 için gerekli her şeyi diske kaydet
#      Save all Block 6 results needed by Block 7
# ===============================================================
#Bundan sonra Block 6 bittiği anda bütün önemli değişkenleri tek bir joblib dosyasına 
#kaydeden bir “Block 6.5 – SAVE STATE” eklemek.Yeni oturumda Block 6’yı tekrar koşturmadan,
#sadece bu dosyadan yükleyip Block 7’yi çalıştırmak.Bugünkü kaybı tamamen kurtaramıyoruz 
#ama bir kez daha (daha kısa ayarlarla da olabilir) çalıştırdığında, artık bir daha asla boşa gitmeyecek.
#Bu hücreyi Block 6 bittikten hemen sonra çalıştırırsan, bütün önemli şeyler tek dosyada saklanıyor olacak.


BLOCK6_STATE_PATH = BASE_OUT_DIR / "block6_state_ecommerce_churn.joblib"

block6_state = {
    "model_names":            model_names,
    "best_hyperparameters":   best_hyperparameters,
    "selected_features_store": selected_features_store,
    "test_confusions":        test_confusions,
    "test_roc_curves":        test_roc_curves,
    "test_pr_curves":         test_pr_curves,
    "test_auc_list":          test_auc_list,
    "test_ap_list":           test_ap_list,
    "test_f1_list":           test_f1_list,
    "final_pipes":            final_pipes,
    "saved_model_paths":      saved_model_paths,
    "split_store":            split_store,
    "ap_records":             ap_records,
}

dump(block6_state, BLOCK6_STATE_PATH, compress=3)

print("\n[INFO] Block 6 state saved to:")
print(" ", BLOCK6_STATE_PATH.resolve())



[INFO] Block 6 state saved to:
  C:\Users\ulku\Desktop\ecommerce\featselect_optuna_final\block6_state_ecommerce_churn.joblib


In [ ]:
# ===============================================================
# 6.5-LOAD) Block 6 State Load — Bunu da normlade kullanmıyoruz. !!!!!!!!
# Bunu Eğitimi yeniden yapmadan sonuçları yüklemek için kullanabilirsin
# ===============================================================
#Yeni oturumda “STATE YÜKLEME” hücresi (6.5-LOAD) Kernel reset/Ekranı kapattın diyelim.
# Bir dahaki sefere Block 0–2–3–4–5’i çalıştırmana gerek yok, sadece: 
# BASE_OUT_DIR tanımlı olsun (Block 0’dan geliyor).
# Bu hücreyi Block 7’den ÖNCE çalıştırıyoruz: Bu hücreyi çalıştırdığında:
# model_names, ap_records, test_auc_list vs. hepsi RAM’e geri gelir.
# Sonra Block 7 hücresini normal şekilde çalıştırabilirsin.

from joblib import load

BLOCK6_STATE_PATH = BASE_OUT_DIR / "block6_state_ecommerce_churn.joblib"

state = load(BLOCK6_STATE_PATH)

# Tüm kaydedilmiş değişkenleri global namespace'e al
for k, v in state.items():
    globals()[k] = v

print("[INFO] Block 6 state loaded from file.")
print("  Models:", model_names)
print("  #ap_records :", len(ap_records))
for m in model_names:
    print(f"   - {m}: #reps with AUC = {len(test_auc_list.get(m, []))}")


In [ ]:
# ===============================================================
# 7.0) GÜVENLİK AMAÇLI! (ASIL 7 SONRAKİ HÜCREDE) Bunu da normlade kullanmıyoruz. !!!!!!!!
# Altaki 7.bloğu kullanıyoruz. 
# ===============================================================
# Sadece Kernel sıfırlandı ve uygulama restart edildi ya da 
# terminal kaptıldıysa öncesini geri getirmek için kullanırız 
# yani tekrar 6.blokta eğitim yapmamak ve 5 saat beklememek için. 

# Run-level Tables, Summaries, ROC/PR (Mean ± Std)
# E-commerce Churn (0 = Active, 1 = Churned)
# ===============================================================
#Block 7’nin başına küçük güvenlik ekle (opsiyonel ama güzel) 
# Şu anki 7. blokta en başa (ilk satırlara) küçük bir kontrol ekleyebilirsin; 
# eğer ap_records yoksa otomatik olarak dosyadan yüklemeyi dener:

from joblib import load

# Eğer Block 6 daha önce bu oturumda çalıştırılmadıysa, state dosyasından yüklemeyi dene
if "ap_records" not in globals() or "model_names" not in globals():
    BLOCK6_STATE_PATH = BASE_OUT_DIR / "block6_state_ecommerce_churn.joblib"
    try:
        state = load(BLOCK6_STATE_PATH)
        for k, v in state.items():
            globals()[k] = v
        print("[INFO] Block 6 state auto-loaded for Block 7 from:", BLOCK6_STATE_PATH)
    except FileNotFoundError:
        raise RuntimeError(
            "ap_records tanımlı değil ve 'block6_state_ecommerce_churn.joblib' bulunamadı.\n"
            "En az bir kez Block 6'yı çalıştırıp 6.5 SAVE STATE hücresini çalıştırman gerekiyor."
        )

#Bundan sonra: Eğer Block 6 aynı oturumda yeni bitti → Zaten değişkenler RAM’de, yüklemeye gerek kalmaz.
# Eğer yeni oturumdaysan → Block 7 otomatik block6_state_ecommerce_churn.joblib dosyasını açıp 
# her şeyi RAM’e yükler.

In [ ]:
# ===============================================================
# 7) Run-level Tables, Summaries, ROC/PR (Mean ± Std)
#     E-commerce Churn (0 = Active, 1 = Churned)
# ===============================================================
# Bu blokta:
#   - Her tekrar (rep) ve model için test F1 / AP / AUC skorlarını kaydederiz.
#   - AUC / AP / F1 için ortalama ± std özet tablosu oluştururuz.
#   - En iyi hiperparametreleri ve seçilen feature setlerini JSON olarak kaydederiz.
#   - Tüm modeller için ROC ve PR eğrilerini (mean ± 1 SD) çizeriz.
#
# Not:
#   Bu  çalışmada, veri seti dengesiz olduğu için ana performans metriği
#   olarak AUC (ROC AUC) baz alınmaktadır (OPTIMIZE_FOR = 'auc').
#
# In this block we:
#   - Save per-run test F1 / AP / AUC scores for each model.
#   - Build mean±std summary tables for AUC / AP / F1.
#   - Save best hyperparameters and selected feature sets as JSON.
#   - Plot ROC and PR curves (mean ± 1 SD) for all models.
#
# Note:
#   Because the dataset is imbalanced, the primary performance metric
#   used AUC (ROC AUC) (OPTIMIZE_FOR = 'auc').
# ===============================================================

# ---------------------------------------------------------------
# 7A) Model listesi — Block 6 ile uyumlu olmalı
#     Model list — must match Block 6
# ---------------------------------------------------------------
if "model_names" in globals():
    models_for_summary = model_names  # ["KNN", "XGBoost", "Bagging", "Random Forest"]
elif "test_auc_list" in globals() and len(test_auc_list) > 0:
    # Eğer kernel reset sonrası sadece sonuç dict'leri kaldıysa buradan türet
    models_for_summary = list(test_auc_list.keys())
    print("\n[WARNING] 'model_names' bulunamadı. "
          "Model listesi test_auc_list anahtarlarından türetildi:")
    print("  ", models_for_summary)
else:
    # Son çare: kullandığın 4 ana modeli varsay
    models_for_summary = ["KNN", "XGBoost", "Bagging", "Random Forest"]
    print("\n[WARNING] 'model_names' ve sonuç listeleri bulunamadı. "
          "Varsayılan model listesi kullanılıyor:")
    print("  ", models_for_summary)

print("\n=== BLOCK 7: Run-level summaries & curves (E-commerce Churn) ===")
print("Models included:", models_for_summary)



# ---------------------------------------------------------------
# 7B) Run-level skor tabloları (F1 / AP / AUC) — CSV + Excel
#     Run-level score tables (per rep & model)
# ---------------------------------------------------------------
# ap_records, Block 6'da her rep/model için dolduruldu.
# ap_records was filled in Block 6 (one row per rep per model).

if len(ap_records) == 0:
    print("\n[WARNING] 'ap_records' boş veya tanımlı değil. Block 6 çalıştırılmamış olabilir.")
else:
    df_scores = (
        pd.DataFrame(ap_records)
        .sort_values(["model", "f1_test"])
        .reset_index(drop=True)
    )

    # F1'e göre sıralanmış skorlar (AUC / AP / F1 kolonları zaten ap_records içinde)
    # Scores sorted by F1 (AUC / AP / F1 columns already in ap_records)
    f1_csv_path = OUT_DIR / "scores_f1_ecommerce_churn.csv"
    df_scores.to_csv(f1_csv_path, index=False)

    # tablolarda kullanmak için Excel çıktısı
    f1_xlsx_path = OUT_DIR / "scores_f1_ecommerce_churn.xlsx"
    df_scores.to_excel(f1_xlsx_path, index=False)

    # AP'e göre sıralı ayrı bir CSV
    df_scores_ap = (
        pd.DataFrame(ap_records)
        .sort_values(["model", "ap_test"])
        .reset_index(drop=True)
    )
    ap_csv_path = OUT_DIR / "scores_ap_ecommerce_churn.csv"
    df_scores_ap.to_csv(ap_csv_path, index=False)

    # AUC'e göre sıralı ayrı bir CSV (özellikle AUC baz alındığı için)
    df_scores_auc = (
        pd.DataFrame(ap_records)
        .sort_values(["model", "auc_test"])
        .reset_index(drop=True)
    )
    auc_csv_path = OUT_DIR / "scores_auc_ecommerce_churn.csv"
    df_scores_auc.to_csv(auc_csv_path, index=False)

    print("\n[INFO] Run-level F1 / AP / AUC score tables saved:")
    print("  - F1-sorted CSV :", f1_csv_path)
    print("  - F1-sorted XLSX:", f1_xlsx_path)
    print("  - AP-sorted CSV :", ap_csv_path)
    print("  - AUC-sorted CSV:", auc_csv_path)


# ---------------------------------------------------------------
# 7C) Özet tablo (AUC / AP / F1 için mean ± std)
#     Summary table (mean ± std for AUC / AP / F1)
# ---------------------------------------------------------------
rows_sum = []

for m in models_for_summary:
    auc_vals = test_auc_list.get(m, [])
    ap_vals  = test_ap_list.get(m, [])
    f1_vals  = test_f1_list.get(m, [])

    rows_sum.append({
        "model":    m,
        "AUC_mean": np.mean(auc_vals) if len(auc_vals) > 0 else np.nan,
        "AUC_std":  np.std(auc_vals, ddof=1) if len(auc_vals) > 1 else 0.0,
        "AP_mean":  np.mean(ap_vals)  if len(ap_vals)  > 0 else np.nan,
        "AP_std":   np.std(ap_vals,  ddof=1) if len(ap_vals)  > 1 else 0.0,
        "F1_mean":  np.mean(f1_vals) if len(f1_vals) > 0 else np.nan,
        "F1_std":   np.std(f1_vals,  ddof=1) if len(f1_vals) > 1 else 0.0,
        "N_reps":   len(auc_vals)
    })

df_summary = pd.DataFrame(rows_sum)

summary_csv_path  = OUT_DIR / "summary_ecommerce_churn_mean_std.csv"
summary_xlsx_path = OUT_DIR / "summary_ecommerce_churn_mean_std.xlsx"

df_summary.to_csv(summary_csv_path, index=False)
df_summary.to_excel(summary_xlsx_path, index=False)

print("\n[INFO] Summary table (AUC / AP / F1 mean±std) saved:")
print("  - CSV :", summary_csv_path)
print("  - XLSX:", summary_xlsx_path)


# ---------------------------------------------------------------
# 7D) Hiperparametreler ve seçilen feature bilgileri
#     Hyperparameters & selected feature info
# ---------------------------------------------------------------
# best_hyperparameters ve selected_features_store, Block 6'da dolduruldu.
# We save them as JSON for later inspection /

hp_json_path   = OUT_DIR / "best_hyperparameters_ecommerce_churn.json"
feat_json_path = OUT_DIR / "selected_features_store_ecommerce_churn.json"

with open(hp_json_path, "w", encoding="utf-8") as f:
    json.dump(best_hyperparameters, f, ensure_ascii=False, indent=2)

with open(feat_json_path, "w", encoding="utf-8") as f:
    json.dump(selected_features_store, f, ensure_ascii=False, indent=2)

print("\n[INFO] Hyperparameters and selected features saved to JSON:")
print("  - HP JSON   :", hp_json_path)
print("  - FEATS JSON:", feat_json_path)


# ---------------------------------------------------------------
# 7E) Diagnostik objelerin joblib ile kaydı
#     Save diagnostic objects via joblib
# ---------------------------------------------------------------
diag_joblib_path = OUT_DIR / "diagnostics_ecommerce_churn.joblib"

dump(
    {
        "test_confusions":   test_confusions,
        "test_roc_curves":   test_roc_curves,
        "test_pr_curves":    test_pr_curves,
        "test_auc_list":     test_auc_list,
        "test_ap_list":      test_ap_list,
        "test_f1_list":      test_f1_list,
        "split_store":       split_store,
        "saved_model_paths": saved_model_paths
    },
    diag_joblib_path,
    compress=3
)

print("\n[INFO] Diagnostics object saved:")
print("  - JOBLIB:", diag_joblib_path)


# ===============================================================
# 7F) ROC CURVE (Mean ± Std) — E-commerce Churn
# ===============================================================
# ROC eğrileri: her model için tüm rep’lerdeki FPR–TPR noktalarını
# ortak bir FPR gridine (0–1 arası) projelendirip ortalama±std olarak çizer.
#
# ROC curves: for each model, interpolate all run-level ROC curves
# on a common FPR grid and plot mean±std.

fpr_grid = np.linspace(0, 1, 201)
plt.figure(figsize=(10, 8))

for m, curves in test_roc_curves.items():
    if m not in models_for_summary:
        continue
    if not curves:
        continue

    interp_tprs = []
    for (fpr, tpr, _) in curves:
        tpr_i = np.interp(fpr_grid, fpr, tpr)
        # Uç noktaları sabitle / ensure endpoints
        tpr_i[0], tpr_i[-1] = 0.0, 1.0
        interp_tprs.append(tpr_i)

    interp_tprs = np.vstack(interp_tprs)
    mean_tpr = interp_tprs.mean(axis=0)
    std_tpr  = interp_tprs.std(axis=0, ddof=1)

    auc_vals = test_auc_list.get(m, [])
    mean_auc = float(np.mean(auc_vals)) if len(auc_vals) > 0 else np.nan
    std_auc  = float(np.std(auc_vals, ddof=1)) if len(auc_vals) > 1 else 0.0

    plt.plot(
        fpr_grid,
        mean_tpr,
        label=f"{m} (AUC={mean_auc:.3f}±{std_auc:.3f})",
        linewidth=2
    )
    plt.fill_between(
        fpr_grid,
        np.maximum(mean_tpr - std_tpr, 0.0),
        np.minimum(mean_tpr + std_tpr, 1.0),
        alpha=0.15
    )

# Random classifier çizgisi / random classifier line
plt.plot([0, 1], [0, 1], linestyle="--", linewidth=1)

plt.xlabel(
    "False Positive Rate\n"
    "(Yanlış Pozitif Oranı — Active müşteri Churn olarak tahmin edildiğinde)"
)
plt.ylabel(
    "True Positive Rate (Sensitivity)\n"
    "(Doğru Pozitif Oranı — Gerçek Churn müşterileri yakalama oranı)"
)
plt.title(
    "E-commerce Customer Churn — ROC Curve (Mean ± 1 SD, Test Set)\n"
    "E-ticaret Müşteri Kaybı — ROC Eğrisi (Ortalama ± 1 SS, Test Kümesi)",
    fontsize=13,
    fontweight="semibold"
)
plt.legend(loc="lower right")
plt.grid(True)
plt.tight_layout()

roc_png_path = OUT_DIR / "roc_ecommerce_churn_mean_std.png"
plt.savefig(roc_png_path, dpi=200)
plt.close()

print("\n[INFO] ROC mean±std figure saved:")
print("  - PNG:", roc_png_path)


# ===============================================================
# 7G) PR CURVE (Mean ± Std) — E-commerce Churn
# ===============================================================
# PR eğrileri: recall ekseninde precision değerlerini ortalama±std
# olarak gösterir ve özellikle dengesiz veri setlerinde AUC-PR (AP)
# metrikleri için daha bilgilendirici bir görseldir.
#
# PR curves: show mean±std precision as a function of recall.
# Especially useful for imbalanced datasets (like churn).

recall_grid = np.linspace(0, 1, 201)
plt.figure(figsize=(10, 8))

for m, curves in test_pr_curves.items():
    if m not in models_for_summary:
        continue
    if not curves:
        continue

    interp_precs = []
    for (rec, prec, _) in curves:
        # Recall değerlerini artan sırada olacak şekilde sıralayıp interpolasyon yap / sort by recall
        order = np.argsort(rec)
        rec_sorted  = rec[order]
        prec_sorted = prec[order]

        prec_i = np.interp(
            recall_grid,
            rec_sorted,
            prec_sorted,
            left=prec_sorted[0],
            right=prec_sorted[-1]
        )
        interp_precs.append(prec_i)

    interp_precs = np.vstack(interp_precs)
    mean_prec = interp_precs.mean(axis=0)
    std_prec  = interp_precs.std(axis=0, ddof=1)

    ap_vals = test_ap_list.get(m, [])
    mean_ap = float(np.mean(ap_vals)) if len(ap_vals) > 0 else np.nan
    std_ap  = float(np.std(ap_vals, ddof=1)) if len(ap_vals) > 1 else 0.0

    plt.plot(
        recall_grid,
        mean_prec,
        label=f"{m} (AP={mean_ap:.3f}±{std_ap:.3f})",
        linewidth=2
    )
    plt.fill_between(
        recall_grid,
        np.maximum(mean_prec - std_prec, 0.0),
        np.minimum(mean_prec + std_prec, 1.0),
        alpha=0.15
    )

plt.xlabel(
    "Recall (Sensitivity)\n"
    "(Duyarlılık — Gerçek Churn müşterilerin yakalanma oranı)"
)
plt.ylabel(
    "Precision (Positive Predictive Value)\n"
    "(Pozitif Tahmin Değeri — Churn tahminlerinin doğruluk oranı)"
)
plt.title(
    "E-commerce Customer Churn — Precision–Recall Curve (Mean ± 1 SD, Test Set)\n"
    "E-ticaret Müşteri Kaybı — Precision–Recall Eğrisi (Ortalama ± 1 SS, Test Kümesi)",
    fontsize=13,
    fontweight="semibold"
)
plt.legend(loc="lower left")
plt.grid(True)
plt.tight_layout()

pr_png_path = OUT_DIR / "pr_ecommerce_churn_mean_std.png"
plt.savefig(pr_png_path, dpi=200)
plt.close()

print("\n[OK] E-commerce churn run-level outputs saved under:")
print("  OUT_DIR:", OUT_DIR.resolve())



=== BLOCK 7: Run-level summaries & curves (E-commerce Churn) ===
Models included: ['KNN', 'XGBoost', 'Bagging', 'Random Forest']

[INFO] Run-level F1 / AP / AUC score tables saved:
  - F1-sorted CSV : C:\Users\ulku\Desktop\ecommerce\featselect_optuna_final\outputs_OBJ_auc\scores_f1_ecommerce_churn.csv
  - F1-sorted XLSX: C:\Users\ulku\Desktop\ecommerce\featselect_optuna_final\outputs_OBJ_auc\scores_f1_ecommerce_churn.xlsx
  - AP-sorted CSV : C:\Users\ulku\Desktop\ecommerce\featselect_optuna_final\outputs_OBJ_auc\scores_ap_ecommerce_churn.csv
  - AUC-sorted CSV: C:\Users\ulku\Desktop\ecommerce\featselect_optuna_final\outputs_OBJ_auc\scores_auc_ecommerce_churn.csv

[INFO] Summary table (AUC / AP / F1 mean±std) saved:
  - CSV : C:\Users\ulku\Desktop\ecommerce\featselect_optuna_final\outputs_OBJ_auc\summary_ecommerce_churn_mean_std.csv
  - XLSX: C:\Users\ulku\Desktop\ecommerce\featselect_optuna_final\outputs_OBJ_auc\summary_ecommerce_churn_mean_std.xlsx

[INFO] Hyperparameters and selecte

In [64]:
# 8. Bloğun en üstünde çalıştırarak, her rep’in toplam test örneğini otomatik kontrol ederiz:
print("\n=== Confusion Matrix Totals (Quick Check) ===")
for m, cms in test_confusions.items():
    print(f"\nModel: {m}")
    for rep, cm in enumerate(cms):
        total = cm.sum()
        print(f"  REP {rep+1}: total={total}")



=== Confusion Matrix Totals (Quick Check) ===

Model: KNN
  REP 1: total=1126
  REP 2: total=1126
  REP 3: total=1126
  REP 4: total=1126
  REP 5: total=1126
  REP 6: total=1126
  REP 7: total=1126
  REP 8: total=1126
  REP 9: total=1126
  REP 10: total=1126

Model: XGBoost
  REP 1: total=1126
  REP 2: total=1126
  REP 3: total=1126
  REP 4: total=1126
  REP 5: total=1126
  REP 6: total=1126
  REP 7: total=1126
  REP 8: total=1126
  REP 9: total=1126
  REP 10: total=1126

Model: Bagging
  REP 1: total=1126
  REP 2: total=1126
  REP 3: total=1126
  REP 4: total=1126
  REP 5: total=1126
  REP 6: total=1126
  REP 7: total=1126
  REP 8: total=1126
  REP 9: total=1126
  REP 10: total=1126

Model: Random Forest
  REP 1: total=1126
  REP 2: total=1126
  REP 3: total=1126
  REP 4: total=1126
  REP 5: total=1126
  REP 6: total=1126
  REP 7: total=1126
  REP 8: total=1126
  REP 9: total=1126
  REP 10: total=1126


In [ ]:
# ===============================================================
# 8) Confusion Matrices (All runs) + Mean±Std Visualizations
#     E-commerce Churn (0 = Active, 1 = Churned)
# ===============================================================
# Bu blokta:
#   - Her model ve her rep için confusion matrix (tn, fp, fn, tp) kayıt edilir.
#   - Tüm confusion matrisleri tek bir DataFrame'e dökülür ve CSV + Excel'e kaydedilir.
#   - Confusion matrislerini mean±std olacak şekilde 3 farklı normalizasyonda (raw / row-normalized / all-normalized)
#     görselleştiririz ve kaydederiz.
#
# In this block we:
#   - Collect all confusion matrices (tn, fp, fn, tp) for each model & repetition.
#   - Store them in a single DataFrame and save to CSV + Excel.
#   - Build mean±std confusion visualizations (raw / row-normalized / all-normalized)
#     and save them as PNG files.
#
# Not:
#   E-ticaret bağlamında sınıf isimleri:
#     - Negatif sınıf (0): "Active"   → Hâlâ alışveriş yapan / aktif müşteri
#     - Pozitif sınıf (1): "Churned"  → Belirlenen eşik süreyi aşmış, kaybedilmiş müşteri
#
# Note:
#   In the e-commerce setting, the classes are:
#     - Negative class (0): "Active"   → Still purchasing / active customer
#     - Positive class (1): "Churned"  → Customer considered lost after some inactivity threshold
# ===============================================================

# ---------------------------------------------------------------
# 8.0) Güvenlik kontrolleri / Safety checks
# ---------------------------------------------------------------
# Block 6 ve 7'nin gerçekten çalıştığından emin olalım.
# Güvenlik: test_confusions değişkeni var mı ve dolu mu?

# Make sure Blocks 6 and 7 have been executed and the required objects exist.
# Safety check: ensure test_confusions exists and is non-empty.

if "test_confusions" not in globals():
    raise RuntimeError(
        "test_confusions bulunamadı. Önce Block 6'yı (ve gerekiyorsa 7'yi) çalıştırmalısın.\n"
        "Cannot find 'test_confusions'. Please run Block 6 before Block 8."
    )

if not isinstance(test_confusions, dict) or len(test_confusions) == 0:
    raise RuntimeError(
        "test_confusions sözlüğü boş görünüyor. Block 6 büyük ihtimalle henüz tamamlanmadı.\n"
        "The 'test_confusions' dict is empty. Did Block 6 finish successfully?"
    )

if "BASE_OUT_DIR" not in globals():
    raise RuntimeError(
        "BASE_OUT_DIR tanımlı değil. Block 0 veya yol tanımlama hücrelerini çalıştırmalısın.\n"
        "'BASE_OUT_DIR' is not defined. Please run Block 0 / path setup cells first."
    )

if "OUT_DIR" not in globals():
    # OUT_DIR yoksa, BASE_OUT_DIR'i kullanarak varsayılan bir OUT_DIR yaratalım.
    # If OUT_DIR is missing, create a default one under BASE_OUT_DIR.
    OUT_DIR = BASE_OUT_DIR / "out_auto_from_block8"
    OUT_DIR.mkdir(parents=True, exist_ok=True)
    print("[WARNING] 'OUT_DIR' tanımlı değildi. OUT_DIR =", OUT_DIR)

# RUN_TAG yoksa, güvenli bir varsayılan verelim.
# If RUN_TAG is missing, define a safe default.
if "RUN_TAG" not in globals():
    RUN_TAG = "EcommerceChurn"
    print("[WARNING] 'RUN_TAG' tanımlı değildi. RUN_TAG = 'EcommerceChurn' olarak ayarlandı.")

# aggregate_cm_mean_std ve plot_confusion_mean_std fonksiyonlarının varlığını uyarı ile kontrol edelim.
# Soft-check for helper functions.
if "aggregate_cm_mean_std" not in globals():
    print("[WARNING] 'aggregate_cm_mean_std' fonksiyonu tanımlı değil görünüyor. "
          "Mean±std confusion görselleri çalışmayabilir.")
if "plot_confusion_mean_std" not in globals():
    print("[WARNING] 'plot_confusion_mean_std' fonksiyonu tanımlı değil görünüyor. "
          "Mean±std confusion görselleri çizilemeyebilir.")


# ---------------------------------------------------------------
# 8A) Confusion matrices → satır bazlı tablo (tn, fp, fn, tp)
#     Confusion matrices → row-wise table
# ---------------------------------------------------------------
rows_all = []

# Sadece Block 6'da kullandığın modelleri dikkate alalım
# Consider only the models actually used in Block 6
for model_name, cm_list in test_confusions.items(): # beklenen: ["KNN", "XGBoost", "Bagging", "Random Forest"]
    for rep0, cm in enumerate(cm_list):
        cm = np.asarray(cm)

        if cm.shape != (2, 2):
            raise ValueError(
                f"[{model_name}] rep={rep0}: Confusion matrix shape {cm.shape} is not 2x2."
            )
        
        # Confusion matrix parçalama:
        # Row 0 = gerçek sınıf 0 (Active), Row 1 = gerçek sınıf 1 (Churned)
        # Col 0 = tahmin 0, Col 1 = tahmin 1
        tn, fp = int(cm[0, 0]), int(cm[0, 1])
        fn, tp = int(cm[1, 0]), int(cm[1, 1])

        rows_all.append({
            "model": model_name,
            "rep": rep0 + 1,
            "tn": tn,
            "fp": fp,
            "fn": fn,
            "tp": tp,
            # Klinik etiketi yerine iş bağlamını yazıyoruz:
            # Instead of clinical labels, we use business context labels.
            "label_neg": "Active",   # 0 = Active
            "label_pos": "Churned"   # 1 = Churned
        })
# Tüm run’lar için confusion tablosu
df_conf_all = pd.DataFrame(
    rows_all,
    columns=[
        "model", "rep", "tn", "fp", "fn", "tp",
        "label_neg", "label_pos"
    ]
)

# Kaydetme yolu: OUT_DIR altında, OBJ tag’i ile
# Save paths: under OUT_DIR, using RUN_TAG (e.g., OBJ_auc)
conf_base_excel = OUT_DIR / f"unseen_confusions_allruns_ecommerce_churn_{RUN_TAG}.xlsx"
conf_base_csv   = OUT_DIR / f"unseen_confusions_allruns_ecommerce_churn_{RUN_TAG}.csv"

df_conf_all.to_excel(
    conf_base_excel,
    sheet_name="confusions_allruns",
    index=False
)
df_conf_all.to_csv(conf_base_csv, index=False)

print("\n[INFO] All-run confusion matrices table saved:")
print("  - EXCEL:", conf_base_excel)
print("  - CSV  :", conf_base_csv)

# OUT_DIR içine de daha basit isimle kaydedelim 
# Also save a simple Excel inside OUT_DIR for direct 
conf_simple_excel = OUT_DIR / "confusions_allruns_ecommerce_churn.xlsx"
df_conf_all.to_excel(conf_simple_excel, index=False)
print("  - EXCEL (simple):", conf_simple_excel)


# ---------------------------------------------------------------
# 8B) Confusion Matrix TOTAL COUNTS (per run & model)
#     (Ek tablo – her test setindeki toplam örnek sayısını gösterir)
# ---------------------------------------------------------------
# Bu tablo, her model ve rep için tn+fp+fn+tp toplamını da yazar.
# Böylece test setinde gerçekten kaç örnek olduğunu (ör. ~1126) net görürsün.
#
# This extra table shows the total number of samples in each test set for
# every model & repetition (tn+fp+fn+tp).

rows_totals = []

for m, cm_list in test_confusions.items():
    for rep_idx, cm in enumerate(cm_list):
        cm = np.asarray(cm)
        if cm.shape != (2, 2):
            raise ValueError(
                f"[{m}] rep={rep_idx}: Confusion matrix shape {cm.shape} is not 2x2."
            )

        tn, fp = int(cm[0, 0]), int(cm[0, 1])
        fn, tp = int(cm[1, 0]), int(cm[1, 1])
        total = tn + fp + fn + tp

        rows_totals.append({
            "model": m,
            "rep": rep_idx + 1,
            "tn": tn,
            "fp": fp,
            "fn": fn,
            "tp": tp,
            "total_samples_in_test": total
        })

df_conf_totals = pd.DataFrame(rows_totals)

conf_totals_csv  = OUT_DIR / "confusion_totals_ecommerce_churn.csv"
conf_totals_xlsx = OUT_DIR / "confusion_totals_ecommerce_churn.xlsx"

df_conf_totals.to_csv(conf_totals_csv, index=False)
df_conf_totals.to_excel(conf_totals_xlsx, index=False)

print("\n[INFO] Confusion matrix TOTALS saved:")
print("  - CSV :", conf_totals_csv)
print("  - XLSX:", conf_totals_xlsx)


# ---------------------------------------------------------------
# 8C) Mean±std confusion plots (raw, row-normalized, all-normalized)
# ---------------------------------------------------------------
# aggregate_cm_mean_std ve plot_confusion_mean_std fonksiyonları
# kodda tanımlanmış olmalı (Block 8 yardımcı fonksiyonları).
# Bu fonksiyonlar önceki bloklarda tanımlı olmalıdır.

# We assume helper functions:
#   - aggregate_cm_mean_std(df_conf_all, normalize=...)
#   - plot_confusion_mean_std(agg, normalize, classes, out_path)
# are already defined as in notebook. 
# Helper functions, assumed to be defined in earlier blocks.

ecommerce_classes = ["Active", "Churned"]  # E-ticaret bağlamındaki sınıf isimleri

if ("aggregate_cm_mean_std" in globals()) and ("plot_confusion_mean_std" in globals()):

    for norm_tag in [None, "row", "all"]:
        # aggregate_cm_mean_std: df_conf_all üzerinden mean±std hesaplar
        # aggregate_cm_mean_std computes mean±std confusion entries from df_conf_all.
        agg = aggregate_cm_mean_std(df_conf_all, normalize=norm_tag)

        tag = "raw" if norm_tag is None else norm_tag

        out_png = OUT_DIR / f"confusion_mean_std_ecommerce_churn_{RUN_TAG}_{tag}.png"

        # ÖNEMLİ: Numeric sınıflar yerine/etiketler yerine e-ticaret sınıf isimlerini kullanıyoruz.
        # IMPORTANT:We use e-commerce labels instead of ecommerce labels. Not numeric labels.
        plot_confusion_mean_std(
            agg,
            normalize=norm_tag,
            classes=ecommerce_classes,
            out_path=out_png
        )
        print(f"[OK] Saved mean±std confusion plot ({tag}): {out_png}")

else:
    print(
        "\n[WARNING] Mean±std confusion plots not generated because "
        "'aggregate_cm_mean_std' or 'plot_confusion_mean_std' is missing."
    )



[INFO] All-run confusion matrices table saved:
  - EXCEL: C:\Users\ulku\Desktop\ecommerce\featselect_optuna_final\outputs_OBJ_auc\unseen_confusions_allruns_ecommerce_churn_OBJ_auc.xlsx
  - CSV  : C:\Users\ulku\Desktop\ecommerce\featselect_optuna_final\outputs_OBJ_auc\unseen_confusions_allruns_ecommerce_churn_OBJ_auc.csv
  - EXCEL (simple): C:\Users\ulku\Desktop\ecommerce\featselect_optuna_final\outputs_OBJ_auc\confusions_allruns_ecommerce_churn.xlsx

[INFO] Confusion matrix TOTALS saved:
  - CSV : C:\Users\ulku\Desktop\ecommerce\featselect_optuna_final\outputs_OBJ_auc\confusion_totals_ecommerce_churn.csv
  - XLSX: C:\Users\ulku\Desktop\ecommerce\featselect_optuna_final\outputs_OBJ_auc\confusion_totals_ecommerce_churn.xlsx


C:\Users\ulku\AppData\Local\Temp\ipykernel_14900\114488497.py:374: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig.tight_layout(rect=[0, 0, 0.90, 0.97])


[OK] Saved mean±std confusion plot (raw): C:\Users\ulku\Desktop\ecommerce\featselect_optuna_final\outputs_OBJ_auc\confusion_mean_std_ecommerce_churn_OBJ_auc_raw.png


C:\Users\ulku\AppData\Local\Temp\ipykernel_14900\114488497.py:374: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig.tight_layout(rect=[0, 0, 0.90, 0.97])


[OK] Saved mean±std confusion plot (row): C:\Users\ulku\Desktop\ecommerce\featselect_optuna_final\outputs_OBJ_auc\confusion_mean_std_ecommerce_churn_OBJ_auc_row.png


C:\Users\ulku\AppData\Local\Temp\ipykernel_14900\114488497.py:374: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig.tight_layout(rect=[0, 0, 0.90, 0.97])


[OK] Saved mean±std confusion plot (all): C:\Users\ulku\Desktop\ecommerce\featselect_optuna_final\outputs_OBJ_auc\confusion_mean_std_ecommerce_churn_OBJ_auc_all.png


In [87]:
##YENİ 9.BÖLÜM
# ===============================================================
# 9) Per-run Metrics + Friedman/Wilcoxon + Significance (Holm-gated)
#     E-commerce Churn (0 = Active, 1 = Churned)
# ===============================================================df_raw.columns
# Bu blokta:
#   - Her (model × run) için tn/fp/fn/tp → türetilmiş metrikleri hesaplarız.
#   - Modeller bazında mean ± std tablo üretiriz.
#   - Friedman testi + pairwise Wilcoxon (Holm düzeltmeli) uygularız.
#   - Model×model significance matrislerini (Excel + heatmaps) kaydederiz.
#
# In this block:
#   - Compute derived per-run metrics from confusion matrices.
#   - Build mean ± std summary table by model.
#   - Run Friedman + Wilcoxon-Holm significance comparisons.
#   - Save model×model significance matrices & heatmaps.
# ===============================================================

ALPHA_SIG = 0.05     # Holm-corrected alpha
TIE_EPS   = 1e-12    # tie threshold for mean_diff

# ---------------------------------------------------------------
# 9A) Per-run Metrics Table (tn/fp/fn/tp → derived metrics)
# ---------------------------------------------------------------

# Per-run metrics table
metrics_list = [
    "sensitivity", "specificity", "precision", "recall",
    "f1", "mcc", "gmean", "accuracy"
]

# compute_row_metrics → (sens, spec, prec, rec, f1, mcc, gmean, acc)
mets = df_conf_all.apply(
    lambda r: compute_row_metrics(r["tn"], r["fp"], r["fn"], r["tp"]),
    axis=1
)

(df_conf_all["sensitivity"],
 df_conf_all["specificity"],
 df_conf_all["precision"],
 df_conf_all["recall"],
 df_conf_all["f1"],
 df_conf_all["mcc"],
 df_conf_all["gmean"],
 df_conf_all["accuracy"]) = zip(*mets)

# Sadece çalışmada kullandığın 4 modeli dahil et
allowed_models = ["KNN", "XGBoost", "Bagging", "Random Forest"]
df_per_run = df_conf_all[df_conf_all["model"].isin(allowed_models)][
    ["model", "rep", "tn", "fp", "fn", "tp"] + metrics_list
].copy()

# Friedman fonksiyonu 'run' kolonunu beklediği için rep → run ismini değiştir
df_per_run.rename(columns={"rep": "run"}, inplace=True)

df_per_run.to_csv(OUT_DIR / "metrics_per_run_ecommerce_churn.csv", index=False)

print("\n[INFO] Saved per-run metrics:",
      OUT_DIR / "metrics_per_run_ecommerce_churn.csv")


# ---------------------------------------------------------------
# 9B) Summary Table (mean ± std per model)
# ---------------------------------------------------------------
summary_agg = (
    df_per_run.groupby("model")[metrics_list]
    .agg(["mean", "std", "count"])
    .reset_index()
)

summary_agg.columns = ["model"] + [
    f"{m}_{s}" for m in metrics_list for s in ["mean", "std", "count"]
]

summary_agg.to_csv(
    OUT_DIR / "metrics_summary_by_model_ecommerce_churn.csv", index=False
)

print("[INFO] Summary table saved:",
      OUT_DIR / "metrics_summary_by_model_ecommerce_churn.csv")


# ---------------------------------------------------------------
# 9C) Friedman + Pairwise Wilcoxon (Holm-corrected)
# ---------------------------------------------------------------
friedman_rows = []
pairwise_all  = []

for mtr in metrics_list:

    df_long = df_per_run[["model", "run", mtr]].copy()

    # Fonksiyon: friedman_and_pairwise()
    res, pw_raw = friedman_and_pairwise(df_long, mtr)
    pw = pw_raw.copy()

    # Direction
    pw["direction_raw"] = np.where(
        pw["mean_diff"] >  TIE_EPS, pw["model_a"],
        np.where(pw["mean_diff"] < -TIE_EPS, pw["model_b"], "tie")
    )

    # Holm correction
    mask = ~pw["p_raw"].isna()
    if mask.any():
        _, p_holm, _, _ = multipletests(pw.loc[mask, "p_raw"], method="holm")
        pw.loc[mask, "p_holm"] = p_holm
    else:
        pw["p_holm"] = np.nan

    # Winner (Holm-gated)
    winners = []
    for _, r in pw.iterrows():
        if pd.isna(r["p_holm"]):
            winners.append("None")
        elif r["p_holm"] < ALPHA_SIG:
            if r["mean_diff"] > TIE_EPS:
                winners.append(r["model_a"])
            elif r["mean_diff"] < -TIE_EPS:
                winners.append(r["model_b"])
            else:
                winners.append("No")
        else:
            winners.append("No")

    pw["winner"] = winners

    friedman_rows.append({
        "metric": mtr,
        "friedman_stat": res["friedman"]["stat"],
        "friedman_p":    res["friedman"]["p"],
        "k_models":      res["friedman"]["k"],
        "n_runs":        res["friedman"]["n"]
    })

    pairwise_all.append(pw)

df_friedman = pd.DataFrame(friedman_rows)
df_pairwise = pd.concat(pairwise_all, ignore_index=True)

# Save CSVs
df_friedman.to_csv(
    OUT_DIR / "stats_friedman_ecommerce_churn.csv", index=False
)
df_pairwise.to_csv(
    OUT_DIR / "stats_pairwise_ecommerce_churn_directional.csv", index=False
)

print("[OK] Friedman + Wilcoxon-Holm stats saved.")


# ---------------------------------------------------------------
# 9D) Build Model×Model Significance Matrices + Heatmaps
# ---------------------------------------------------------------
signif_xlsx = BASE_OUT_DIR / f"pairwise_matrices_ecommerce_churn_{RUN_TAG}.xlsx"
models_all = sorted(set(df_pairwise["model_a"]) | set(df_pairwise["model_b"]))
# --- Plot-only short labels to avoid overlap ---
NAME_MAP = {
    "Random Forest": "RF",
    "XGBoost": "XGB",
    "Bagging": "Bagging",
    "KNN": "KNN"
}
def _short(x): 
    return NAME_MAP.get(x, x)

def _plot_matrix_text(M_df, title, out_png, highlight_mask=None):

    
    # Copy and shorten labels/text only for plotting
    
    M_df = M_df.copy()
    M_df.index   = [_short(x) for x in M_df.index]
    M_df.columns = [_short(x) for x in M_df.columns]
    M_df = M_df.replace(NAME_MAP)   # hücre içindeki kazanan adları da kısalsın

    code = np.zeros(M_df.shape)
    
           # --- Hücre yazıları (winner) ---
    txt = M_df.copy().astype(str).applymap(_short)

    for i in range(M_df.shape[0]):
        for j in range(M_df.shape[1]):
            code[i, j] = np.nan if i == j else (
                1 if highlight_mask is not None and highlight_mask[i, j] else 0
            )

    code_df = pd.DataFrame(0.0, index=M_df.index, columns=M_df.columns)
    if highlight_mask is not None:
        code_df.values[highlight_mask] = 1.0
    np.fill_diagonal(code_df.values, np.nan)
    plt.figure(figsize=(7.2, 6.2))
    ax = sns.heatmap(
        code_df,
        annot=txt, fmt="",
        cbar=False,
        square=True,
        cmap=sns.color_palette(["#f0f0f0", "#bfe3bf"], as_cmap=True),
        linewidths=.6, linecolor="white",
        annot_kws={"fontsize": 12}
    )

    ax.set_xticklabels(ax.get_xticklabels(), rotation=30, ha="right", fontsize=11)
    ax.set_yticklabels(ax.get_yticklabels(), rotation=0, fontsize=11)

    plt.title(title, fontsize=14, pad=10)
    plt.tight_layout(pad=1.2)
    plt.savefig(out_png, dpi=300)
    plt.close()


# Write matrices
with pd.ExcelWriter(signif_xlsx) as writer:

    for mtr in metrics_list:
        sub = df_pairwise[df_pairwise["metric"] == mtr]

        P = pd.DataFrame(np.nan, index=models_all, columns=models_all)
        M = pd.DataFrame("None", index=models_all, columns=models_all)

        for _, r in sub.iterrows():
            a, b, p, w = r["model_a"], r["model_b"], r["p_holm"], r["winner"]
            P.loc[a, b] = P.loc[b, a] = p
            M.loc[a, b] = M.loc[b, a] = w

        np.fill_diagonal(P.values, 0.0)
        np.fill_diagonal(M.values, "-")

        # Excel sheets
        P.to_excel(writer, sheet_name=f"{mtr}_pHolm")
        M.to_excel(writer, sheet_name=f"{mtr}_winner")

        # Heatmap: pHolm → -log10
        # Heatmap: pHolm → -log10
        P_plot = -np.log10(P.replace(0, np.nan))

        # keep full names; wrap only if mapped (e.g., Random\nForest)
        P_plot = P_plot.copy()
        P_plot.index   = [_short(x) for x in P_plot.index]
        P_plot.columns = [_short(x) for x in P_plot.columns]


        # Short labels only for plotting
        P_plot.index   = [_short(x) for x in P_plot.index]
        P_plot.columns = [_short(x) for x in P_plot.columns]

        plt.figure(figsize=(7.2, 6.2))
        ax = sns.heatmap(
            P_plot,
            cmap="mako",
            square=True,
            linewidths=.6, linecolor="white",
            cbar_kws={"label": "-log10(pHolm)", "shrink": 0.85, "pad": 0.02}
        )

        ax.set_xticklabels(ax.get_xticklabels(), rotation=30, ha="right", fontsize=11)
        ax.set_yticklabels(ax.get_yticklabels(), rotation=0, fontsize=11)

        plt.title(f"{mtr} — Holm-Corrected Significance", fontsize=14, pad=10)
        plt.tight_layout(pad=1.2)
        plt.savefig(STATS_PLOTS / f"{mtr}_pHolm_heatmap_ecommerce_churn.png", dpi=300)
        plt.close()


        # Winner matrix highlight
        hl = np.zeros(M.shape, dtype=bool)
        for i in range(M.shape[0]):
            for j in range(M.shape[1]):
                val = M.iloc[i, j]
                hl[i, j] = (i != j and val not in {"No", "None", "-"})


        _plot_matrix_text(
            M,
            title=f"{mtr} — Pairwise Model Superiority (Holm-gated)\nE-commerce Churn",
            out_png=STATS_PLOTS / f"{mtr}_winner_matrix_ecommerce_churn.png",
            highlight_mask=hl
        )

print("\n[OK] All significance matrices and plots saved:")
print("  Excel :", signif_xlsx)
print("  Plots :", STATS_PLOTS)



[INFO] Saved per-run metrics: C:\Users\ulku\Desktop\ecommerce\featselect_optuna_final\outputs_OBJ_auc\metrics_per_run_ecommerce_churn.csv
[INFO] Summary table saved: C:\Users\ulku\Desktop\ecommerce\featselect_optuna_final\outputs_OBJ_auc\metrics_summary_by_model_ecommerce_churn.csv
[OK] Friedman + Wilcoxon-Holm stats saved.


C:\Users\ulku\AppData\Local\Temp\ipykernel_14900\3768908377.py:180: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  txt = M_df.copy().astype(str).applymap(_short)
C:\Users\ulku\AppData\Local\Temp\ipykernel_14900\3768908377.py:180: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  txt = M_df.copy().astype(str).applymap(_short)
C:\Users\ulku\AppData\Local\Temp\ipykernel_14900\3768908377.py:180: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  txt = M_df.copy().astype(str).applymap(_short)
C:\Users\ulku\AppData\Local\Temp\ipykernel_14900\3768908377.py:180: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  txt = M_df.copy().astype(str).applymap(_short)
C:\Users\ulku\AppData\Local\Temp\ipykernel_14900\3768908377.py:180: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  txt = M_df.copy().astype(str).applymap(_short)
C:\Us


[OK] All significance matrices and plots saved:
  Excel : C:\Users\ulku\Desktop\ecommerce\featselect_optuna_final\pairwise_matrices_ecommerce_churn_OBJ_auc.xlsx
  Plots : C:\Users\ulku\Desktop\ecommerce\featselect_optuna_final\stats_plots_OBJ_auc


In [ ]:
# ===============================================================
# 10) 95% t-CI for ALL metrics + AUC/AP + Paired t-tests (AUC)
#     E-commerce Churn (0 = Active, 1 = Churned)
# ===============================================================
# Bu blokta:
#   - Her model için tüm metrikler (sensitivity, specificity, precision,
#     recall, f1, mcc, gmean, accuracy) için %95 t-güven aralıkları hesaplanır.
#   - AUC ve AP için de model bazında %95 t-CI hesaplanır.
#   - AUC değerleri üzerinden modeller arası ikili eşleştirilmiş t-testleri
#     (Holm düzeltmeli) yapılır.
#
# In this block:
#   - Compute 95% t-confidence intervals for all per-run metrics.
#   - Compute 95% t-CI for AUC and AP per model.
#   - Run paired t-tests on AUC between models (Holm-corrected).
#
# Not / Note:
#   Bu veri seti dengesiz olduğu için AUC, temel karşılaştırma metriğidir.
#   Since the dataset is imbalanced, AUC is used as the primary comparison metric.
# ===============================================================

ALPHA_CI = 0.05   # 95% CI → alpha = 0.05

# Sadece çalışmada kullandığın 4 modeli dikkate alalım
# Only keep the 4 models used in the study
allowed_models = ["KNN", "XGBoost", "Bagging", "Random Forest"]

# ---------------------------------------------------------------
# 10A) CI for per-run metrics (sensitivity, specificity, ...)
# ---------------------------------------------------------------
rows_ci = []

# df_per_run Block 9'da oluşturuldu (model × rep × metrikler)
# df_per_run was built in Block 9 (model × rep × metrics)
for model_name, g in df_per_run.groupby("model"):
    if model_name not in allowed_models:
        continue

    for mtr in metrics_list:
        # mean_std_ci_t: (mean, std, ci_low, ci_high, n)
        m, s, lo, hi, n = mean_std_ci_t(g[mtr].values, alpha=ALPHA_CI)
        rows_ci.append({
            "model":      model_name,
            "metric":     mtr,
            "mean":       m,
            "std":        s,
            "ci95_low":   lo,
            "ci95_high":  hi,
            "n_runs_used": n
        })

df_ci_long = (
    pd.DataFrame(rows_ci)
    .sort_values(["model", "metric"])
    .reset_index(drop=True)
)

ci_long_path = OUT_DIR / "metrics_ci95_by_model_ecommerce_churn_long.csv"
df_ci_long.to_csv(ci_long_path, index=False)

print("\n[INFO] 95% t-CI (long format) saved:")
print("  - CSV:", ci_long_path)

# ---------------------------------------------------------------
# 10B) Wide variant (her model tek satır, metrikler kolon)
#     Wide variant (one row per model, metrics as columns)
# ---------------------------------------------------------------
wide_rows = []

for model_name, g in df_per_run.groupby("model"):
    if model_name not in allowed_models:
        continue

    row = {"model": model_name}
    for mtr in metrics_list:
        m, s, lo, hi, n = mean_std_ci_t(g[mtr].values, alpha=ALPHA_CI)
        row[f"{mtr}_mean"]      = m
        row[f"{mtr}_std"]       = s
        row[f"{mtr}_ci95_low"]  = lo
        row[f"{mtr}_ci95_high"] = hi
        row[f"{mtr}_n"]         = n
    wide_rows.append(row)

df_ci_wide = (
    pd.DataFrame(wide_rows)
    .sort_values("model")
    .reset_index(drop=True)
)

ci_wide_path = OUT_DIR / "metrics_ci95_by_model_ecommerce_churn_wide.csv"
df_ci_wide.to_csv(ci_wide_path, index=False)

print("[INFO] 95% t-CI (wide format) saved:")
print("  - CSV:", ci_wide_path)

# ---------------------------------------------------------------
# 10C) CI for AUC and AP (per model)
# ---------------------------------------------------------------
rows_auc_ap = []

# test_auc_list / test_ap_list: Block 6–7'de doldurulmuştu
# We copy them into plain lists to avoid side effects
auc_src = {k: list(v) for k, v in test_auc_list.items() if k in allowed_models}
ap_src  = {k: list(v) for k, v in test_ap_list.items()  if k in allowed_models}

# AUC için CI
for model, arr in auc_src.items():
    m, s, lo, hi, n = mean_std_ci_t(arr, alpha=ALPHA_CI)
    rows_auc_ap.append({
        "model":      model,
        "metric":     "auc",
        "mean":       m,
        "std":        s,
        "ci95_low":   lo,
        "ci95_high":  hi,
        "n_runs_used": n
    })

# AP için CI
for model, arr in ap_src.items():
    m, s, lo, hi, n = mean_std_ci_t(arr, alpha=ALPHA_CI)
    rows_auc_ap.append({
        "model":      model,
        "metric":     "ap",
        "mean":       m,
        "std":        s,
        "ci95_low":   lo,
        "ci95_high":  hi,
        "n_runs_used": n
    })

if rows_auc_ap:
    df_auc_ap_ci = (
        pd.DataFrame(rows_auc_ap)
        .sort_values(["model", "metric"])
        .reset_index(drop=True)
    )
    auc_ap_ci_path = OUT_DIR / "auc_ap_ci95_by_model_ecommerce_churn.csv"
    df_auc_ap_ci.to_csv(auc_ap_ci_path, index=False)

    print("\n[INFO] 95% t-CI for AUC & AP saved:")
    print("  - CSV:", auc_ap_ci_path)

# ---------------------------------------------------------------
# 10D) Paired t-tests for AUC (Model A vs Model B)
# ---------------------------------------------------------------
pairs = []
models_auc = list(auc_src.keys())  # only allowed_models with AUC data

for a, b in itertools.combinations(models_auc, 2):
    # paired_ttest_ci: AUC(A) vs AUC(B) eşleştirilmiş t-test
    res = paired_ttest_ci(auc_src[a], auc_src[b], alpha=ALPHA_CI)
    pairs.append({
        "model_a": a,
        "model_b": b,
        "n_pairs": res["n"],
        "delta_mean_auc_(a-b)": res["mean_diff"],
        "ci95_low":  res["ci95"][0],
        "ci95_high": res["ci95"][1],
        "t_stat":    res["t"],
        "p_value":   res["p"]
    })

df_pairwise_auc = pd.DataFrame(pairs)

if not df_pairwise_auc.empty:
    mask = ~df_pairwise_auc["p_value"].isna()
    if mask.any():
        # Holm-corrected p-values
        _, p_holm, _, _ = multipletests(
            df_pairwise_auc.loc[mask, "p_value"].values,
            method="holm"
        )
        df_pairwise_auc.loc[mask, "p_value_holm"] = p_holm
    else:
        df_pairwise_auc["p_value_holm"] = np.nan

    auc_ttest_path = OUT_DIR / "auc_pairwise_paired_ttest_ecommerce_churn.csv"
    df_pairwise_auc.to_csv(auc_ttest_path, index=False)

    print("\n[INFO] Paired t-tests for AUC (with Holm correction) saved:")
    print("  - CSV:", auc_ttest_path)



[INFO] 95% t-CI (long format) saved:
  - CSV: C:\Users\ulku\Desktop\ecommerce\featselect_optuna_final\outputs_OBJ_auc\metrics_ci95_by_model_ecommerce_churn_long.csv
[INFO] 95% t-CI (wide format) saved:
  - CSV: C:\Users\ulku\Desktop\ecommerce\featselect_optuna_final\outputs_OBJ_auc\metrics_ci95_by_model_ecommerce_churn_wide.csv

[INFO] 95% t-CI for AUC & AP saved:
  - CSV: C:\Users\ulku\Desktop\ecommerce\featselect_optuna_final\outputs_OBJ_auc\auc_ap_ci95_by_model_ecommerce_churn.csv

[INFO] Paired t-tests for AUC (with Holm correction) saved:
  - CSV: C:\Users\ulku\Desktop\ecommerce\featselect_optuna_final\outputs_OBJ_auc\auc_pairwise_paired_ttest_ecommerce_churn.csv


In [ ]:
# ===============================================================
# 11) XAI: SHAP & LIME (SELECTED feature space)
#     E-commerce Churn (0 = Active, 1 = Churned)
# ===============================================================
# Bu blokta:
#   - Her model için, seçilmiş feature uzayında SHAP ve LIME analizleri yapılır.
#   - Orta-performans (mid-AP) run’lar arasından sınırlı sayıda temsilci rep
#     seçilerek, sadece bunlar üzerinde açıklanabilirlik görselleri üretilir.
#
# In this block:
#   - Run SHAP and LIME explanations in the selected-feature space for each model.
#   - Use a small number of mid-AP runs per model to keep XAI feasible but
#     still representative.
# ===============================================================

if DO_EXPLAINERS:
    try:
        import shap
        from lime.lime_tabular import LimeTabularExplainer
        from scipy import sparse as sp
        import matplotlib as mpl
        import matplotlib.pyplot as plt
        mpl.rcParams["figure.max_open_warning"] = 0
    except Exception as e:
        warnings.warn(f"Explainability packages not available; SHAP/LIME skipped: {e}")
        DO_EXPLAINERS = False

# ---------------------------------------------------------------
# CLASS NAMES MUST BE "Churned/Active" customer , NOT NUMERIC -
# Sınıf isimleri: iş bağlamında (business-class labels)
# ---------------------------------------------------------------
# 0 = Active customer (aktif müşteri)
# 1 = Churned customer (kaybedilmiş / ayrılmış müşteri)
BUSINESS_CLASS_NAMES = ["Active Customer", "Churned Customer"]

if DO_EXPLAINERS:

    # -----------------------------------------------------------
    # Yardımcı fonksiyonlar / Helper functions
    # -----------------------------------------------------------

    def transform_until_estimator(pipe: Pipeline, X_in):
        """
        Pipeline içindeki 'clf' adımına kadar bütün adımları uygular
        ve çıktı feature matrisini döner.
        Apply all steps in the pipeline up to (but not including) 'clf'
        and return the transformed feature matrix.
        """
        Xt = X_in
        for name, step in pipe.named_steps.items():
            if name == "clf":
                break
            Xt = step.transform(Xt)
            if sp.issparse(Xt):
                Xt = Xt.toarray()
        return Xt

    def get_selected_names_for_pipe(pipe: Pipeline, X_ref):
        """
        Seçilen feature maskesini ve isimlerini döner.
        Return mask and selected feature names for a fitted pipeline.
        """
        mask, sel_names, sel_names_clean, _ = get_selected_feature_info(pipe, X_ref)
        return sel_names, sel_names_clean

    # Orta performanslı run penceresi (mid-AP window)
    # Mid-AP runs: we pick a small window around the median AP.
    def select_median_window(df_scores, model_name, window=2):
        dfm = (
            df_scores[df_scores["model"] == model_name]
            .sort_values("ap_test")
            .reset_index(drop=True)
        )
        if dfm.empty:
            return dfm
        k = len(dfm)
        mid = k // 2
        lo = max(0, mid - window)
        hi = min(k, mid + window + 1)
        return dfm.iloc[lo:hi][["model", "rep", "ap_test", "auc_test"]]

    WINDOW = 2  # her model için median etrafında ±2 run
    df_scores_ap = pd.DataFrame(ap_records)
    selected_rows = {m: select_median_window(df_scores_ap, m, window=WINDOW)
                     for m in model_names}

    # Seçilen run’ların logunu kaydet / Save selected runs
    sel_log = []
    for m, d in selected_rows.items():
        if d is None or d.empty:
            continue
        for _, r in d.iterrows():
            sel_log.append({
                "model": m,
                "rep": int(r["rep"]),
                "ap_test": float(r.get("ap_test", np.nan)),
                "auc_test": float(r.get("auc_test", np.nan)),
            })

    if sel_log:
        sel_csv = EXPL_DIR / "selected_runs_for_explanations_ecommerce_churn.csv"
        pd.DataFrame(sel_log).to_csv(sel_csv, index=False)
        print("[INFO] Selected mid-AP candidate runs for XAI saved to:", sel_csv)

    # -----------------------------------------------------------
    # XAI için örnek boyutları ve rep sınırı
    # -----------------------------------------------------------
    # Arka plan ve test altörnek boyutları / background & test subsample sizes
    BGN = 100    # (200 den daha düşük, ama açıklama için yeterli)
    TSN = 25     # 50 yerine her model-run için 25 müşteri
    TOPK_DEP = 5 # kaç feature için dependence plot

    # Her model için XAI yapılacak maksimum rep sayısı
    # (mid-AP penceresinden seçilen en fazla 2 run)
    MAX_XAI_REPS = 2

    rng = np.random.default_rng(42)

    # -----------------------------------------------------------
    # SHAP helper for tree-based models
    # -----------------------------------------------------------
    def compute_shap_2d_tree(clf, X_bg_sel_np, X_ts_sel_np, pos_idx_local):
        """
        Ağaç tabanlı modeller için (RF, XGBoost, vb.) TreeSHAP uygular
        ve pozitif sınıf için 2D SHAP matrisi döner.
        Apply TreeSHAP for tree-based models and return 2D SHAP values
        for the positive class.
        """
        try:
            K = min(80, max(10, X_bg_sel_np.shape[0] // 2))  # 100 yerine 80 (hız için)
            bg = shap.kmeans(X_bg_sel_np, K)
        except Exception:
            idx = rng.choice(
                X_bg_sel_np.shape[0],
                size=min(80, X_bg_sel_np.shape[0]),
                replace=False,
            )
            bg = X_bg_sel_np[idx]

        explainer = shap.TreeExplainer(
            clf,
            data=bg,
            feature_perturbation="interventional",
            model_output="probability",
        )
        shap_values = explainer.shap_values(X_ts_sel_np)

        # Çıktıyı 2D hale getir / unify output shape to 2D
        if isinstance(shap_values, list):
            sv = shap_values[pos_idx_local]
        else:
            sv = shap_values
            if sv.ndim == 3 and sv.shape[-1] == len(clf.classes_):
                sv = sv[:, :, pos_idx_local]
            elif sv.ndim != 2:
                raise RuntimeError(f"Unexpected SHAP shape: {sv.shape}")

        return sv

    # -----------------------------------------------------------
    # ANA DÖNGÜ: her model ve seçilen her rep için SHAP + LIME
    # MAIN LOOP: per-model, per-selected-rep explainability
    # -----------------------------------------------------------
    for model_name in model_names:

        sel_df = selected_rows.get(model_name, pd.DataFrame())
        if sel_df is None or sel_df.empty:
            print(f"[WARN] {model_name}: no selected runs; skipping XAI.")
            continue

        # 🔹 YENİ: Her model için en fazla MAX_XAI_REPS run açıklanacak
        sel_df = sel_df.head(MAX_XAI_REPS)

        safe_model = model_name.replace(" ", "")

        for _, row in sel_df.iterrows():
            rep = int(row["rep"])
            if rep not in split_store:
                print(f"[WARN] {model_name}: rep={rep} missing in split_store; skipped.")
                continue

            idx_trainval = split_store[rep]["trainval_idx"]
            idx_test     = split_store[rep]["test_idx"]

            # X, Block 2-3'te oluşturulan orijinal feature tablosu
            X_bg_full = X.iloc[idx_trainval]
            X_ts_full = X.iloc[idx_test]

            # Hız için altörnekleme / subsample for speed
            b_idx = rng.choice(len(X_bg_full), size=min(BGN, len(X_bg_full)), replace=False)
            t_idx = rng.choice(len(X_ts_full), size=min(TSN, len(X_ts_full)), replace=False)
            X_bg = X_bg_full.iloc[b_idx]
            X_ts = X_ts_full.iloc[t_idx]

            pipe = final_pipes[model_name][rep]
            clf  = pipe.named_steps["clf"]

            pos_idx_local = int(np.where(clf.classes_ == pos_label)[0][0])

            # ---------------- Seçilmiş feature uzayına transform ----------------
             # Transform into selected-feature space
            # Transform into selected-feature space
            X_bg_sel = transform_until_estimator(pipe, X_bg)
            X_ts_sel = transform_until_estimator(pipe, X_ts)

            feat_names_sel, feat_names_sel_clean = get_selected_names_for_pipe(pipe, X)
            # OneHot vb. isimlerden son kısmı al (PreferredPaymentMode__CreditCard → CreditCard)
            feat_names_sel_clean = [f.split("__")[-1] for f in feat_names_sel_clean]

            X_bg_np = np.asarray(X_bg_sel, dtype=float)
            X_ts_np = np.asarray(X_ts_sel, dtype=float)

            model_type = type(clf).__name__.lower()
            is_tree = any(s in model_type for s in ["randomforest", "xgb", "decisiontree", "extratrees"])

            # ---------------------------------------------------
            # SHAP
            # ---------------------------------------------------
            sv_arr = None

            # TreeSHAP mümkünse onu kullan
            if is_tree:
                try:
                    sv_arr = compute_shap_2d_tree(clf, X_bg_np, X_ts_np, pos_idx_local)
                except Exception as e_tree:
                    warnings.warn(
                        f"[INFO] {model_name} rep={rep}: TreeSHAP failed ({e_tree}); "
                        "falling back to KernelSHAP."
                    )

            # Diğer durumlarda KernelSHAP
            if sv_arr is None:
                K = min(80, max(20, X_bg_np.shape[0] // 2))  # 100 yerine 80
                try:
                    bg_kernel = shap.kmeans(X_bg_np, K)
                except Exception:
                    idx = rng.choice(X_bg_np.shape[0], size=min(K, X_bg_np.shape[0]), replace=False)
                    bg_kernel = X_bg_np[idx]

                def proba_fn_sel(z):
                    return clf.predict_proba(np.asarray(z))[:, pos_idx_local]

                explainer = shap.KernelExplainer(proba_fn_sel, bg_kernel, link="logit")
                # 🔹 nsamples 1024 → 256 (çok büyük hız kazancı)
                sv_arr = explainer.shap_values(X_ts_np, nsamples=256)
                if isinstance(sv_arr, list):
                    sv_arr = sv_arr[0]

            # Trim mismatch - Boyut uyuşmazlığı durumunda isimleri kırp / align feature names
            if sv_arr.shape[1] != len(feat_names_sel_clean):
                feat_names_sel_clean = feat_names_sel_clean[: sv_arr.shape[1]]

            # ---------------- SHAP PLOT: Bar ----------------
            try:
                shap.summary_plot(
                    sv_arr,
                    feature_names=feat_names_sel_clean,
                    plot_type="bar",
                    show=False,
                    max_display=20,
                )
                fig = plt.gcf()
                fig.tight_layout()
                fig.savefig(
                    EXPL_DIR / f"{safe_model}__rep{rep:02d}__shap_bar_ecommerce_churn.png",
                    dpi=150,
                    bbox_inches="tight",
                )
                plt.close(fig)
            except Exception as e:
                warnings.warn(f"[WARN] {model_name} rep={rep} SHAP bar error: {e}")

            # ---------------- SHAP PLOT: Beeswarm ----------------
            try:
                shap.summary_plot(
                    sv_arr,
                    X_ts_np,
                    feature_names=feat_names_sel_clean,
                    show=False,
                    max_display=20,
                )
                fig = plt.gcf()
                fig.tight_layout()
                fig.savefig(
                    EXPL_DIR / f"{safe_model}__rep{rep:02d}__shap_beeswarm_ecommerce_churn.png",
                    dpi=150,
                    bbox_inches="tight",
                )
                plt.close(fig)
            except Exception as e:
                warnings.warn(f"[WARN] {model_name} rep={rep} SHAP beeswarm error: {e}")

            # ---------------- SHAP Dependence plots ----------------
            try:
                mean_abs = np.mean(np.abs(sv_arr), axis=0)
                top_idx = np.argsort(mean_abs)[::-1][: min(TOPK_DEP, len(mean_abs))]
                for j in top_idx:
                    shap.dependence_plot(
                        ind=int(j),
                        shap_values=sv_arr,
                        features=X_ts_np,
                        feature_names=feat_names_sel_clean,
                        interaction_index=None,
                        show=False,
                    )
                    fig = plt.gcf()
                    fig.tight_layout()
                    fig.savefig(
                        EXPL_DIR / f"{safe_model}__rep{rep:02d}__shap_dependence_feat{int(j)}_ecommerce_churn.png",
                        dpi=150,
                        bbox_inches="tight",
                    )
                    plt.close(fig)
            except Exception as e:
                warnings.warn(f"[WARN] {model_name} rep={rep} dependence error: {e}")

            # ---------------------------------------------------
            # LIME
            # ---------------------------------------------------
            try:
                lime_expl = LimeTabularExplainer(
                    training_data=X_bg_np,
                    feature_names=feat_names_sel_clean,
                    class_names=BUSINESS_CLASS_NAMES,  # <--- business labels
                    mode="classification",
                    discretize_continuous=True,
                )

                def predict_fn_lime(data):
                    data = np.asarray(data)
                    return clf.predict_proba(data)

                for i in range(min(3, X_ts_np.shape[0])):  # her model-run için max 3 örnek
                    exp = lime_expl.explain_instance(
                        X_ts_np[i],
                        predict_fn_lime,
                        num_features=min(X_ts_np.shape[1], 10),
                        labels=[pos_idx_local],
                    )
                    html = exp.as_html(labels=[pos_idx_local])
                    with open(
                        LIME_DIR / f"{safe_model}__rep{rep:02d}__lime_idx{i}_ecommerce_churn.html",
                        "w",
                        encoding="utf-8",
                    ) as f:
                        f.write(html)

            except Exception as e:
                warnings.warn(f"[WARN] {model_name} rep={rep} LIME error: {e}")

            print(f"[OK] {model_name} rep={rep}: SHAP & LIME saved (E-commerce Churn).")

print("\nAll explainability steps for E-commerce Churn completed (optimized XAI).")


[INFO] Selected mid-AP candidate runs for XAI saved to: C:\Users\ulku\Desktop\ecommerce\featselect_optuna_final\explanations_plots_OBJ_auc\selected_runs_for_explanations_ecommerce_churn.csv


100%|██████████| 25/25 [00:00<00:00, 30.12it/s]


[OK] KNN rep=3: SHAP & LIME saved (E-commerce Churn).


100%|██████████| 25/25 [00:14<00:00,  1.72it/s]


[OK] KNN rep=6: SHAP & LIME saved (E-commerce Churn).


...

[OK] Bagging rep=9: SHAP & LIME saved (E-commerce Churn).


C:\Users\ulku\AppData\Local\Temp\ipykernel_14900\366147603.py:226: UserWarning: [INFO] Random Forest rep=5: TreeSHAP failed (Unsupported masker type: <class 'shap.utils._legacy.DenseData'>!); falling back to KernelSHAP.
  warnings.warn(
100%|██████████| 25/25 [00:13<00:00,  1.89it/s]


[OK] Random Forest rep=5: SHAP & LIME saved (E-commerce Churn).


C:\Users\ulku\AppData\Local\Temp\ipykernel_14900\366147603.py:226: UserWarning: [INFO] Random Forest rep=6: TreeSHAP failed (Unsupported masker type: <class 'shap.utils._legacy.DenseData'>!); falling back to KernelSHAP.
  warnings.warn(
100%|██████████| 25/25 [00:25<00:00,  1.01s/it]


[OK] Random Forest rep=6: SHAP & LIME saved (E-commerce Churn).

All explainability steps for E-commerce Churn completed (thesis-optimized XAI).


In [75]:
# ============================================================
# 12) Selected features + frequency + best hyperparameters
#     E-commerce Churn (0 = Active, 1 = Churned)
#     LONG / WIDE tablolar ve özet grafikler
# ============================================================
# Bu blokta:
#   - Her model × rep için seçilen feature’ları uzun formatta kaydederiz.
#   - Feature seçim frekansını (kaç rep’te seçildi?) hesaplarız.
#   - En çok seçilen ilk 15 feature için bar grafikleri çizeriz.
#   - En iyi hiperparametreleri hem LONG hem WIDE formatta kaydederiz.
#
# In this block:
#   - Save selected features per model × repetition (long format).
#   - Compute selection frequency across runs.
#   - Plot top-15 most frequently selected features per model.
#   - Save best hyperparameters in both LONG and WIDE formats.
# ============================================================

# ------------------------------------------------------------
# 12A) Çıktı klasörleri / Output directories
# ------------------------------------------------------------
SEL_DIR = BASE_OUT_DIR / f"selected_features_{RUN_TAG}_ecommerce_churn"
SEL_DIR.mkdir(parents=True, exist_ok=True)

PLOT_DIR = SEL_DIR / "plots"
PLOT_DIR.mkdir(parents=True, exist_ok=True)

# Çalışmada kullandığın ana modeller
# Main models used in the study
allowed_models = ["KNN", "XGBoost", "Bagging", "Random Forest"]

# Küçük yardımcı fonksiyon: None → NaN array
# Small helper: convert None to NaN array
def safe_array(a, target_len):
    if a is None:
        return np.array([np.nan] * target_len, dtype=float)
    a = np.asarray(a)
    L = min(len(a), target_len)
    return np.asarray(a[:L], dtype=float)


# ------------------------------------------------------------
# 12B) LONG TABLO: Her rep için seçilen feature’lar
#      LONG TABLE: Selected features per model × rep
# ------------------------------------------------------------
rows = []

for model_name in allowed_models:
    pipes_for_model = final_pipes.get(model_name, {})
    store_for_model = selected_features_store.get(model_name, {})

    for rep, pipe in pipes_for_model.items():

        # Bu rep için daha önce kaydedilmiş seçim bilgisi
        # Selection info stored in Block 6
        sel_info = store_for_model.get(rep, {})
        k_selected = int(sel_info.get("k_best", 0))
        sel_names_clean = sel_info.get("selected_feature_names_clean", []) or []

        # Orijinal feature isimleri (X kolon isimleri)
        # Original feature names (columns of X)
        base_features = list(X.columns)
        L = len(base_features)

        # SelectKBest nesnesi (varsa) — skor/p-value almayı deneriz
        # SelectKBest step (if exists) — try to read score/p-values
        feat_step = pipe.named_steps.get("feat", None)
        if feat_step is not None and hasattr(feat_step, "get_support"):
            mask_after = feat_step.get_support()          # boolean mask in transformed space
            scores_raw = getattr(feat_step, "scores_", None)
            pvals_raw  = getattr(feat_step, "pvalues_", None)
        else:
            mask_after = None
            scores_raw = None
            pvals_raw  = None

        # Basitleştirme:
        #   - Orijinal feature listesi üzerinden ilerliyoruz (X.columns).
        #   - selected = feature adı 'sel_names_clean' listesinde mi?
        #   - score_f_classif / p_value bilgilerine ihtiyaç varsa daha ileri
        #     analizlerde eklenebilir; şimdilik NaN tutuyoruz.
        #
        # Simplification:
        #   - Iterate over original X.columns.
        #   - selected = whether feature name is in 'sel_names_clean'.
        #   - score_f_classif / p_value left as NaN; can be enhanced later.

        for idx_in_X, cname in enumerate(base_features):
            is_selected = cname in sel_names_clean

            rows.append({
                "model": model_name,
                "rep": int(rep),
                "k_selected": k_selected,
                "feat_idx_in_X": int(idx_in_X),
                "feature": cname,
                "selected": bool(is_selected),
                "score_f_classif": np.nan,
                "p_value": np.nan,
                # Sadece basit bir sıra bilgisi: orijinal kolon sırası
                # Simple rank info: original column order
                "rank_desc_score": idx_in_X + 1,
            })

df_selected_long = (
    pd.DataFrame(rows)
      .sort_values(["model", "rep", "selected", "rank_desc_score"],
                   ascending=[True, True, False, True])
      .reset_index(drop=True)
)

out_long = SEL_DIR / "selected_features_ecommerce_churn_long.csv"
df_selected_long.to_csv(out_long, index=False)

print(f"[OK] Selected-features (LONG) saved to:\n  {out_long}")


# ------------------------------------------------------------
# 12C) SEÇİM FREKANSI ÖZETİ
#      SELECTION FREQUENCY SUMMARY PER MODEL
# ------------------------------------------------------------
freq_rows = []

if not df_selected_long.empty:
    for (model_name, feat), g in df_selected_long.groupby(["model", "feature"]):
        n_runs = g["rep"].nunique()                          # toplam kaç rep’te bu feature var?
        n_sel  = int(g.loc[g["selected"], "rep"].nunique())  # kaç rep’te seçilmiş?
        freq_rows.append({
            "model": model_name,
            "feature": feat,
            "n_runs": n_runs,
            "n_selected": n_sel,
            "selection_rate": (n_sel / n_runs) if n_runs > 0 else np.nan,
            # score_f_classif şu anda NaN, ama kolon yapısını koruyoruz
            "mean_score_selected": g.loc[
                g["selected"] & g["score_f_classif"].notna(),
                "score_f_classif"
            ].mean(),
            "median_rank_among_all": g["rank_desc_score"].median()
        })

df_freq = (
    pd.DataFrame(freq_rows)
      .sort_values(["model", "selection_rate", "median_rank_among_all"],
                   ascending=[True, False, True])
      .reset_index(drop=True)
)

out_freq = SEL_DIR / "selection_frequency_ecommerce_churn.csv"
df_freq.to_csv(out_freq, index=False)

print(f"[OK] Selection frequency table saved to:\n  {out_freq}")


# ------------------------------------------------------------
# 12D) TOP-15 FEATURE BAR GRAFİKLERİ
#      TOP-15 FEATURE BAR PLOTS PER MODEL
# ------------------------------------------------------------
for m in allowed_models:
    sub = df_freq[df_freq["model"] == m].copy()
    if sub.empty:
        continue

    sub = sub.sort_values("selection_rate", ascending=False).head(18)

    plt.figure(figsize=(8, 6))
    sns.barplot(data=sub, x="selection_rate", y="feature")
    plt.xlabel("Selection Rate across runs\n(Rep’ler boyunca seçilme oranı)")
    plt.ylabel("Feature (Özellik)")
    plt.title(f"Top-15 Selected Features — {m} — E-commerce Churn")
    plt.tight_layout()

    plot_path = PLOT_DIR / f"top_features_{m.replace(' ', '_')}_ecommerce_churn.png"
    plt.savefig(plot_path, dpi=200)
    plt.close()

    print(f"[OK] Top-15 feature plot saved for {m}:\n  {plot_path}")


# ------------------------------------------------------------
# 12E) BEST HYPERPARAMETERS — LONG + WIDE
#      EN İYİ HİPERPARAMETRELER — UZUN + GENİŞ TABLO
# ------------------------------------------------------------
def _flatten_dict(d, parent_key=""):
    """
    İç içe sözlükleri 'parent.child' şeklinde düzleştirir.
    Flatten nested dicts into 'parent.child' keys.
    """
    items = []
    if d is None:
        return items
    for k, v in d.items():
        key = f"{parent_key}.{k}" if parent_key else str(k)
        if isinstance(v, dict):
            items.extend(_flatten_dict(v, key))
        else:
            items.append((key, v))
    return items

hp_long_rows = []
hp_wide_rows = []

for model_name in allowed_models:
    for rec in best_hyperparameters.get(model_name, []):
        rep   = rec.get("rep", np.nan)
        bp    = rec.get("best_params", {}) or {}
        bps   = rec.get("best_params_structured", {}) or {}
        kbest = rec.get("k_best", None)
        valf1 = rec.get("val_f1", np.nan)
        valap = rec.get("val_ap", np.nan)
        valau = rec.get("val_auc", np.nan)
        opttg = rec.get("optimize_for", None)

        # Bu rep için seçilmiş feature listesi
        # Selected features for this rep (if available)
        sel_info = selected_features_store.get(model_name, {}).get(rep, {})
        sel_list = sel_info.get("selected_feature_names_clean", None)

        meta = {
            "meta.val_f1": valf1,
            "meta.val_ap": valap,
            "meta.val_auc": valau,
            "meta.k_best": kbest,
            "meta.optimize_for": opttg,
            "meta.selected_features_joined": ";".join(map(str, sel_list)) if sel_list else None
        }

        # ---------- LONG ----------
        for k, v in meta.items():
            hp_long_rows.append({
                "model": model_name,
                "rep": rep,
                "param": k,
                "value": v
            })
        for k, v in sorted(_flatten_dict(bp)):
            hp_long_rows.append({
                "model": model_name,
                "rep": rep,
                "param": f"best_params.{k}",
                "value": v
            })
        for k, v in sorted(_flatten_dict(bps)):
            hp_long_rows.append({
                "model": model_name,
                "rep": rep,
                "param": f"best_params_structured.{k}",
                "value": v
            })

        # ---------- WIDE ----------
        wide_row = {
            "model": model_name,
            "rep": rep,
            "val_f1": valf1,
            "val_ap": valap,
            "val_auc": valau,
            "k_best": kbest,
            "optimize_for": opttg,
            "selected_features_joined": ";".join(map(str, sel_list)) if sel_list else None
        }
        for k, v in _flatten_dict(bp, "best_params"):
            wide_row[k] = v
        for k, v in _flatten_dict(bps, "best_params_structured"):
            wide_row[k] = v

        hp_wide_rows.append(wide_row)

df_hp_long = (
    pd.DataFrame(hp_long_rows)
      .sort_values(["model", "rep", "param"])
      .reset_index(drop=True)
)
df_hp_wide = (
    pd.DataFrame(hp_wide_rows)
      .sort_values(["model", "rep"])
      .reset_index(drop=True)
)

out_hp_long = SEL_DIR / "best_hyperparameters_ecommerce_churn_long.csv"
out_hp_wide = SEL_DIR / "best_hyperparameters_ecommerce_churn_wide.csv"

df_hp_long.to_csv(out_hp_long, index=False)
df_hp_wide.to_csv(out_hp_wide, index=False)

print(f"[OK] Best hyperparameters (LONG) saved to:\n  {out_hp_long}")
print(f"[OK] Best hyperparameters (WIDE) saved to:\n  {out_hp_wide}")

# JSON snapshot (orijinal best_hyperparameters sözlüğü)
# JSON snapshot of original best_hyperparameters dict
bhp_json_out = SEL_DIR / "best_hyperparameters_ecommerce_churn_with_selected.json"
try:
    with open(bhp_json_out, "w", encoding="utf-8") as f:
        json.dump(best_hyperparameters, f, ensure_ascii=False, indent=2)
    print(f"[OK] JSON snapshot (best_hyperparameters) saved to:\n  {bhp_json_out}")
except Exception as e:
    print(f"[WARN] JSON write skipped: {e}")

print("\n[REPORT — BLOCK 12 COMPLETED]")
print(f"  - Selected features (LONG)      : {out_long}")
print(f"  - Selection frequency table     : {out_freq}")
print(f"  - Best HP (LONG)                : {out_hp_long}")
print(f"  - Best HP (WIDE)                : {out_hp_wide}")
print(f"  - JSON snapshot (HP dictionary) : {bhp_json_out}")
print(f"  - Plots folder                  : {PLOT_DIR}")


[OK] Selected-features (LONG) saved to:
  C:\Users\ulku\Desktop\ecommerce\featselect_optuna_final\selected_features_OBJ_auc_ecommerce_churn\selected_features_ecommerce_churn_long.csv
[OK] Selection frequency table saved to:
  C:\Users\ulku\Desktop\ecommerce\featselect_optuna_final\selected_features_OBJ_auc_ecommerce_churn\selection_frequency_ecommerce_churn.csv
[OK] Top-15 feature plot saved for KNN:
  C:\Users\ulku\Desktop\ecommerce\featselect_optuna_final\selected_features_OBJ_auc_ecommerce_churn\plots\top_features_KNN_ecommerce_churn.png
[OK] Top-15 feature plot saved for XGBoost:
  C:\Users\ulku\Desktop\ecommerce\featselect_optuna_final\selected_features_OBJ_auc_ecommerce_churn\plots\top_features_XGBoost_ecommerce_churn.png
[OK] Top-15 feature plot saved for Bagging:
  C:\Users\ulku\Desktop\ecommerce\featselect_optuna_final\selected_features_OBJ_auc_ecommerce_churn\plots\top_features_Bagging_ecommerce_churn.png
[OK] Top-15 feature plot saved for Random Forest:
  C:\Users\ulku\Deskt

In [ ]:
# Dosya kontrolü
try:
    df = pd.read_excel("results/all_model_runs_detailed_MAIN.xlsx")
    print("✅ Dosya bulundu!   ")
    print(f"Sütunlar: {df.columns. tolist()}")
    print(f"\nİlk 3 satır:\n{df[['model', 'rep', 'run_time_min']].head(3)}")
    
    # Özet
    print("\n" + "=" * 60)
    print("Model Bazında Eğitim Süresi Özeti:")
    print(df.groupby("model")["run_time_min"].describe())
    
except FileNotFoundError:
    print("❌ Dosya bulunamadı!   Yolu kontrol edin.")
    print("Şu klasörlerde arayın:")
    print("  - results/")
    print("  - RESULTS/")
    print("  - .")

✅ Dosya bulundu!   
Sütunlar: ['model', 'rep', 'n_trainval', 'n_train', 'n_val', 'n_test', 'auc', 'ap', 'f1', 'accuracy', 'sensitivity', 'specificity', 'mcc', 'gmean', 'run_time_min', 'best_params_raw', 'best_params_clean', 'k_best', 'seed_outer', 'seed_inner', 'seed_tpe', 'seed_model']

İlk 3 satır:
     model  rep  run_time_min
0      KNN    0      0.264685
1  XGBoost    0     17.752953
2  Bagging    0     18.628677

Model Bazında Eğitim Süresi Özeti:
               count       mean       std       min       25%        50%  \
model                                                                      
Bagging         10.0  13.181223  5.956959  3.348025  8.828485  14.801923   
KNN             10.0   0.349745  0.081622  0.226398  0.280134   0.358523   
Random Forest   10.0   7.277297  1.909772  5.078297  5.488219   7.130470   
XGBoost         10.0   7.049964  4.795575  2.432142  3.935287   5.311811   

                     75%        max  
model                                
Bagging  

In [ ]:
# ============================================
# 1) Mevcut dosyayı oku
# ============================================
try:
    df_runs = pd.read_excel("results/all_model_runs_detailed_MAIN.xlsx")
except FileNotFoundError:
    print("❌ Dosya bulunamadı!   Yolu kontrol edin.")
    exit()

# ============================================
# 2) Eğitim süresi özetini çıkar
# ============================================
df_time_summary = df_runs.groupby("model").agg({
    "run_time_min": ["mean", "std", "count"]
}).round(2)

df_time_summary.columns = ["mean_time_min", "std_time_min", "n_runs"]
df_time_summary = df_time_summary.reset_index()

# Saniye cinsine çevir
df_time_summary["mean_time_sec"] = df_time_summary["mean_time_min"] * 60
df_time_summary["std_time_sec"] = df_time_summary["std_time_min"] * 60

# ============ DÜZELTİLMİŞ FORMATLANMIŞ SÜRE ============
def format_time(row):
    """Dakika ve saniyeyi güvenli şekilde formatla"""
    try:
        minutes = int(row["mean_time_min"])
        seconds = int(row["mean_time_sec"] % 60)
        return f"{minutes} dk {seconds} sn"
    except:
        return "N/A"

df_time_summary["formatted_time"] = df_time_summary.apply(format_time, axis=1)
# =======================================================

# Kaydet
df_time_summary.to_csv("results/training_times_summary_from_existing.csv", index=False)

print("=" * 60)
print("EĞİTİM SÜRESİ ÖZETİ")
print("=" * 60)
print(df_time_summary[["model", "mean_time_min", "std_time_min", "formatted_time"]].to_string(index=False))
print("=" * 60)

# ============================================
# 3) Performans metrikleriyle birleştir
# ============================================
try:
    df_metrics = pd.read_excel("results/all_model_test_metrics_MAIN.xlsx")
except FileNotFoundError:
    print("❌ Metrik dosyası bulunamadı!")
    exit()

df_perf_summary = df_metrics. groupby("model").agg({
    "auc": ["mean", "std"],
    "f1": ["mean", "std"],
    "mcc": ["mean", "std"]
}).round(3)

df_perf_summary.columns = ["auc_mean", "auc_std", "f1_mean", "f1_std", "mcc_mean", "mcc_std"]
df_perf_summary = df_perf_summary.reset_index()

# Birleştir
df_final = df_perf_summary. merge(
    df_time_summary[["model", "formatted_time"]], 
    on="model"
)

# ============ DÜZELTİLMİŞ FORMATLA ============
df_final["AUC"] = df_final. apply(
    lambda x: f"{x['auc_mean']:.3f} ± {x['auc_std']:.3f}", axis=1
)
df_final["F1-Score"] = df_final.apply(
    lambda x: f"{x['f1_mean']:.3f} ± {x['f1_std']:.3f}", axis=1
)
df_final["MCC"] = df_final.apply(
    lambda x: f"{x['mcc_mean']:.3f} ± {x['mcc_std']:.3f}", axis=1
)
# ===============================================

# Son tablo
final_table = df_final[["model", "AUC", "F1-Score", "MCC", "formatted_time"]]
final_table.columns = ["Model Adı", "AUC", "F1-Score", "MCC", "Modelin Eğitim Süresi"]

print("\n" + "=" * 80)
print("FİNAL TABLO (Performans + Eğitim Süresi)")
print("=" * 80)
print(final_table.to_string(index=False))

# Kaydet
final_table. to_csv("results/table_performance_with_time_FINAL.csv", index=False)
print("\n✅ Kaydedildi:  results/table_performance_with_time_FINAL.csv")

EĞİTİM SÜRESİ ÖZETİ
        model  mean_time_min  std_time_min formatted_time
      Bagging          13.18          5.96    13 dk 10 sn
          KNN           0.35          0.08     0 dk 21 sn
Random Forest           7.28          1.91     7 dk 16 sn
      XGBoost           7.05          4.80      7 dk 3 sn

FİNAL TABLO (Performans + Eğitim Süresi)
    Model Adı           AUC      F1-Score           MCC Modelin Eğitim Süresi
      Bagging 0.986 ± 0.007 0.830 ± 0.067 0.816 ± 0.062           13 dk 10 sn
          KNN 0.955 ± 0.025 0.783 ± 0.080 0.752 ± 0.088            0 dk 21 sn
Random Forest 0.988 ± 0.006 0.894 ± 0.023 0.873 ± 0.027            7 dk 16 sn
      XGBoost 0.988 ± 0.005 0.915 ± 0.009 0.898 ± 0.011             7 dk 3 sn

✅ Kaydedildi:  results/table_performance_with_time_FINAL.csv
